In [1]:
import requests
from scrapy.http import HtmlResponse

In [2]:
from urllib.parse import quote, unquote, quote_plus

In [3]:
def get_new_token(response):
    return response.headers['Set-Cookie'].split(";")[0].split('=')[1]

In [4]:
class self:
    price_discount = 0.1
    order_id = '7196072'
    application_id = '37069032'
    user_agent = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36'

In [5]:
token = "CmEONAYxUzwAL1AhBmtbagAzV2AEDQYjC3BTYQMlByZVPlFuDTNQA1w%2FAGQHWF0qA2xVJFc6BzECNQ06BA1RdA5kUmQHZ1BkAjhXYVQ1VG8KYg44BjBTMwBmUGcGNVtjAGJXbQQ0BmMLZlM1AzQHY1U0UTkNbVBqXGUANAdhXWkDM1VvVzMHMgJgDTQEZVEwDmRSNwdiUDsCZldjVGZUawoyDmsGYVMzADBQYQZjW20AYFdvBGUGOAtnU2ADYwdnVQtRIw1mUC9cbAAxBzRdMgMLVXVXaQd3Al4NYQQ1UTIOdFJgByFQcQJcVyVUP1QrCjoOOQY%2FU1oAdlBiBn9bawAtV2oEfwYxC19TJgNtByZVbVEwDW1QZlwKACIHcV17AzJVJVdfB2YCZg1lBD9RIg5aUicHaVBxAjpXNlQ0VGwKOg5SBidTSwA7UCgGOFs3AG9XOwR%2BBjQLLVM0A3YHfVUPUTANbFBnXHYATAdtXWYDIlUvVyAHfwI5DTYEDlFgDjJSLAdyUEMCcFd3VGhUPQpXDmsGZ1NNAD1QJgZ%2BW24AMFdtBH8GMgs1UyQDfgceVR9RVQ0QUBBcegAgB2hdYQM8VTJXIAdAAmQNYwQ6UTkOL1IlBxFQagJyV2hUaVQ9Ci8ONwY8UygAZFB8BmVbYwA3V24EfwY0CzJTJAMFBzRVMVFgDS9QNVx5ADUHN10%2FA3lVZFc2B1sCIw07BCJRbA43UjYHaFBeAiJXa1RlVCsKdA5RBmRTZQAgUDsGJ1syAHdXIwQNBiMLOFNtA2wHZFVhUTUNZVBoXGYAOAczXTEDZVVsV30%3D"

In [6]:
cookies = {
    'show_filter_app': 'block',
#     'ci_session': self.get_token_scrapy(response)
#     'ci_session': get_new_token(response)
    'ci_session': token
}

headers = {
    'Connection': 'keep-alive',
    'Cache-Control': 'max-age=0',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': self.user_agent,
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-Dest': 'document',
    'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="99", "Google Chrome";v="99"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Linux"',
    'Referer': f'https://v3bl.goszakup.gov.kz/ru/application/docs/{self.order_id}/{self.application_id}',
    'Accept-Language': 'en-US,en;q=0.9',
}

# request = scrapy.Request(
#     method='GET',
#     url=f'https://v3bl.goszakup.gov.kz/ru/application/priceoffers/{self.order_id}/{self.application_id}', 
#     headers=headers, 
#     cookies=cookies,
#     dont_filter=True,
#     callback=self.prices_3
# )

r = requests.get(
    url=f'https://v3bl.goszakup.gov.kz/ru/application/priceoffers/{self.order_id}/{self.application_id}', 
    headers=headers, 
    cookies=cookies,
    verify=False
)

r

/home/necuk/.local/lib/python3.8/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'v3bl.goszakup.gov.kz'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


<Response [200]>

In [7]:
def gen_lot_xml(order_id, buy_lot_id, buy_lot_point_id, price):
    return (
        f'<?xml version="1.0" encoding="UTF-8"?>'
        f'<priceoffer>'
        f'<trd_buy_id>{order_id}</trd_buy_id>'
        f'<trd_buy_lot_id>{buy_lot_id}</trd_buy_lot_id>'
        f'<trd_buy_lot_point_id>{buy_lot_point_id}</trd_buy_lot_point_id>'
        f'<price>{price}</price>'
        f'</priceoffer>'
    )

In [8]:
# def prices_3(self, response):
response = HtmlResponse(url="", body=r.content)
rows = response.xpath('.//div[@class="panel panel-default"]/table//tr')[1::2]

self.prices = []

for row in rows:
    start_price = row.xpath('./td[6]/text()').get()
    new_price = round(float(start_price.replace(' ', '')) * (1 - self.price_discount), 2)

    record = {
        'price': new_price,
        'buy_lot_id': row.xpath('./td[9]/div/input/@trd_lot_id').get(),
        'buy_lot_point_id': row.xpath('./td[9]/div/input/@buy_lot_point_id').get(),
        'app_lot_id': row.xpath('./td[9]/div/input/@app_lot_id').get()
    }

    record['xml_string'] = gen_lot_xml(self.order_id, record['buy_lot_id'], record['buy_lot_point_id'], record['price'])

    self.prices.append(record)
    
len(rows), len(self.prices)

(3, 3)

In [53]:
self.prices

[{'price': 466087.63,
  'buy_lot_id': '19114250',
  'buy_lot_point_id': '18457975',
  'app_lot_id': '34280398',
  'xml_string': '<?xml version="1.0" encoding="UTF-8"?><priceoffer><trd_buy_id>7196072</trd_buy_id><trd_buy_lot_id>19114250</trd_buy_lot_id><trd_buy_lot_point_id>18457975</trd_buy_lot_point_id><price>466087.63</price></priceoffer>'},
 {'price': 247433.04,
  'buy_lot_id': '19114251',
  'buy_lot_point_id': '18457976',
  'app_lot_id': '34280399',
  'xml_string': '<?xml version="1.0" encoding="UTF-8"?><priceoffer><trd_buy_id>7196072</trd_buy_id><trd_buy_lot_id>19114251</trd_buy_lot_id><trd_buy_lot_point_id>18457976</trd_buy_lot_point_id><price>247433.04</price></priceoffer>'},
 {'price': 416330.36,
  'buy_lot_id': '19114252',
  'buy_lot_point_id': '18457977',
  'app_lot_id': '34280400',
  'xml_string': '<?xml version="1.0" encoding="UTF-8"?><priceoffer><trd_buy_id>7196072</trd_buy_id><trd_buy_lot_id>19114252</trd_buy_lot_id><trd_buy_lot_point_id>18457977</trd_buy_lot_point_id><pr

In [105]:
token = 'XTYKMAQzVzgCLQJzUTwAMQIxADcDClF0AXoFN1F3U3ICaVRrV2kBUlk6Wz9TDAdwVDtXJgZrUmRUYwcwCgNWcwExUTZcP1UzUGdRYwNlU2ZdMwpoBDZXZgJiAjhRMwBlAmQAOANhUWcBbAVkUWdTNQIzVGJXPgE%2BWWNbaVM0BzdUZVczBmBSNlRnB2sKZlZmATxRPlxrVTJQZVFgAzZTZ11jCjIEMVc7Am8CYlFjADECMgA9AzNRbgFsBWNRYlMxAlxUJlc8AX5ZaVtqU2AHaFRcV3cGOFIiVAgHawo7VjUBe1FjXHpVdFAOUSMDaFMsXW0KPQQ9V14CdAIwUSgAMAIvAD0DeFFmAVUFcFE%2FU3ICOlQ1VzcBN1kPW3lTJQchVGVXJwYOUjNUMAdvCjFWJQFVUSRcMlV0UGhRMANjU2tdbQpWBCVXTwI5AnpRbwBsAm0AbAN5UWMBJwViUSRTKQJYVDVXNgE2WXNbF1M5BzxUdVctBnFSKlRvBzwKAFZnAT1RL1wpVUZQIlFxAz9TOl0ACm8EZVdJAj8CdFEpADUCMgA6A3hRZQE%2FBXJRLFNKAkhUUFdKAUFZf1t7UzwHO1RrVzAGcVIVVDIHaQo0Vj4BIFEmXEpVb1AgUW4DPlM6XXgKMwQ%2BVywCZgIuUTIAOAI1ADkDeFFjATgFclFXU2ACZlRlV3UBZFl8W25TYwdlVC5XZgZnUg5UdQcxCixWawE4UTVcM1VbUHBRbQMyUyxdIwpVBGZXYQIiAmlRcABpAnUAdAMKUXQBMgU7UT5TMAI2VDBXPwE%2FWWJbY1NkB2NUOFduBiw%3D'

In [9]:
cookies = {
    'show_filter_app': 'block',
#         'ci_session': self.get_token_scrapy(response)
    'ci_session': get_new_token(r)
#     'ci_session': token
}

headers = {
    'Connection': 'keep-alive',
    'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="99", "Google Chrome";v="99"',
    'Accept': 'application/json, text/javascript, */*; q=0.01',
    'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
    'X-Requested-With': 'XMLHttpRequest',
    'sec-ch-ua-mobile': '?0',
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36',
    'sec-ch-ua-platform': '"Linux"',
    'Origin': 'https://v3bl.goszakup.gov.kz',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Dest': 'empty',
    'Referer': f'https://v3bl.goszakup.gov.kz/ru/application/priceoffers/{self.order_id}/{self.application_id}',
    'Accept-Language': 'en-US,en;q=0.9',
}

# data = {
#     'offer[34246483][18457975][price]': '466087.62',
#     'offer[34246484][18457976][price]': '247433.03',
#     'offer[34246485][18457977][price]': '416330.36',
# }

data = {
    f"offer[{record['app_lot_id']}][{record['buy_lot_point_id']}][price]": record['price']
    for record in self.prices
}

r2 = requests.post(
    url=f'https://v3bl.goszakup.gov.kz/ru/application/ajax_check_priceoffers/{self.order_id}/{self.application_id}', 
    headers=headers, 
    cookies=cookies, 
    data=data,
    verify=False
)

r2

#     request = scrapy.FormRequest(
#         method='POST',
#         url=f'https://v3bl.goszakup.gov.kz/ru/application/ajax_check_priceoffers/{self.order_id}/{self.application_id}',
#         cookies=cookies, 
#         data=data,
#         dont_filter=True,
#         callback=self.prices_4
#     )

#     yield request

/home/necuk/.local/lib/python3.8/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'v3bl.goszakup.gov.kz'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


<Response [200]>

In [10]:
r2.text

'{"status":1}'

In [11]:
self.prices

[{'price': 466087.63,
  'buy_lot_id': '19114250',
  'buy_lot_point_id': '18457975',
  'app_lot_id': '34280541',
  'xml_string': '<?xml version="1.0" encoding="UTF-8"?><priceoffer><trd_buy_id>7196072</trd_buy_id><trd_buy_lot_id>19114250</trd_buy_lot_id><trd_buy_lot_point_id>18457975</trd_buy_lot_point_id><price>466087.63</price></priceoffer>'},
 {'price': 247433.04,
  'buy_lot_id': '19114251',
  'buy_lot_point_id': '18457976',
  'app_lot_id': '34280542',
  'xml_string': '<?xml version="1.0" encoding="UTF-8"?><priceoffer><trd_buy_id>7196072</trd_buy_id><trd_buy_lot_id>19114251</trd_buy_lot_id><trd_buy_lot_point_id>18457976</trd_buy_lot_point_id><price>247433.04</price></priceoffer>'},
 {'price': 416330.36,
  'buy_lot_id': '19114252',
  'buy_lot_point_id': '18457977',
  'app_lot_id': '34280543',
  'xml_string': '<?xml version="1.0" encoding="UTF-8"?><priceoffer><trd_buy_id>7196072</trd_buy_id><trd_buy_lot_id>19114252</trd_buy_lot_id><trd_buy_lot_point_id>18457977</trd_buy_lot_point_id><pr

In [22]:
response = HtmlResponse(url="", body=r.content)

In [110]:
rows = response.xpath('.//div[@class="panel-body"]/table[@id="affil_queue"]/tbody/tr')

In [111]:
done = all(row.xpath('./td[4]/text()').get() for row in rows)
done

True

In [23]:
token = get_new_token(r2)

In [45]:
token = 'WjFYYltsVzgOIVUkB2oENQY1UmUDClJ3CHNWZFx6Xn8FbgU6BjhUBwBjD2tQD1cgAG9SIwZrATdWYQE2V14BJA01UzdVZQBnVzNYbFRmBjtaZFhuW2lXZA5vVW8HYAQ8BmRSPANkUmIIZVY1XDBePQVjBTMGYFRsADIPPVA1V2QANVJgBmIBM1ZtAWpXZAE3DTdTPFVmADdXbVhuVDUGa1oyWGpbOlczDmhVZgdiBDwGMlJoA2BSZQg5VmRcPF5uBVsFdwZtVCsAMA8%2BUGNXOAAIUnIGOAFxVgoBbVdmAWINd1NhVXMAIVcJWCpUPwZ5WmpYb1tiV14OeFVnB34ENAYrUm8DeFJlCFxWI1wyXn8FPQVkBmZUYgBWDy1QJldxADFSIgYOAWBWMgFpV2wBcg1ZUyZVOwAhV29YOVQ0Bj5aalgEW3pXTw41VS0HOQRoBmlSPgN5UmAILlYxXCleJAVfBWQGZ1RjACoPQ1A6V2wAIVIoBnEBeVZtATpXXQEwDTFTLVUgABNXJVh4VGgGb1oHWD1bOldJDjNVIwd%2FBDEGNlJoA3hSZgg2ViFcIV5HBU8FAQYbVBQAJg8vUD9XawA%2FUjUGcQFGVjABb1dpAWkNLFMkVUMAOlcnWGdUaQZvWn9YYVthVywOalV5B2QEPAYxUmsDeFJgCDFWIVxaXm0FYQU0BiRUMQAlDzpQYFc1AHpSYwZnAV1WdwE3V3EBPA00UzdVOgAOV3dYZFRlBnlaJFgHWzlXYQ4uVT4HJgRtBnFSJgMKUncIO1ZoXDNePQUxBWEGblRqAD4PPlBiVzMAY1JrBiw%3D'

In [58]:
my_data='xmlData%5B18457975%5D=%3C%3Fxml+version%3D%221.0%22+encoding%3D%22UTF-8%22%3F%3E%3Cpriceoffer%3E%3Ctrd_buy_id%3E7196072%3C%2Ftrd_buy_id%3E%3Ctrd_buy_lot_id%3E19114250%3C%2Ftrd_buy_lot_id%3E%3Ctrd_buy_lot_point_id%3E18457975%3C%2Ftrd_buy_lot_point_id%3E%3Cprice%3E466087.63%3C%2Fprice%3E%3C%2Fpriceoffer%3E&xmlData%5B18457976%5D=%3C%3Fxml+version%3D%221.0%22+encoding%3D%22UTF-8%22%3F%3E%3Cpriceoffer%3E%3Ctrd_buy_id%3E7196072%3C%2Ftrd_buy_id%3E%3Ctrd_buy_lot_id%3E19114251%3C%2Ftrd_buy_lot_id%3E%3Ctrd_buy_lot_point_id%3E18457976%3C%2Ftrd_buy_lot_point_id%3E%3Cprice%3E247433.04%3C%2Fprice%3E%3C%2Fpriceoffer%3E&xmlData%5B18457977%5D=%3C%3Fxml+version%3D%221.0%22+encoding%3D%22UTF-8%22%3F%3E%3Cpriceoffer%3E%3Ctrd_buy_id%3E7196072%3C%2Ftrd_buy_id%3E%3Ctrd_buy_lot_id%3E19114252%3C%2Ftrd_buy_lot_id%3E%3Ctrd_buy_lot_point_id%3E18457977%3C%2Ftrd_buy_lot_point_id%3E%3Cprice%3E416330.36%3C%2Fprice%3E%3C%2Fpriceoffer%3E&signData%5B18457975%5D=MIIHLwYJKoZIhvcNAQcCoIIHIDCCBxwCAQExDjAMBggqgw4DCgEDAQUAMIHmBgkqhkiG9w0BBwGggdgEgdU8P3htbCB2ZXJzaW9uPSIxLjAiIGVuY29kaW5nPSJVVEYtOCI%2FPjxwcmljZW9mZmVyPjx0cmRfYnV5X2lkPjcxOTYwNzI8L3RyZF9idXlfaWQ%2BPHRyZF9idXlfbG90X2lkPjE5MTE0MjUwPC90cmRfYnV5X2xvdF9pZD48dHJkX2J1eV9sb3RfcG9pbnRfaWQ%2BMTg0NTc5NzU8L3RyZF9idXlfbG90X3BvaW50X2lkPjxwcmljZT40NjYwODcuNjM8L3ByaWNlPjwvcHJpY2VvZmZlcj6gggTaMIIE1jCCBICgAwIBAgIULBFucGgIEwDyHT8jjFMCo2xU7H4wDQYJKoMOAwoBAQECBQAwUzELMAkGA1UEBhMCS1oxRDBCBgNVBAMMO9Kw0JvQotCi0KvSmiDQmtCj05jQm9CQ0J3QlNCr0KDQo9Co0Ksg0J7QoNCi0JDQm9Cr0pogKEdPU1QpMB4XDTIxMDYwNDA3NTE1NVoXDTIyMDYwNDA3NTE1NVowggF1MSowKAYDVQQDDCHQmtCQ0JnQk9Ce0KDQntCU0KbQldCS0JAg0JzQkNCZ0K8xITAfBgNVBAQMGNCa0JDQmdCT0J7QoNCe0JTQptCV0JLQkDEYMBYGA1UEBRMPSUlOODMwNTA0NDAwMDQ5MQswCQYDVQQGEwJLWjGBnDCBmQYDVQQKDIGR0KLQntCS0JDQoNCY0KnQldCh0KLQktCeINChINCe0JPQoNCQ0J3QmNCn0JXQndCd0J7QmSDQntCi0JLQldCi0KHQotCS0JXQndCd0J7QodCi0KzQriAi0JjQndCW0JjQndCY0KDQmNCd0JPQntCS0JDQryDQmtCe0JzQn9CQ0J3QmNCvICLQm9CY0JTQldCgIjEYMBYGA1UECwwPQklOMTEwOTQwMDA0OTA5MSMwIQYDVQQqDBrQodCi0JDQndCY0KHQm9CQ0JLQntCS0J3QkDEfMB0GCSqGSIb3DQEJARYQQ0VPQExFQURFUkVORy5LWjBsMCUGCSqDDgMKAQEBATAYBgoqgw4DCgEBAQEBBgoqgw4DCgEDAQEAA0MABEDk36lRIduJHXYf2mZSoaY46q06pz8fTkInD91Y8hQSbNfdw6zp7FB1Qp6pIal0sIrIl70JpgQOjxzDYNIV%2Fmkno4IB9jCCAfIwDgYDVR0PAQH%2FBAQDAgbAMDMGA1UdJQQsMCoGCCsGAQUFBwMEBggqgw4DAwQBAgYJKoMOAwMEAQIBBgkqgw4DAwQDAgEwDwYDVR0jBAgwBoAEW2pz6TAdBgNVHQ4EFgQU3hE7WqNKTTjuiDbD4q4uPI4j0GwwXgYDVR0gBFcwVTBTBgcqgw4DAwIBMEgwIQYIKwYBBQUHAgEWFWh0dHA6Ly9wa2kuZ292Lmt6L2NwczAjBggrBgEFBQcCAjAXDBVodHRwOi8vcGtpLmdvdi5rei9jcHMwWAYDVR0fBFEwTzBNoEugSYYiaHR0cDovL2NybC5wa2kuZ292Lmt6L25jYV9nb3N0LmNybIYjaHR0cDovL2NybDEucGtpLmdvdi5rei9uY2FfZ29zdC5jcmwwXAYDVR0uBFUwUzBRoE%2BgTYYkaHR0cDovL2NybC5wa2kuZ292Lmt6L25jYV9kX2dvc3QuY3JshiVodHRwOi8vY3JsMS5wa2kuZ292Lmt6L25jYV9kX2dvc3QuY3JsMGMGCCsGAQUFBwEBBFcwVTAvBggrBgEFBQcwAoYjaHR0cDovL3BraS5nb3Yua3ovY2VydC9uY2FfZ29zdC5jZXIwIgYIKwYBBQUHMAGGFmh0dHA6Ly9vY3NwLnBraS5nb3Yua3owDQYJKoMOAwoBAQECBQADQQBep8zG1qucVUVmucAEmFw6%2FUrSaOzZ1UErJMiq8IXZzQ6EW7WbjaD4jOZ7eBZjgkuKqecCxgiQA0g4E%2BZP9MgxMYIBPjCCAToCAQEwazBTMQswCQYDVQQGEwJLWjFEMEIGA1UEAww70rDQm9Ci0KLQq9KaINCa0KPTmNCb0JDQndCU0KvQoNCj0KjQqyDQntCg0KLQkNCb0KvSmiAoR09TVCkCFCwRbnBoCBMA8h0%2FI4xTAqNsVOx%2BMAwGCCqDDgMKAQMBBQCgaTAYBgkqhkiG9w0BCQMxCwYJKoZIhvcNAQcBMBwGCSqGSIb3DQEJBTEPFw0yMjAzMjUyMDU4MThaMC8GCSqGSIb3DQEJBDEiBCCget%2F6ejnka%2FT1LA8tCDYu6NX8fAFE%2F24TskhkHKif8jANBgkqgw4DCgEBAQIFAARAzsmvV5HFwE0Z%2FO1UwsJlt8uI7yDANa8boHSNFq67Yf9EJUVyVkG1aKVnuq8ewD%2FdJRdi4Tux3FO7%2Fy34NuD%2FXQ%3D%3D&signData%5B18457976%5D=MIIHLwYJKoZIhvcNAQcCoIIHIDCCBxwCAQExDjAMBggqgw4DCgEDAQUAMIHmBgkqhkiG9w0BBwGggdgEgdU8P3htbCB2ZXJzaW9uPSIxLjAiIGVuY29kaW5nPSJVVEYtOCI%2FPjxwcmljZW9mZmVyPjx0cmRfYnV5X2lkPjcxOTYwNzI8L3RyZF9idXlfaWQ%2BPHRyZF9idXlfbG90X2lkPjE5MTE0MjUxPC90cmRfYnV5X2xvdF9pZD48dHJkX2J1eV9sb3RfcG9pbnRfaWQ%2BMTg0NTc5NzY8L3RyZF9idXlfbG90X3BvaW50X2lkPjxwcmljZT4yNDc0MzMuMDQ8L3ByaWNlPjwvcHJpY2VvZmZlcj6gggTaMIIE1jCCBICgAwIBAgIULBFucGgIEwDyHT8jjFMCo2xU7H4wDQYJKoMOAwoBAQECBQAwUzELMAkGA1UEBhMCS1oxRDBCBgNVBAMMO9Kw0JvQotCi0KvSmiDQmtCj05jQm9CQ0J3QlNCr0KDQo9Co0Ksg0J7QoNCi0JDQm9Cr0pogKEdPU1QpMB4XDTIxMDYwNDA3NTE1NVoXDTIyMDYwNDA3NTE1NVowggF1MSowKAYDVQQDDCHQmtCQ0JnQk9Ce0KDQntCU0KbQldCS0JAg0JzQkNCZ0K8xITAfBgNVBAQMGNCa0JDQmdCT0J7QoNCe0JTQptCV0JLQkDEYMBYGA1UEBRMPSUlOODMwNTA0NDAwMDQ5MQswCQYDVQQGEwJLWjGBnDCBmQYDVQQKDIGR0KLQntCS0JDQoNCY0KnQldCh0KLQktCeINChINCe0JPQoNCQ0J3QmNCn0JXQndCd0J7QmSDQntCi0JLQldCi0KHQotCS0JXQndCd0J7QodCi0KzQriAi0JjQndCW0JjQndCY0KDQmNCd0JPQntCS0JDQryDQmtCe0JzQn9CQ0J3QmNCvICLQm9CY0JTQldCgIjEYMBYGA1UECwwPQklOMTEwOTQwMDA0OTA5MSMwIQYDVQQqDBrQodCi0JDQndCY0KHQm9CQ0JLQntCS0J3QkDEfMB0GCSqGSIb3DQEJARYQQ0VPQExFQURFUkVORy5LWjBsMCUGCSqDDgMKAQEBATAYBgoqgw4DCgEBAQEBBgoqgw4DCgEDAQEAA0MABEDk36lRIduJHXYf2mZSoaY46q06pz8fTkInD91Y8hQSbNfdw6zp7FB1Qp6pIal0sIrIl70JpgQOjxzDYNIV%2Fmkno4IB9jCCAfIwDgYDVR0PAQH%2FBAQDAgbAMDMGA1UdJQQsMCoGCCsGAQUFBwMEBggqgw4DAwQBAgYJKoMOAwMEAQIBBgkqgw4DAwQDAgEwDwYDVR0jBAgwBoAEW2pz6TAdBgNVHQ4EFgQU3hE7WqNKTTjuiDbD4q4uPI4j0GwwXgYDVR0gBFcwVTBTBgcqgw4DAwIBMEgwIQYIKwYBBQUHAgEWFWh0dHA6Ly9wa2kuZ292Lmt6L2NwczAjBggrBgEFBQcCAjAXDBVodHRwOi8vcGtpLmdvdi5rei9jcHMwWAYDVR0fBFEwTzBNoEugSYYiaHR0cDovL2NybC5wa2kuZ292Lmt6L25jYV9nb3N0LmNybIYjaHR0cDovL2NybDEucGtpLmdvdi5rei9uY2FfZ29zdC5jcmwwXAYDVR0uBFUwUzBRoE%2BgTYYkaHR0cDovL2NybC5wa2kuZ292Lmt6L25jYV9kX2dvc3QuY3JshiVodHRwOi8vY3JsMS5wa2kuZ292Lmt6L25jYV9kX2dvc3QuY3JsMGMGCCsGAQUFBwEBBFcwVTAvBggrBgEFBQcwAoYjaHR0cDovL3BraS5nb3Yua3ovY2VydC9uY2FfZ29zdC5jZXIwIgYIKwYBBQUHMAGGFmh0dHA6Ly9vY3NwLnBraS5nb3Yua3owDQYJKoMOAwoBAQECBQADQQBep8zG1qucVUVmucAEmFw6%2FUrSaOzZ1UErJMiq8IXZzQ6EW7WbjaD4jOZ7eBZjgkuKqecCxgiQA0g4E%2BZP9MgxMYIBPjCCAToCAQEwazBTMQswCQYDVQQGEwJLWjFEMEIGA1UEAww70rDQm9Ci0KLQq9KaINCa0KPTmNCb0JDQndCU0KvQoNCj0KjQqyDQntCg0KLQkNCb0KvSmiAoR09TVCkCFCwRbnBoCBMA8h0%2FI4xTAqNsVOx%2BMAwGCCqDDgMKAQMBBQCgaTAYBgkqhkiG9w0BCQMxCwYJKoZIhvcNAQcBMBwGCSqGSIb3DQEJBTEPFw0yMjAzMjUyMDU4MThaMC8GCSqGSIb3DQEJBDEiBCDwlGwLUvL5B5NVdcwyFZhAA7jPk7A5tu7DmEHGKy5NBDANBgkqgw4DCgEBAQIFAARAR0j07qRxvf120T%2F1cNT4Tr46xdKVkCoV%2FVetsIqwByDaJPMZnUfKoms1nfwzsOueQnrTyO4m0XWeuC%2FlBmoVVw%3D%3D&signData%5B18457977%5D=MIIHLwYJKoZIhvcNAQcCoIIHIDCCBxwCAQExDjAMBggqgw4DCgEDAQUAMIHmBgkqhkiG9w0BBwGggdgEgdU8P3htbCB2ZXJzaW9uPSIxLjAiIGVuY29kaW5nPSJVVEYtOCI%2FPjxwcmljZW9mZmVyPjx0cmRfYnV5X2lkPjcxOTYwNzI8L3RyZF9idXlfaWQ%2BPHRyZF9idXlfbG90X2lkPjE5MTE0MjUyPC90cmRfYnV5X2xvdF9pZD48dHJkX2J1eV9sb3RfcG9pbnRfaWQ%2BMTg0NTc5Nzc8L3RyZF9idXlfbG90X3BvaW50X2lkPjxwcmljZT40MTYzMzAuMzY8L3ByaWNlPjwvcHJpY2VvZmZlcj6gggTaMIIE1jCCBICgAwIBAgIULBFucGgIEwDyHT8jjFMCo2xU7H4wDQYJKoMOAwoBAQECBQAwUzELMAkGA1UEBhMCS1oxRDBCBgNVBAMMO9Kw0JvQotCi0KvSmiDQmtCj05jQm9CQ0J3QlNCr0KDQo9Co0Ksg0J7QoNCi0JDQm9Cr0pogKEdPU1QpMB4XDTIxMDYwNDA3NTE1NVoXDTIyMDYwNDA3NTE1NVowggF1MSowKAYDVQQDDCHQmtCQ0JnQk9Ce0KDQntCU0KbQldCS0JAg0JzQkNCZ0K8xITAfBgNVBAQMGNCa0JDQmdCT0J7QoNCe0JTQptCV0JLQkDEYMBYGA1UEBRMPSUlOODMwNTA0NDAwMDQ5MQswCQYDVQQGEwJLWjGBnDCBmQYDVQQKDIGR0KLQntCS0JDQoNCY0KnQldCh0KLQktCeINChINCe0JPQoNCQ0J3QmNCn0JXQndCd0J7QmSDQntCi0JLQldCi0KHQotCS0JXQndCd0J7QodCi0KzQriAi0JjQndCW0JjQndCY0KDQmNCd0JPQntCS0JDQryDQmtCe0JzQn9CQ0J3QmNCvICLQm9CY0JTQldCgIjEYMBYGA1UECwwPQklOMTEwOTQwMDA0OTA5MSMwIQYDVQQqDBrQodCi0JDQndCY0KHQm9CQ0JLQntCS0J3QkDEfMB0GCSqGSIb3DQEJARYQQ0VPQExFQURFUkVORy5LWjBsMCUGCSqDDgMKAQEBATAYBgoqgw4DCgEBAQEBBgoqgw4DCgEDAQEAA0MABEDk36lRIduJHXYf2mZSoaY46q06pz8fTkInD91Y8hQSbNfdw6zp7FB1Qp6pIal0sIrIl70JpgQOjxzDYNIV%2Fmkno4IB9jCCAfIwDgYDVR0PAQH%2FBAQDAgbAMDMGA1UdJQQsMCoGCCsGAQUFBwMEBggqgw4DAwQBAgYJKoMOAwMEAQIBBgkqgw4DAwQDAgEwDwYDVR0jBAgwBoAEW2pz6TAdBgNVHQ4EFgQU3hE7WqNKTTjuiDbD4q4uPI4j0GwwXgYDVR0gBFcwVTBTBgcqgw4DAwIBMEgwIQYIKwYBBQUHAgEWFWh0dHA6Ly9wa2kuZ292Lmt6L2NwczAjBggrBgEFBQcCAjAXDBVodHRwOi8vcGtpLmdvdi5rei9jcHMwWAYDVR0fBFEwTzBNoEugSYYiaHR0cDovL2NybC5wa2kuZ292Lmt6L25jYV9nb3N0LmNybIYjaHR0cDovL2NybDEucGtpLmdvdi5rei9uY2FfZ29zdC5jcmwwXAYDVR0uBFUwUzBRoE%2BgTYYkaHR0cDovL2NybC5wa2kuZ292Lmt6L25jYV9kX2dvc3QuY3JshiVodHRwOi8vY3JsMS5wa2kuZ292Lmt6L25jYV9kX2dvc3QuY3JsMGMGCCsGAQUFBwEBBFcwVTAvBggrBgEFBQcwAoYjaHR0cDovL3BraS5nb3Yua3ovY2VydC9uY2FfZ29zdC5jZXIwIgYIKwYBBQUHMAGGFmh0dHA6Ly9vY3NwLnBraS5nb3Yua3owDQYJKoMOAwoBAQECBQADQQBep8zG1qucVUVmucAEmFw6%2FUrSaOzZ1UErJMiq8IXZzQ6EW7WbjaD4jOZ7eBZjgkuKqecCxgiQA0g4E%2BZP9MgxMYIBPjCCAToCAQEwazBTMQswCQYDVQQGEwJLWjFEMEIGA1UEAww70rDQm9Ci0KLQq9KaINCa0KPTmNCb0JDQndCU0KvQoNCj0KjQqyDQntCg0KLQkNCb0KvSmiAoR09TVCkCFCwRbnBoCBMA8h0%2FI4xTAqNsVOx%2BMAwGCCqDDgMKAQMBBQCgaTAYBgkqhkiG9w0BCQMxCwYJKoZIhvcNAQcBMBwGCSqGSIb3DQEJBTEPFw0yMjAzMjUyMDU4MTlaMC8GCSqGSIb3DQEJBDEiBCCtrT1zL9sc9G87JyEWAvokjqmXkbIubvdKXNBuxuRGYzANBgkqgw4DCgEBAQIFAARANQM3DJSc5zfM17ySehOkbkiw5WbWNevnSeKPKAbKp1V1NdW68OGFeajJ%2FMGzMiInTZrXzePbqSWVdzIbuSgeFA%3D%3D&formData=offer%5B34280398%5D%5B18457975%5D%5Bprice%5D=466087.63&offer%5B34280399%5D%5B18457976%5D%5Bprice%5D=247433.04&offer%5B34280400%5D%5B18457977%5D%5Bprice%5D=416330.36'

In [62]:

my_data = 'xmlData%5B18457977%5D=%3C%3Fxml+version%3D%221.0%22+encoding%3D%22UTF-8%22%3F%3E%3Cpriceoffer%3E%3Ctrd_buy_id%3E7196072%3C%2Ftrd_buy_id%3E%3Ctrd_buy_lot_id%3E19114252%3C%2Ftrd_buy_lot_id%3E%3Ctrd_buy_lot_point_id%3E18457977%3C%2Ftrd_buy_lot_point_id%3E%3Cprice%3E416330.36%3C%2Fprice%3E%3C%2Fpriceoffer%3E&signData%5B18457977%5D=MIIHLwYJKoZIhvcNAQcCoIIHIDCCBxwCAQExDjAMBggqgw4DCgEDAQUAMIHmBgkqhkiG9w0BBwGggdgEgdU8P3htbCB2ZXJzaW9uPSIxLjAiIGVuY29kaW5nPSJVVEYtOCI%2FPjxwcmljZW9mZmVyPjx0cmRfYnV5X2lkPjcxOTYwNzI8L3RyZF9idXlfaWQ%2BPHRyZF9idXlfbG90X2lkPjE5MTE0MjUyPC90cmRfYnV5X2xvdF9pZD48dHJkX2J1eV9sb3RfcG9pbnRfaWQ%2BMTg0NTc5Nzc8L3RyZF9idXlfbG90X3BvaW50X2lkPjxwcmljZT40MTYzMzAuMzY8L3ByaWNlPjwvcHJpY2VvZmZlcj6gggTaMIIE1jCCBICgAwIBAgIULBFucGgIEwDyHT8jjFMCo2xU7H4wDQYJKoMOAwoBAQECBQAwUzELMAkGA1UEBhMCS1oxRDBCBgNVBAMMO9Kw0JvQotCi0KvSmiDQmtCj05jQm9CQ0J3QlNCr0KDQo9Co0Ksg0J7QoNCi0JDQm9Cr0pogKEdPU1QpMB4XDTIxMDYwNDA3NTE1NVoXDTIyMDYwNDA3NTE1NVowggF1MSowKAYDVQQDDCHQmtCQ0JnQk9Ce0KDQntCU0KbQldCS0JAg0JzQkNCZ0K8xITAfBgNVBAQMGNCa0JDQmdCT0J7QoNCe0JTQptCV0JLQkDEYMBYGA1UEBRMPSUlOODMwNTA0NDAwMDQ5MQswCQYDVQQGEwJLWjGBnDCBmQYDVQQKDIGR0KLQntCS0JDQoNCY0KnQldCh0KLQktCeINChINCe0JPQoNCQ0J3QmNCn0JXQndCd0J7QmSDQntCi0JLQldCi0KHQotCS0JXQndCd0J7QodCi0KzQriAi0JjQndCW0JjQndCY0KDQmNCd0JPQntCS0JDQryDQmtCe0JzQn9CQ0J3QmNCvICLQm9CY0JTQldCgIjEYMBYGA1UECwwPQklOMTEwOTQwMDA0OTA5MSMwIQYDVQQqDBrQodCi0JDQndCY0KHQm9CQ0JLQntCS0J3QkDEfMB0GCSqGSIb3DQEJARYQQ0VPQExFQURFUkVORy5LWjBsMCUGCSqDDgMKAQEBATAYBgoqgw4DCgEBAQEBBgoqgw4DCgEDAQEAA0MABEDk36lRIduJHXYf2mZSoaY46q06pz8fTkInD91Y8hQSbNfdw6zp7FB1Qp6pIal0sIrIl70JpgQOjxzDYNIV%2Fmkno4IB9jCCAfIwDgYDVR0PAQH%2FBAQDAgbAMDMGA1UdJQQsMCoGCCsGAQUFBwMEBggqgw4DAwQBAgYJKoMOAwMEAQIBBgkqgw4DAwQDAgEwDwYDVR0jBAgwBoAEW2pz6TAdBgNVHQ4EFgQU3hE7WqNKTTjuiDbD4q4uPI4j0GwwXgYDVR0gBFcwVTBTBgcqgw4DAwIBMEgwIQYIKwYBBQUHAgEWFWh0dHA6Ly9wa2kuZ292Lmt6L2NwczAjBggrBgEFBQcCAjAXDBVodHRwOi8vcGtpLmdvdi5rei9jcHMwWAYDVR0fBFEwTzBNoEugSYYiaHR0cDovL2NybC5wa2kuZ292Lmt6L25jYV9nb3N0LmNybIYjaHR0cDovL2NybDEucGtpLmdvdi5rei9uY2FfZ29zdC5jcmwwXAYDVR0uBFUwUzBRoE%2BgTYYkaHR0cDovL2NybC5wa2kuZ292Lmt6L25jYV9kX2dvc3QuY3JshiVodHRwOi8vY3JsMS5wa2kuZ292Lmt6L25jYV9kX2dvc3QuY3JsMGMGCCsGAQUFBwEBBFcwVTAvBggrBgEFBQcwAoYjaHR0cDovL3BraS5nb3Yua3ovY2VydC9uY2FfZ29zdC5jZXIwIgYIKwYBBQUHMAGGFmh0dHA6Ly9vY3NwLnBraS5nb3Yua3owDQYJKoMOAwoBAQECBQADQQBep8zG1qucVUVmucAEmFw6%2FUrSaOzZ1UErJMiq8IXZzQ6EW7WbjaD4jOZ7eBZjgkuKqecCxgiQA0g4E%2BZP9MgxMYIBPjCCAToCAQEwazBTMQswCQYDVQQGEwJLWjFEMEIGA1UEAww70rDQm9Ci0KLQq9KaINCa0KPTmNCb0JDQndCU0KvQoNCj0KjQqyDQntCg0KLQkNCb0KvSmiAoR09TVCkCFCwRbnBoCBMA8h0%2FI4xTAqNsVOx%2BMAwGCCqDDgMKAQMBBQCgaTAYBgkqhkiG9w0BCQMxCwYJKoZIhvcNAQcBMBwGCSqGSIb3DQEJBTEPFw0yMjAzMjUyMTAyMzlaMC8GCSqGSIb3DQEJBDEiBCCtrT1zL9sc9G87JyEWAvokjqmXkbIubvdKXNBuxuRGYzANBgkqgw4DCgEBAQIFAARAC6U9PaDWdgu77q%2Bl9z773shSxaYoKM8V0ypXcYyrDbh0khLOsTue4BGILbAKyKjM0zfWaGyzL%2Fv2swPTSt2Jjw%3D%3D&formData=offer%5B34280400%5D%5B18457977%5D%5Bprice%5D=416330.36'

In [12]:
# 3 prices but only 1st gets accepted
my_data = 'xmlData%5B18457975%5D=%3C%3Fxml+version%3D%221.0%22+encoding%3D%22UTF-8%22%3F%3E%3Cpriceoffer%3E%3Ctrd_buy_id%3E7196072%3C%2Ftrd_buy_id%3E%3Ctrd_buy_lot_id%3E19114250%3C%2Ftrd_buy_lot_id%3E%3Ctrd_buy_lot_point_id%3E18457975%3C%2Ftrd_buy_lot_point_id%3E%3Cprice%3E466087.63%3C%2Fprice%3E%3C%2Fpriceoffer%3E&xmlData%5B18457976%5D=%3C%3Fxml+version%3D%221.0%22+encoding%3D%22UTF-8%22%3F%3E%3Cpriceoffer%3E%3Ctrd_buy_id%3E7196072%3C%2Ftrd_buy_id%3E%3Ctrd_buy_lot_id%3E19114251%3C%2Ftrd_buy_lot_id%3E%3Ctrd_buy_lot_point_id%3E18457976%3C%2Ftrd_buy_lot_point_id%3E%3Cprice%3E247433.04%3C%2Fprice%3E%3C%2Fpriceoffer%3E&xmlData%5B18457977%5D=%3C%3Fxml+version%3D%221.0%22+encoding%3D%22UTF-8%22%3F%3E%3Cpriceoffer%3E%3Ctrd_buy_id%3E7196072%3C%2Ftrd_buy_id%3E%3Ctrd_buy_lot_id%3E19114252%3C%2Ftrd_buy_lot_id%3E%3Ctrd_buy_lot_point_id%3E18457977%3C%2Ftrd_buy_lot_point_id%3E%3Cprice%3E416330.36%3C%2Fprice%3E%3C%2Fpriceoffer%3E&signData%5B18457975%5D=MIIHLwYJKoZIhvcNAQcCoIIHIDCCBxwCAQExDjAMBggqgw4DCgEDAQUAMIHmBgkqhkiG9w0BBwGggdgEgdU8P3htbCB2ZXJzaW9uPSIxLjAiIGVuY29kaW5nPSJVVEYtOCI%2FPjxwcmljZW9mZmVyPjx0cmRfYnV5X2lkPjcxOTYwNzI8L3RyZF9idXlfaWQ%2BPHRyZF9idXlfbG90X2lkPjE5MTE0MjUwPC90cmRfYnV5X2xvdF9pZD48dHJkX2J1eV9sb3RfcG9pbnRfaWQ%2BMTg0NTc5NzU8L3RyZF9idXlfbG90X3BvaW50X2lkPjxwcmljZT40NjYwODcuNjM8L3ByaWNlPjwvcHJpY2VvZmZlcj6gggTaMIIE1jCCBICgAwIBAgIULBFucGgIEwDyHT8jjFMCo2xU7H4wDQYJKoMOAwoBAQECBQAwUzELMAkGA1UEBhMCS1oxRDBCBgNVBAMMO9Kw0JvQotCi0KvSmiDQmtCj05jQm9CQ0J3QlNCr0KDQo9Co0Ksg0J7QoNCi0JDQm9Cr0pogKEdPU1QpMB4XDTIxMDYwNDA3NTE1NVoXDTIyMDYwNDA3NTE1NVowggF1MSowKAYDVQQDDCHQmtCQ0JnQk9Ce0KDQntCU0KbQldCS0JAg0JzQkNCZ0K8xITAfBgNVBAQMGNCa0JDQmdCT0J7QoNCe0JTQptCV0JLQkDEYMBYGA1UEBRMPSUlOODMwNTA0NDAwMDQ5MQswCQYDVQQGEwJLWjGBnDCBmQYDVQQKDIGR0KLQntCS0JDQoNCY0KnQldCh0KLQktCeINChINCe0JPQoNCQ0J3QmNCn0JXQndCd0J7QmSDQntCi0JLQldCi0KHQotCS0JXQndCd0J7QodCi0KzQriAi0JjQndCW0JjQndCY0KDQmNCd0JPQntCS0JDQryDQmtCe0JzQn9CQ0J3QmNCvICLQm9CY0JTQldCgIjEYMBYGA1UECwwPQklOMTEwOTQwMDA0OTA5MSMwIQYDVQQqDBrQodCi0JDQndCY0KHQm9CQ0JLQntCS0J3QkDEfMB0GCSqGSIb3DQEJARYQQ0VPQExFQURFUkVORy5LWjBsMCUGCSqDDgMKAQEBATAYBgoqgw4DCgEBAQEBBgoqgw4DCgEDAQEAA0MABEDk36lRIduJHXYf2mZSoaY46q06pz8fTkInD91Y8hQSbNfdw6zp7FB1Qp6pIal0sIrIl70JpgQOjxzDYNIV%2Fmkno4IB9jCCAfIwDgYDVR0PAQH%2FBAQDAgbAMDMGA1UdJQQsMCoGCCsGAQUFBwMEBggqgw4DAwQBAgYJKoMOAwMEAQIBBgkqgw4DAwQDAgEwDwYDVR0jBAgwBoAEW2pz6TAdBgNVHQ4EFgQU3hE7WqNKTTjuiDbD4q4uPI4j0GwwXgYDVR0gBFcwVTBTBgcqgw4DAwIBMEgwIQYIKwYBBQUHAgEWFWh0dHA6Ly9wa2kuZ292Lmt6L2NwczAjBggrBgEFBQcCAjAXDBVodHRwOi8vcGtpLmdvdi5rei9jcHMwWAYDVR0fBFEwTzBNoEugSYYiaHR0cDovL2NybC5wa2kuZ292Lmt6L25jYV9nb3N0LmNybIYjaHR0cDovL2NybDEucGtpLmdvdi5rei9uY2FfZ29zdC5jcmwwXAYDVR0uBFUwUzBRoE%2BgTYYkaHR0cDovL2NybC5wa2kuZ292Lmt6L25jYV9kX2dvc3QuY3JshiVodHRwOi8vY3JsMS5wa2kuZ292Lmt6L25jYV9kX2dvc3QuY3JsMGMGCCsGAQUFBwEBBFcwVTAvBggrBgEFBQcwAoYjaHR0cDovL3BraS5nb3Yua3ovY2VydC9uY2FfZ29zdC5jZXIwIgYIKwYBBQUHMAGGFmh0dHA6Ly9vY3NwLnBraS5nb3Yua3owDQYJKoMOAwoBAQECBQADQQBep8zG1qucVUVmucAEmFw6%2FUrSaOzZ1UErJMiq8IXZzQ6EW7WbjaD4jOZ7eBZjgkuKqecCxgiQA0g4E%2BZP9MgxMYIBPjCCAToCAQEwazBTMQswCQYDVQQGEwJLWjFEMEIGA1UEAww70rDQm9Ci0KLQq9KaINCa0KPTmNCb0JDQndCU0KvQoNCj0KjQqyDQntCg0KLQkNCb0KvSmiAoR09TVCkCFCwRbnBoCBMA8h0%2FI4xTAqNsVOx%2BMAwGCCqDDgMKAQMBBQCgaTAYBgkqhkiG9w0BCQMxCwYJKoZIhvcNAQcBMBwGCSqGSIb3DQEJBTEPFw0yMjAzMjcxOTI5NDRaMC8GCSqGSIb3DQEJBDEiBCCget%2F6ejnka%2FT1LA8tCDYu6NX8fAFE%2F24TskhkHKif8jANBgkqgw4DCgEBAQIFAARAhpRrKMHtLifr%2By%2Bn0%2F1Jy3mjT9068J6cIKG1f8RsZTUxhIzOuOpd7zceIgqyhis0TWp2fNXeOCxNrmnMV0Oh1g%3D%3D&signData%5B18457976%5D=MIIHLwYJKoZIhvcNAQcCoIIHIDCCBxwCAQExDjAMBggqgw4DCgEDAQUAMIHmBgkqhkiG9w0BBwGggdgEgdU8P3htbCB2ZXJzaW9uPSIxLjAiIGVuY29kaW5nPSJVVEYtOCI%2FPjxwcmljZW9mZmVyPjx0cmRfYnV5X2lkPjcxOTYwNzI8L3RyZF9idXlfaWQ%2BPHRyZF9idXlfbG90X2lkPjE5MTE0MjUxPC90cmRfYnV5X2xvdF9pZD48dHJkX2J1eV9sb3RfcG9pbnRfaWQ%2BMTg0NTc5NzY8L3RyZF9idXlfbG90X3BvaW50X2lkPjxwcmljZT4yNDc0MzMuMDQ8L3ByaWNlPjwvcHJpY2VvZmZlcj6gggTaMIIE1jCCBICgAwIBAgIULBFucGgIEwDyHT8jjFMCo2xU7H4wDQYJKoMOAwoBAQECBQAwUzELMAkGA1UEBhMCS1oxRDBCBgNVBAMMO9Kw0JvQotCi0KvSmiDQmtCj05jQm9CQ0J3QlNCr0KDQo9Co0Ksg0J7QoNCi0JDQm9Cr0pogKEdPU1QpMB4XDTIxMDYwNDA3NTE1NVoXDTIyMDYwNDA3NTE1NVowggF1MSowKAYDVQQDDCHQmtCQ0JnQk9Ce0KDQntCU0KbQldCS0JAg0JzQkNCZ0K8xITAfBgNVBAQMGNCa0JDQmdCT0J7QoNCe0JTQptCV0JLQkDEYMBYGA1UEBRMPSUlOODMwNTA0NDAwMDQ5MQswCQYDVQQGEwJLWjGBnDCBmQYDVQQKDIGR0KLQntCS0JDQoNCY0KnQldCh0KLQktCeINChINCe0JPQoNCQ0J3QmNCn0JXQndCd0J7QmSDQntCi0JLQldCi0KHQotCS0JXQndCd0J7QodCi0KzQriAi0JjQndCW0JjQndCY0KDQmNCd0JPQntCS0JDQryDQmtCe0JzQn9CQ0J3QmNCvICLQm9CY0JTQldCgIjEYMBYGA1UECwwPQklOMTEwOTQwMDA0OTA5MSMwIQYDVQQqDBrQodCi0JDQndCY0KHQm9CQ0JLQntCS0J3QkDEfMB0GCSqGSIb3DQEJARYQQ0VPQExFQURFUkVORy5LWjBsMCUGCSqDDgMKAQEBATAYBgoqgw4DCgEBAQEBBgoqgw4DCgEDAQEAA0MABEDk36lRIduJHXYf2mZSoaY46q06pz8fTkInD91Y8hQSbNfdw6zp7FB1Qp6pIal0sIrIl70JpgQOjxzDYNIV%2Fmkno4IB9jCCAfIwDgYDVR0PAQH%2FBAQDAgbAMDMGA1UdJQQsMCoGCCsGAQUFBwMEBggqgw4DAwQBAgYJKoMOAwMEAQIBBgkqgw4DAwQDAgEwDwYDVR0jBAgwBoAEW2pz6TAdBgNVHQ4EFgQU3hE7WqNKTTjuiDbD4q4uPI4j0GwwXgYDVR0gBFcwVTBTBgcqgw4DAwIBMEgwIQYIKwYBBQUHAgEWFWh0dHA6Ly9wa2kuZ292Lmt6L2NwczAjBggrBgEFBQcCAjAXDBVodHRwOi8vcGtpLmdvdi5rei9jcHMwWAYDVR0fBFEwTzBNoEugSYYiaHR0cDovL2NybC5wa2kuZ292Lmt6L25jYV9nb3N0LmNybIYjaHR0cDovL2NybDEucGtpLmdvdi5rei9uY2FfZ29zdC5jcmwwXAYDVR0uBFUwUzBRoE%2BgTYYkaHR0cDovL2NybC5wa2kuZ292Lmt6L25jYV9kX2dvc3QuY3JshiVodHRwOi8vY3JsMS5wa2kuZ292Lmt6L25jYV9kX2dvc3QuY3JsMGMGCCsGAQUFBwEBBFcwVTAvBggrBgEFBQcwAoYjaHR0cDovL3BraS5nb3Yua3ovY2VydC9uY2FfZ29zdC5jZXIwIgYIKwYBBQUHMAGGFmh0dHA6Ly9vY3NwLnBraS5nb3Yua3owDQYJKoMOAwoBAQECBQADQQBep8zG1qucVUVmucAEmFw6%2FUrSaOzZ1UErJMiq8IXZzQ6EW7WbjaD4jOZ7eBZjgkuKqecCxgiQA0g4E%2BZP9MgxMYIBPjCCAToCAQEwazBTMQswCQYDVQQGEwJLWjFEMEIGA1UEAww70rDQm9Ci0KLQq9KaINCa0KPTmNCb0JDQndCU0KvQoNCj0KjQqyDQntCg0KLQkNCb0KvSmiAoR09TVCkCFCwRbnBoCBMA8h0%2FI4xTAqNsVOx%2BMAwGCCqDDgMKAQMBBQCgaTAYBgkqhkiG9w0BCQMxCwYJKoZIhvcNAQcBMBwGCSqGSIb3DQEJBTEPFw0yMjAzMjcxOTI5NDRaMC8GCSqGSIb3DQEJBDEiBCDwlGwLUvL5B5NVdcwyFZhAA7jPk7A5tu7DmEHGKy5NBDANBgkqgw4DCgEBAQIFAARAFVYWNDj0o6EMv2vC5Odcxon8fbX3SGgQGlcLYKMbksO0zdPouX7seR4dWjEQzWfvuJXiIF4OqSX4Wwi9vCeuxw%3D%3D&signData%5B18457977%5D=MIIHLwYJKoZIhvcNAQcCoIIHIDCCBxwCAQExDjAMBggqgw4DCgEDAQUAMIHmBgkqhkiG9w0BBwGggdgEgdU8P3htbCB2ZXJzaW9uPSIxLjAiIGVuY29kaW5nPSJVVEYtOCI%2FPjxwcmljZW9mZmVyPjx0cmRfYnV5X2lkPjcxOTYwNzI8L3RyZF9idXlfaWQ%2BPHRyZF9idXlfbG90X2lkPjE5MTE0MjUyPC90cmRfYnV5X2xvdF9pZD48dHJkX2J1eV9sb3RfcG9pbnRfaWQ%2BMTg0NTc5Nzc8L3RyZF9idXlfbG90X3BvaW50X2lkPjxwcmljZT40MTYzMzAuMzY8L3ByaWNlPjwvcHJpY2VvZmZlcj6gggTaMIIE1jCCBICgAwIBAgIULBFucGgIEwDyHT8jjFMCo2xU7H4wDQYJKoMOAwoBAQECBQAwUzELMAkGA1UEBhMCS1oxRDBCBgNVBAMMO9Kw0JvQotCi0KvSmiDQmtCj05jQm9CQ0J3QlNCr0KDQo9Co0Ksg0J7QoNCi0JDQm9Cr0pogKEdPU1QpMB4XDTIxMDYwNDA3NTE1NVoXDTIyMDYwNDA3NTE1NVowggF1MSowKAYDVQQDDCHQmtCQ0JnQk9Ce0KDQntCU0KbQldCS0JAg0JzQkNCZ0K8xITAfBgNVBAQMGNCa0JDQmdCT0J7QoNCe0JTQptCV0JLQkDEYMBYGA1UEBRMPSUlOODMwNTA0NDAwMDQ5MQswCQYDVQQGEwJLWjGBnDCBmQYDVQQKDIGR0KLQntCS0JDQoNCY0KnQldCh0KLQktCeINChINCe0JPQoNCQ0J3QmNCn0JXQndCd0J7QmSDQntCi0JLQldCi0KHQotCS0JXQndCd0J7QodCi0KzQriAi0JjQndCW0JjQndCY0KDQmNCd0JPQntCS0JDQryDQmtCe0JzQn9CQ0J3QmNCvICLQm9CY0JTQldCgIjEYMBYGA1UECwwPQklOMTEwOTQwMDA0OTA5MSMwIQYDVQQqDBrQodCi0JDQndCY0KHQm9CQ0JLQntCS0J3QkDEfMB0GCSqGSIb3DQEJARYQQ0VPQExFQURFUkVORy5LWjBsMCUGCSqDDgMKAQEBATAYBgoqgw4DCgEBAQEBBgoqgw4DCgEDAQEAA0MABEDk36lRIduJHXYf2mZSoaY46q06pz8fTkInD91Y8hQSbNfdw6zp7FB1Qp6pIal0sIrIl70JpgQOjxzDYNIV%2Fmkno4IB9jCCAfIwDgYDVR0PAQH%2FBAQDAgbAMDMGA1UdJQQsMCoGCCsGAQUFBwMEBggqgw4DAwQBAgYJKoMOAwMEAQIBBgkqgw4DAwQDAgEwDwYDVR0jBAgwBoAEW2pz6TAdBgNVHQ4EFgQU3hE7WqNKTTjuiDbD4q4uPI4j0GwwXgYDVR0gBFcwVTBTBgcqgw4DAwIBMEgwIQYIKwYBBQUHAgEWFWh0dHA6Ly9wa2kuZ292Lmt6L2NwczAjBggrBgEFBQcCAjAXDBVodHRwOi8vcGtpLmdvdi5rei9jcHMwWAYDVR0fBFEwTzBNoEugSYYiaHR0cDovL2NybC5wa2kuZ292Lmt6L25jYV9nb3N0LmNybIYjaHR0cDovL2NybDEucGtpLmdvdi5rei9uY2FfZ29zdC5jcmwwXAYDVR0uBFUwUzBRoE%2BgTYYkaHR0cDovL2NybC5wa2kuZ292Lmt6L25jYV9kX2dvc3QuY3JshiVodHRwOi8vY3JsMS5wa2kuZ292Lmt6L25jYV9kX2dvc3QuY3JsMGMGCCsGAQUFBwEBBFcwVTAvBggrBgEFBQcwAoYjaHR0cDovL3BraS5nb3Yua3ovY2VydC9uY2FfZ29zdC5jZXIwIgYIKwYBBQUHMAGGFmh0dHA6Ly9vY3NwLnBraS5nb3Yua3owDQYJKoMOAwoBAQECBQADQQBep8zG1qucVUVmucAEmFw6%2FUrSaOzZ1UErJMiq8IXZzQ6EW7WbjaD4jOZ7eBZjgkuKqecCxgiQA0g4E%2BZP9MgxMYIBPjCCAToCAQEwazBTMQswCQYDVQQGEwJLWjFEMEIGA1UEAww70rDQm9Ci0KLQq9KaINCa0KPTmNCb0JDQndCU0KvQoNCj0KjQqyDQntCg0KLQkNCb0KvSmiAoR09TVCkCFCwRbnBoCBMA8h0%2FI4xTAqNsVOx%2BMAwGCCqDDgMKAQMBBQCgaTAYBgkqhkiG9w0BCQMxCwYJKoZIhvcNAQcBMBwGCSqGSIb3DQEJBTEPFw0yMjAzMjcxOTI5NDVaMC8GCSqGSIb3DQEJBDEiBCCtrT1zL9sc9G87JyEWAvokjqmXkbIubvdKXNBuxuRGYzANBgkqgw4DCgEBAQIFAARAZzg%2FszFzeTTQrzNBq%2FC2%2FdWcrVPK7CqlMA067jX%2BLfRRUvVJoS30XsYWY%2F1R1%2Bv24VCA3aqEsMvFqksflpWIyA%3D%3D&formData=offer%5B34280541%5D%5B18457975%5D%5Bprice%5D=466087.63&offer%5B34280542%5D%5B18457976%5D%5Bprice%5D=247433.04&offer%5B34280543%5D%5B18457977%5D%5Bprice%5D=416330.36'

In [15]:
# 1 price (3rd) and it is accepted
my_data = 'xmlData%5B18457977%5D=%3C%3Fxml+version%3D%221.0%22+encoding%3D%22UTF-8%22%3F%3E%3Cpriceoffer%3E%3Ctrd_buy_id%3E7196072%3C%2Ftrd_buy_id%3E%3Ctrd_buy_lot_id%3E19114252%3C%2Ftrd_buy_lot_id%3E%3Ctrd_buy_lot_point_id%3E18457977%3C%2Ftrd_buy_lot_point_id%3E%3Cprice%3E416330.36%3C%2Fprice%3E%3C%2Fpriceoffer%3E&signData%5B18457977%5D=MIIHLwYJKoZIhvcNAQcCoIIHIDCCBxwCAQExDjAMBggqgw4DCgEDAQUAMIHmBgkqhkiG9w0BBwGggdgEgdU8P3htbCB2ZXJzaW9uPSIxLjAiIGVuY29kaW5nPSJVVEYtOCI%2FPjxwcmljZW9mZmVyPjx0cmRfYnV5X2lkPjcxOTYwNzI8L3RyZF9idXlfaWQ%2BPHRyZF9idXlfbG90X2lkPjE5MTE0MjUyPC90cmRfYnV5X2xvdF9pZD48dHJkX2J1eV9sb3RfcG9pbnRfaWQ%2BMTg0NTc5Nzc8L3RyZF9idXlfbG90X3BvaW50X2lkPjxwcmljZT40MTYzMzAuMzY8L3ByaWNlPjwvcHJpY2VvZmZlcj6gggTaMIIE1jCCBICgAwIBAgIULBFucGgIEwDyHT8jjFMCo2xU7H4wDQYJKoMOAwoBAQECBQAwUzELMAkGA1UEBhMCS1oxRDBCBgNVBAMMO9Kw0JvQotCi0KvSmiDQmtCj05jQm9CQ0J3QlNCr0KDQo9Co0Ksg0J7QoNCi0JDQm9Cr0pogKEdPU1QpMB4XDTIxMDYwNDA3NTE1NVoXDTIyMDYwNDA3NTE1NVowggF1MSowKAYDVQQDDCHQmtCQ0JnQk9Ce0KDQntCU0KbQldCS0JAg0JzQkNCZ0K8xITAfBgNVBAQMGNCa0JDQmdCT0J7QoNCe0JTQptCV0JLQkDEYMBYGA1UEBRMPSUlOODMwNTA0NDAwMDQ5MQswCQYDVQQGEwJLWjGBnDCBmQYDVQQKDIGR0KLQntCS0JDQoNCY0KnQldCh0KLQktCeINChINCe0JPQoNCQ0J3QmNCn0JXQndCd0J7QmSDQntCi0JLQldCi0KHQotCS0JXQndCd0J7QodCi0KzQriAi0JjQndCW0JjQndCY0KDQmNCd0JPQntCS0JDQryDQmtCe0JzQn9CQ0J3QmNCvICLQm9CY0JTQldCgIjEYMBYGA1UECwwPQklOMTEwOTQwMDA0OTA5MSMwIQYDVQQqDBrQodCi0JDQndCY0KHQm9CQ0JLQntCS0J3QkDEfMB0GCSqGSIb3DQEJARYQQ0VPQExFQURFUkVORy5LWjBsMCUGCSqDDgMKAQEBATAYBgoqgw4DCgEBAQEBBgoqgw4DCgEDAQEAA0MABEDk36lRIduJHXYf2mZSoaY46q06pz8fTkInD91Y8hQSbNfdw6zp7FB1Qp6pIal0sIrIl70JpgQOjxzDYNIV%2Fmkno4IB9jCCAfIwDgYDVR0PAQH%2FBAQDAgbAMDMGA1UdJQQsMCoGCCsGAQUFBwMEBggqgw4DAwQBAgYJKoMOAwMEAQIBBgkqgw4DAwQDAgEwDwYDVR0jBAgwBoAEW2pz6TAdBgNVHQ4EFgQU3hE7WqNKTTjuiDbD4q4uPI4j0GwwXgYDVR0gBFcwVTBTBgcqgw4DAwIBMEgwIQYIKwYBBQUHAgEWFWh0dHA6Ly9wa2kuZ292Lmt6L2NwczAjBggrBgEFBQcCAjAXDBVodHRwOi8vcGtpLmdvdi5rei9jcHMwWAYDVR0fBFEwTzBNoEugSYYiaHR0cDovL2NybC5wa2kuZ292Lmt6L25jYV9nb3N0LmNybIYjaHR0cDovL2NybDEucGtpLmdvdi5rei9uY2FfZ29zdC5jcmwwXAYDVR0uBFUwUzBRoE%2BgTYYkaHR0cDovL2NybC5wa2kuZ292Lmt6L25jYV9kX2dvc3QuY3JshiVodHRwOi8vY3JsMS5wa2kuZ292Lmt6L25jYV9kX2dvc3QuY3JsMGMGCCsGAQUFBwEBBFcwVTAvBggrBgEFBQcwAoYjaHR0cDovL3BraS5nb3Yua3ovY2VydC9uY2FfZ29zdC5jZXIwIgYIKwYBBQUHMAGGFmh0dHA6Ly9vY3NwLnBraS5nb3Yua3owDQYJKoMOAwoBAQECBQADQQBep8zG1qucVUVmucAEmFw6%2FUrSaOzZ1UErJMiq8IXZzQ6EW7WbjaD4jOZ7eBZjgkuKqecCxgiQA0g4E%2BZP9MgxMYIBPjCCAToCAQEwazBTMQswCQYDVQQGEwJLWjFEMEIGA1UEAww70rDQm9Ci0KLQq9KaINCa0KPTmNCb0JDQndCU0KvQoNCj0KjQqyDQntCg0KLQkNCb0KvSmiAoR09TVCkCFCwRbnBoCBMA8h0%2FI4xTAqNsVOx%2BMAwGCCqDDgMKAQMBBQCgaTAYBgkqhkiG9w0BCQMxCwYJKoZIhvcNAQcBMBwGCSqGSIb3DQEJBTEPFw0yMjAzMjcxOTMzMjZaMC8GCSqGSIb3DQEJBDEiBCCtrT1zL9sc9G87JyEWAvokjqmXkbIubvdKXNBuxuRGYzANBgkqgw4DCgEBAQIFAARAbhFsRdA4jxLNrymPKOl%2FQN88%2FY34EsfziqUTG9YT44Fmy%2B6iEVuIYVyZR%2FUCohYi7yJsjLSiWSz%2BCuBFEMctUw%3D%3D&formData=offer%5B34280543%5D%5B18457977%5D%5Bprice%5D=416330.36'

In [16]:
token = "WjFaYFptUj1SfQV0VjtQYQc0ADcGDwMmXSYFN1N1UHEDaFBvDDJXBAtoC28GWVIlUzwPflo3BzFTZAM0Vl8AJV9uVDsEYFdgBDMCagM1BT1aNVpoWmBSZFI%2BBWNWMVBnB2IAbwY1AzRdNwU3U2BQMAMyUDEMPVdtCzkLbgY1UjNTNw88Wj8HY1NnAztWNABmXzFUMAQwVzIEPwJrAzUFOlozWjtabVJmUmQFNVY5UGQHMwA1BjYDZl1jBTNTYFA0A11QIgxnVygLOws6BjVSPVNbDy9aZAd3Uw8Db1ZnAGNfJVRmBCJXdgRaAnADaAV6WmpabVpjUltSJAU3Vi9QYAcqAD0GfQM0XQkFcFM9UHEDO1AxDGxXYQtdCykGcFJ0U2IPf1pSB2ZTNwNrVm0Ac18LVCEEald2BDwCYwNjBT1aaloGWntSSlJpBX1WaFA8B2gAbAZ8AzFdewViUyZQKgNZUDEMbVdgCyELRwZsUmlTcg91Wi0Hf1NoAzhWXAAxX2NUKgRxV0QEdgIiAz8FbFoHWj9aO1JMUm8Fc1YuUGUHNwA6Bn0DN11jBXJTLlBJA0lQVAwRVxcLLQsrBmlSblNsD2haLQdAUzUDbVZoAGhfflQjBBJXbQR0Aj0DPgVsWn9aY1pgUilSNgUpVjVQaAcwADkGfQMxXWQFclNVUGMDZ1BhDC5XMgsuCz4GNlIwUykPPlo7B1tTcgM1VnAAPV9mVDAEa1dZBCQCPgMyBXpaJFoFWjhSZFJyBW5Wd1A5B3AAdAYPAyZdbgU7UzxQMwM3UDQMZFdvCzELMgYyUjJTMw82WnA%3D"

In [17]:
import requests

cookies = {
    'show_filter_app': 'block',
    'ci_session': token
#     'ci_session': get_new_token(r2)
}

headers = {
    'Connection': 'keep-alive',
    'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="99", "Google Chrome";v="99"',
    'Accept': 'application/json, text/javascript, */*; q=0.01',
    'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
    'X-Requested-With': 'XMLHttpRequest',
    'sec-ch-ua-mobile': '?0',
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36',
    'sec-ch-ua-platform': '"Linux"',
    'Origin': 'https://v3bl.goszakup.gov.kz',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Dest': 'empty',
#     'Referer': 'https://v3bl.goszakup.gov.kz/ru/application/priceoffers/7196072/37068725',
    'Accept-Language': 'en-US,en;q=0.9',
    # Requests sorts cookies= alphabetically
    # 'Cookie': 'show_filter_app=block; ci_session=CmEAOlViUj0EKwR1VTgAMQAzADcCCwAlXSYHNQchBCUCaVZpV2kNXg9sCm5WCV0qAW5VJAFsBjANOgI1BQwEIVsyU2IHZQVvAGdVPAU3Az0KOQBjVTNSNARgBDNVNAAzAGAAOwJgAD9dYwcxBzMEYAIwVjFXPg04DzIKM1ZhXTwBNlUxATIGPw02AjsFNARgWzZTMwdrBWYAYVUxBTYDago4ADNVN1I3BGIEZFU0AGIANwA9AmUAYl1kBzYHMARkAlxWJFc8DXIPPwo7VmVdMgEJVXUBPwZ2DVECbgU0BGdbIVNhByEFJABeVScFbgN8CjoAN1VsUlsEcgQ2VSwAMAAtAD0CeQA3XQkHcgdpBCUCOlY3VzcNOw9ZCihWIF17ATBVJQEJBmcNaQJqBT4Ed1sPUyYHaQUkADhVNAVlAzsKOgBcVXRSSgQ%2FBHxVawBsAG8AbAJ4ADJdewdgB3IEfgJYVjdXNg06DyUKRlY8XWYBIFUvAXYGfg02AjkFDwQ1W2dTLQdyBRYAclV1BTkDagpXAGVVNFJMBDkEclUtADUAMAA6AnkANF1jB3AHegQdAkhWUldKDU0PKQoqVjldYQE%2BVTIBdgZBDWsCbAU7BGxbelMkBxEFPwBwVWoFOANqCi8AOVVvUikEYAQoVTYAOAA3ADkCeQAyXWQHcAcBBDcCZlZnV3UNaA8qCj9WZl0%2FAXtVZAFgBloNLAI0BSMEOVtiUzcHaAULACBVaQU0A3wKdABfVTdSZAQkBG9VdABpAHcAdAILACVdbgc5B2gEZwI2VjJXPw0zDzYKPFZjXTEBYFVsASs%3D',
}

# data = 'xmlData%5B18457975%5D=%3C%3Fxml+version%3D%221.0%22+encoding%3D%22UTF-8%22%3F%3E%3Cpriceoffer%3E%3Ctrd_buy_id%3E7196072%3C%2Ftrd_buy_id%3E%3Ctrd_buy_lot_id%3E19114250%3C%2Ftrd_buy_lot_id%3E%3Ctrd_buy_lot_point_id%3E18457975%3C%2Ftrd_buy_lot_point_id%3E%3Cprice%3E111.11%3C%2Fprice%3E%3C%2Fpriceoffer%3E&xmlData%5B18457976%5D=%3C%3Fxml+version%3D%221.0%22+encoding%3D%22UTF-8%22%3F%3E%3Cpriceoffer%3E%3Ctrd_buy_id%3E7196072%3C%2Ftrd_buy_id%3E%3Ctrd_buy_lot_id%3E19114251%3C%2Ftrd_buy_lot_id%3E%3Ctrd_buy_lot_point_id%3E18457976%3C%2Ftrd_buy_lot_point_id%3E%3Cprice%3E222.22%3C%2Fprice%3E%3C%2Fpriceoffer%3E&xmlData%5B18457977%5D=%3C%3Fxml+version%3D%221.0%22+encoding%3D%22UTF-8%22%3F%3E%3Cpriceoffer%3E%3Ctrd_buy_id%3E7196072%3C%2Ftrd_buy_id%3E%3Ctrd_buy_lot_id%3E19114252%3C%2Ftrd_buy_lot_id%3E%3Ctrd_buy_lot_point_id%3E18457977%3C%2Ftrd_buy_lot_point_id%3E%3Cprice%3E333.33%3C%2Fprice%3E%3C%2Fpriceoffer%3E&signData%5B18457975%5D=MIIP7gYJKoZIhvcNAQcCoIIP3zCCD9sCAQExDjAMBggqgw4DCgEDAQUAMAsGCSqGSIb3DQEHAaCCBNowggTWMIIEgKADAgECAhQsEW5waAgTAPIdPyOMUwKjbFTsfjANBgkqgw4DCgEBAQIFADBTMQswCQYDVQQGEwJLWjFEMEIGA1UEAww70rDQm9Ci0KLQq9KaINCa0KPTmNCb0JDQndCU0KvQoNCj0KjQqyDQntCg0KLQkNCb0KvSmiAoR09TVCkwHhcNMjEwNjA0MDc1MTU1WhcNMjIwNjA0MDc1MTU1WjCCAXUxKjAoBgNVBAMMIdCa0JDQmdCT0J7QoNCe0JTQptCV0JLQkCDQnNCQ0JnQrzEhMB8GA1UEBAwY0JrQkNCZ0JPQntCg0J7QlNCm0JXQktCQMRgwFgYDVQQFEw9JSU44MzA1MDQ0MDAwNDkxCzAJBgNVBAYTAktaMYGcMIGZBgNVBAoMgZHQotCe0JLQkNCg0JjQqdCV0KHQotCS0J4g0KEg0J7Qk9Cg0JDQndCY0KfQldCd0J3QntCZINCe0KLQktCV0KLQodCi0JLQldCd0J3QntCh0KLQrNCuICLQmNCd0JbQmNCd0JjQoNCY0J3Qk9Ce0JLQkNCvINCa0J7QnNCf0JDQndCY0K8gItCb0JjQlNCV0KAiMRgwFgYDVQQLDA9CSU4xMTA5NDAwMDQ5MDkxIzAhBgNVBCoMGtCh0KLQkNCd0JjQodCb0JDQktCe0JLQndCQMR8wHQYJKoZIhvcNAQkBFhBDRU9ATEVBREVSRU5HLktaMGwwJQYJKoMOAwoBAQEBMBgGCiqDDgMKAQEBAQEGCiqDDgMKAQMBAQADQwAEQOTfqVEh24kddh%2FaZlKhpjjqrTqnPx9OQicP3VjyFBJs193DrOnsUHVCnqkhqXSwisiXvQmmBA6PHMNg0hX%2BaSejggH2MIIB8jAOBgNVHQ8BAf8EBAMCBsAwMwYDVR0lBCwwKgYIKwYBBQUHAwQGCCqDDgMDBAECBgkqgw4DAwQBAgEGCSqDDgMDBAMCATAPBgNVHSMECDAGgARbanPpMB0GA1UdDgQWBBTeETtao0pNOO6INsPiri48jiPQbDBeBgNVHSAEVzBVMFMGByqDDgMDAgEwSDAhBggrBgEFBQcCARYVaHR0cDovL3BraS5nb3Yua3ovY3BzMCMGCCsGAQUFBwICMBcMFWh0dHA6Ly9wa2kuZ292Lmt6L2NwczBYBgNVHR8EUTBPME2gS6BJhiJodHRwOi8vY3JsLnBraS5nb3Yua3ovbmNhX2dvc3QuY3JshiNodHRwOi8vY3JsMS5wa2kuZ292Lmt6L25jYV9nb3N0LmNybDBcBgNVHS4EVTBTMFGgT6BNhiRodHRwOi8vY3JsLnBraS5nb3Yua3ovbmNhX2RfZ29zdC5jcmyGJWh0dHA6Ly9jcmwxLnBraS5nb3Yua3ovbmNhX2RfZ29zdC5jcmwwYwYIKwYBBQUHAQEEVzBVMC8GCCsGAQUFBzAChiNodHRwOi8vcGtpLmdvdi5rei9jZXJ0L25jYV9nb3N0LmNlcjAiBggrBgEFBQcwAYYWaHR0cDovL29jc3AucGtpLmdvdi5rejANBgkqgw4DCgEBAQIFAANBAF6nzMbWq5xVRWa5wASYXDr9StJo7NnVQSskyKrwhdnNDoRbtZuNoPiM5nt4FmOCS4qp5wLGCJADSDgT5k%2F0yDExggrZMIIK1QIBATBrMFMxCzAJBgNVBAYTAktaMUQwQgYDVQQDDDvSsNCb0KLQotCr0pog0JrQo9OY0JvQkNCd0JTQq9Cg0KPQqNCrINCe0KDQotCQ0JvQq9KaIChHT1NUKQIULBFucGgIEwDyHT8jjFMCo2xU7H4wDAYIKoMOAwoBAwEFAKCBojAYBgkqhkiG9w0BCQMxCwYJKoZIhvcNAQcBMBwGCSqGSIb3DQEJBTEPFw0yMjAzMjUxOTMyMzJaMC8GCSqGSIb3DQEJBDEiBCBcAdl2F0R08L1Ftyim6p%2BnYFZUN2vCrbCQyIcgkusjjTA3BgsqhkiG9w0BCRACLzEoMCYwJDAiBCDtSxHdcyUCMLpJK8z8EZFN1Szhqk3bdjuIAn0o7cdZcTANBgkqgw4DCgEBAQIFAARAb4KAp%2FJmEPuxF0ZSOf9x5F%2BHRtNS0BBTj4Sy8jkLHUt9Xea0s5qI9vluM3ovmO4OLVp0TnSspZz2C6OqX1LbmaGCCV0wgglZBgsqhkiG9w0BCRACDjGCCUgwgglEBgkqhkiG9w0BBwKgggk1MIIJMQIBAzEPMA0GCWCGSAFlAwQCAQUAMIGEBgsqhkiG9w0BCRABBKB1BHMwcQIBAQYIKoMOAwMCBgIwMTANBglghkgBZQMEAgEFAAQgNLQBaUfeeBUkDugOElnk%2FYPRHUnytaNxEhfpQCm44kACFCtGJ3jLSq63YAycSsFhTIu%2FXu3fGA8yMDIyMDMyNTE5MzIzMloCCOy1JFbjznbuoIIGXjCCBlowggRCoAMCAQICFD2d5W1fJ5xdBux9i4OqUL3rQ300MA0GCSqGSIb3DQEBCwUAMFIxCzAJBgNVBAYTAktaMUMwQQYDVQQDDDrSsNCb0KLQotCr0pog0JrQo9OY0JvQkNCd0JTQq9Cg0KPQqNCrINCe0KDQotCQ0JvQq9KaIChSU0EpMB4XDTE5MTIxMjA1MjIwNVoXDTIyMTIxMTA1MjIwNVowggESMRQwEgYDVQQDDAtUU0EgU0VSVklDRTEYMBYGA1UEBRMPSUlONzYxMjMxMzAwMzEzMQswCQYDVQQGEwJLWjEcMBoGA1UEBwwT0J3Qo9CgLdCh0KPQm9Ci0JDQnTEcMBoGA1UECAwT0J3Qo9CgLdCh0KPQm9Ci0JDQnTF9MHsGA1UECgx00JDQmtCm0JjQntCd0JXQoNCd0J7QlSDQntCR0KnQldCh0KLQktCeICLQndCQ0KbQmNCe0J3QkNCb0KzQndCr0JUg0JjQndCk0J7QoNCc0JDQptCY0J7QndCd0KvQlSDQotCV0KXQndCe0JvQntCT0JjQmCIxGDAWBgNVBAsMD0JJTjAwMDc0MDAwMDcyODCCASIwDQYJKoZIhvcNAQEBBQADggEPADCCAQoCggEBAIuFfpSCo5uenI8Cn9234hpG8uQZnRMBT2QwrndJaxIAHAMEQ9T3sxi2KWDL2mJArmVs48gdhhkm9JRVQCnYGb%2BOZ%2BnmePaeU826nawS83OVlJggIxbtAE4xdr8n2jCKMqjYgaQjXtllIC3NV5LvnZFeaoy%2Fndih%2Bo3iNbiSfEoWsX1xB8MW7ZHIqZBnHYOQnnpSW9TRejYF4B%2BFC4Sr70IXFRYHuicNJZo4V9ijGiacBA3WWOITxOPd0TZcd54G4kNFSSzocGiqMLVtWGNWBKzRtqxHPKdvoK2M3oZ5Ihqzjf0RnuS30HSQsXzGtYE1d7b7y%2BR5fr%2BpWdfp3C2ECdUCAwEAAaOCAWQwggFgMBYGA1UdJQEB%2FwQMMAoGCCsGAQUFBwMIMA8GA1UdIwQIMAaABFtqdBEwHQYDVR0OBBYEFFqrRbGX3k3FKok5zH%2FA2RUXLTKfMFYGA1UdHwRPME0wS6BJoEeGIWh0dHA6Ly9jcmwucGtpLmdvdi5rei9uY2FfcnNhLmNybIYiaHR0cDovL2NybDEucGtpLmdvdi5rei9uY2FfcnNhLmNybDBaBgNVHS4EUzBRME%2BgTaBLhiNodHRwOi8vY3JsLnBraS5nb3Yua3ovbmNhX2RfcnNhLmNybIYkaHR0cDovL2NybDEucGtpLmdvdi5rei9uY2FfZF9yc2EuY3JsMGIGCCsGAQUFBwEBBFYwVDAuBggrBgEFBQcwAoYiaHR0cDovL3BraS5nb3Yua3ovY2VydC9uY2FfcnNhLmNlcjAiBggrBgEFBQcwAYYWaHR0cDovL29jc3AucGtpLmdvdi5rejANBgkqhkiG9w0BAQsFAAOCAgEApPnpRXedzDgnZBPKJPhOJ8dWXzuNu65vSymgTH1hDXwE1lR90n5kZvu%2FkHO1shsId6TEg6X%2F1IE6Gp8992JsTezkxUexb6MdWWnUYjNhuPYzsYfBMG%2BZ%2BUMjMhf2ER5SUjDiEXrdwqKkRPF5HfzuKEdQfjoFctYveqirISBdEOUIGD0LCiqqTD7W5dAIQaVbJZLGCG5nKgZ8rvzYPyQJU3AqizXeHglOWLxINWmYs69NCqmvk6XI7oFVyVe8UTLQ%2BqMtl1MfrNTvSHtU2Uei2n2NKcN1oof5E2RDASf3ImNDeSJuFKzIXnwdc9xmesfUCBVJJLgt4HU6RHh%2B5StrR%2BWz7%2FOYBoVuhUE0YMSM%2FH74BrKbI6k8rmPw08QoU1v4cfYEH2jGzrJt4IephTkvxbv8ENcJX5LpUUux6oJHLTIzTU9q1BjitoODmV2EmmBn1hzU%2BYaSPEffIztXt8zuoVrH1gkT1s5bzc4nqc63OYur6XSTf6d66%2B2mavaVSQObNcYPz6YPnQFbDHzUy16oxX4FxlhnFSwul6gZg4b9kYh1BTdpXTGrfo4hD%2Fc%2F4xF3OHWxprGnWb9uFxJqthhHkwEHloMcZaR5xSLT7kE4Lpy5uRz9qh4DVcbSTw2sqCJx%2F57YioRy7GUssbhG51%2BYBnDG9OEF7xhV6xdvF%2FSVVvwxggIwMIICLAIBATBqMFIxCzAJBgNVBAYTAktaMUMwQQYDVQQDDDrSsNCb0KLQotCr0pog0JrQo9OY0JvQkNCd0JTQq9Cg0KPQqNCrINCe0KDQotCQ0JvQq9KaIChSU0EpAhQ9neVtXyecXQbsfYuDqlC960N9NDANBglghkgBZQMEAgEFAKCBmDAaBgkqhkiG9w0BCQMxDQYLKoZIhvcNAQkQAQQwHAYJKoZIhvcNAQkFMQ8XDTIyMDMyNTE5MzIzMlowKwYLKoZIhvcNAQkQAgwxHDAaMBgwFgQUyyVFl1vW4xC%2B7Z6ZVsJ4kYybLQgwLwYJKoZIhvcNAQkEMSIEIMoYCtIgp39pQ18Ctyx6UuRYxI8afS%2BI0DpBJWlbaUGJMA0GCSqGSIb3DQEBCwUABIIBAFp9U%2B%2FeaDanBMyzRwW4JVw5YQKHiJc5skdygAevMYG0U3D3mVJ3zpiG1BpnugdJW3UrGgAIGLRsxYVYpJeJsRcV50EWIVFnZ%2Bk9dvJQdlgCmQcG%2BEuxrZBN6evlTIF9BVLuCqvk46RfrhOibWMJwXpzhG9ojMCSHI06ljT5QeCko%2FoVlTkgCR1Yt1iYm%2FlWCFk2l7mClUYkdC%2FahzL94MmYGf5rRIykKpImD2lS4tandML1veY%2BHu3YPWRjOQ4%2FCUzyDi5gVQixo2Ddu%2BdT8JDs7oB9BhyZxMSKvcCU2%2FGyLjmqrX%2Boq%2FzJ5yfRMKXvlb3v%2BNdR48WIUwBasRCBlIs%3D&signData%5B18457976%5D=MIIP7gYJKoZIhvcNAQcCoIIP3zCCD9sCAQExDjAMBggqgw4DCgEDAQUAMAsGCSqGSIb3DQEHAaCCBNowggTWMIIEgKADAgECAhQsEW5waAgTAPIdPyOMUwKjbFTsfjANBgkqgw4DCgEBAQIFADBTMQswCQYDVQQGEwJLWjFEMEIGA1UEAww70rDQm9Ci0KLQq9KaINCa0KPTmNCb0JDQndCU0KvQoNCj0KjQqyDQntCg0KLQkNCb0KvSmiAoR09TVCkwHhcNMjEwNjA0MDc1MTU1WhcNMjIwNjA0MDc1MTU1WjCCAXUxKjAoBgNVBAMMIdCa0JDQmdCT0J7QoNCe0JTQptCV0JLQkCDQnNCQ0JnQrzEhMB8GA1UEBAwY0JrQkNCZ0JPQntCg0J7QlNCm0JXQktCQMRgwFgYDVQQFEw9JSU44MzA1MDQ0MDAwNDkxCzAJBgNVBAYTAktaMYGcMIGZBgNVBAoMgZHQotCe0JLQkNCg0JjQqdCV0KHQotCS0J4g0KEg0J7Qk9Cg0JDQndCY0KfQldCd0J3QntCZINCe0KLQktCV0KLQodCi0JLQldCd0J3QntCh0KLQrNCuICLQmNCd0JbQmNCd0JjQoNCY0J3Qk9Ce0JLQkNCvINCa0J7QnNCf0JDQndCY0K8gItCb0JjQlNCV0KAiMRgwFgYDVQQLDA9CSU4xMTA5NDAwMDQ5MDkxIzAhBgNVBCoMGtCh0KLQkNCd0JjQodCb0JDQktCe0JLQndCQMR8wHQYJKoZIhvcNAQkBFhBDRU9ATEVBREVSRU5HLktaMGwwJQYJKoMOAwoBAQEBMBgGCiqDDgMKAQEBAQEGCiqDDgMKAQMBAQADQwAEQOTfqVEh24kddh%2FaZlKhpjjqrTqnPx9OQicP3VjyFBJs193DrOnsUHVCnqkhqXSwisiXvQmmBA6PHMNg0hX%2BaSejggH2MIIB8jAOBgNVHQ8BAf8EBAMCBsAwMwYDVR0lBCwwKgYIKwYBBQUHAwQGCCqDDgMDBAECBgkqgw4DAwQBAgEGCSqDDgMDBAMCATAPBgNVHSMECDAGgARbanPpMB0GA1UdDgQWBBTeETtao0pNOO6INsPiri48jiPQbDBeBgNVHSAEVzBVMFMGByqDDgMDAgEwSDAhBggrBgEFBQcCARYVaHR0cDovL3BraS5nb3Yua3ovY3BzMCMGCCsGAQUFBwICMBcMFWh0dHA6Ly9wa2kuZ292Lmt6L2NwczBYBgNVHR8EUTBPME2gS6BJhiJodHRwOi8vY3JsLnBraS5nb3Yua3ovbmNhX2dvc3QuY3JshiNodHRwOi8vY3JsMS5wa2kuZ292Lmt6L25jYV9nb3N0LmNybDBcBgNVHS4EVTBTMFGgT6BNhiRodHRwOi8vY3JsLnBraS5nb3Yua3ovbmNhX2RfZ29zdC5jcmyGJWh0dHA6Ly9jcmwxLnBraS5nb3Yua3ovbmNhX2RfZ29zdC5jcmwwYwYIKwYBBQUHAQEEVzBVMC8GCCsGAQUFBzAChiNodHRwOi8vcGtpLmdvdi5rei9jZXJ0L25jYV9nb3N0LmNlcjAiBggrBgEFBQcwAYYWaHR0cDovL29jc3AucGtpLmdvdi5rejANBgkqgw4DCgEBAQIFAANBAF6nzMbWq5xVRWa5wASYXDr9StJo7NnVQSskyKrwhdnNDoRbtZuNoPiM5nt4FmOCS4qp5wLGCJADSDgT5k%2F0yDExggrZMIIK1QIBATBrMFMxCzAJBgNVBAYTAktaMUQwQgYDVQQDDDvSsNCb0KLQotCr0pog0JrQo9OY0JvQkNCd0JTQq9Cg0KPQqNCrINCe0KDQotCQ0JvQq9KaIChHT1NUKQIULBFucGgIEwDyHT8jjFMCo2xU7H4wDAYIKoMOAwoBAwEFAKCBojAYBgkqhkiG9w0BCQMxCwYJKoZIhvcNAQcBMBwGCSqGSIb3DQEJBTEPFw0yMjAzMjUxOTMyMzJaMC8GCSqGSIb3DQEJBDEiBCBMtLCV6Ji1UFFtLAudeTlQ%2F9rrDWE4mlXeUgExqNEyBTA3BgsqhkiG9w0BCRACLzEoMCYwJDAiBCDtSxHdcyUCMLpJK8z8EZFN1Szhqk3bdjuIAn0o7cdZcTANBgkqgw4DCgEBAQIFAARAThW1cyaNCwYFSu3l5Vnv4ZGqmr437hZxGSeyJUBRdgTJ95NcrOxbrD3bIHayVNbeWW%2BAWJNiuutO1BEwbrGbSaGCCV0wgglZBgsqhkiG9w0BCRACDjGCCUgwgglEBgkqhkiG9w0BBwKgggk1MIIJMQIBAzEPMA0GCWCGSAFlAwQCAQUAMIGEBgsqhkiG9w0BCRABBKB1BHMwcQIBAQYIKoMOAwMCBgIwMTANBglghkgBZQMEAgEFAAQg%2BXmv62ji8PITiVcWR%2FaNCh2gZ3t0KuObzo9DyirAXooCFJxv8wJVq8dVPhxongnro6bLkAUXGA8yMDIyMDMyNTE5MzIzMloCCJtQLUTkU61KoIIGXjCCBlowggRCoAMCAQICFD2d5W1fJ5xdBux9i4OqUL3rQ300MA0GCSqGSIb3DQEBCwUAMFIxCzAJBgNVBAYTAktaMUMwQQYDVQQDDDrSsNCb0KLQotCr0pog0JrQo9OY0JvQkNCd0JTQq9Cg0KPQqNCrINCe0KDQotCQ0JvQq9KaIChSU0EpMB4XDTE5MTIxMjA1MjIwNVoXDTIyMTIxMTA1MjIwNVowggESMRQwEgYDVQQDDAtUU0EgU0VSVklDRTEYMBYGA1UEBRMPSUlONzYxMjMxMzAwMzEzMQswCQYDVQQGEwJLWjEcMBoGA1UEBwwT0J3Qo9CgLdCh0KPQm9Ci0JDQnTEcMBoGA1UECAwT0J3Qo9CgLdCh0KPQm9Ci0JDQnTF9MHsGA1UECgx00JDQmtCm0JjQntCd0JXQoNCd0J7QlSDQntCR0KnQldCh0KLQktCeICLQndCQ0KbQmNCe0J3QkNCb0KzQndCr0JUg0JjQndCk0J7QoNCc0JDQptCY0J7QndCd0KvQlSDQotCV0KXQndCe0JvQntCT0JjQmCIxGDAWBgNVBAsMD0JJTjAwMDc0MDAwMDcyODCCASIwDQYJKoZIhvcNAQEBBQADggEPADCCAQoCggEBAIuFfpSCo5uenI8Cn9234hpG8uQZnRMBT2QwrndJaxIAHAMEQ9T3sxi2KWDL2mJArmVs48gdhhkm9JRVQCnYGb%2BOZ%2BnmePaeU826nawS83OVlJggIxbtAE4xdr8n2jCKMqjYgaQjXtllIC3NV5LvnZFeaoy%2Fndih%2Bo3iNbiSfEoWsX1xB8MW7ZHIqZBnHYOQnnpSW9TRejYF4B%2BFC4Sr70IXFRYHuicNJZo4V9ijGiacBA3WWOITxOPd0TZcd54G4kNFSSzocGiqMLVtWGNWBKzRtqxHPKdvoK2M3oZ5Ihqzjf0RnuS30HSQsXzGtYE1d7b7y%2BR5fr%2BpWdfp3C2ECdUCAwEAAaOCAWQwggFgMBYGA1UdJQEB%2FwQMMAoGCCsGAQUFBwMIMA8GA1UdIwQIMAaABFtqdBEwHQYDVR0OBBYEFFqrRbGX3k3FKok5zH%2FA2RUXLTKfMFYGA1UdHwRPME0wS6BJoEeGIWh0dHA6Ly9jcmwucGtpLmdvdi5rei9uY2FfcnNhLmNybIYiaHR0cDovL2NybDEucGtpLmdvdi5rei9uY2FfcnNhLmNybDBaBgNVHS4EUzBRME%2BgTaBLhiNodHRwOi8vY3JsLnBraS5nb3Yua3ovbmNhX2RfcnNhLmNybIYkaHR0cDovL2NybDEucGtpLmdvdi5rei9uY2FfZF9yc2EuY3JsMGIGCCsGAQUFBwEBBFYwVDAuBggrBgEFBQcwAoYiaHR0cDovL3BraS5nb3Yua3ovY2VydC9uY2FfcnNhLmNlcjAiBggrBgEFBQcwAYYWaHR0cDovL29jc3AucGtpLmdvdi5rejANBgkqhkiG9w0BAQsFAAOCAgEApPnpRXedzDgnZBPKJPhOJ8dWXzuNu65vSymgTH1hDXwE1lR90n5kZvu%2FkHO1shsId6TEg6X%2F1IE6Gp8992JsTezkxUexb6MdWWnUYjNhuPYzsYfBMG%2BZ%2BUMjMhf2ER5SUjDiEXrdwqKkRPF5HfzuKEdQfjoFctYveqirISBdEOUIGD0LCiqqTD7W5dAIQaVbJZLGCG5nKgZ8rvzYPyQJU3AqizXeHglOWLxINWmYs69NCqmvk6XI7oFVyVe8UTLQ%2BqMtl1MfrNTvSHtU2Uei2n2NKcN1oof5E2RDASf3ImNDeSJuFKzIXnwdc9xmesfUCBVJJLgt4HU6RHh%2B5StrR%2BWz7%2FOYBoVuhUE0YMSM%2FH74BrKbI6k8rmPw08QoU1v4cfYEH2jGzrJt4IephTkvxbv8ENcJX5LpUUux6oJHLTIzTU9q1BjitoODmV2EmmBn1hzU%2BYaSPEffIztXt8zuoVrH1gkT1s5bzc4nqc63OYur6XSTf6d66%2B2mavaVSQObNcYPz6YPnQFbDHzUy16oxX4FxlhnFSwul6gZg4b9kYh1BTdpXTGrfo4hD%2Fc%2F4xF3OHWxprGnWb9uFxJqthhHkwEHloMcZaR5xSLT7kE4Lpy5uRz9qh4DVcbSTw2sqCJx%2F57YioRy7GUssbhG51%2BYBnDG9OEF7xhV6xdvF%2FSVVvwxggIwMIICLAIBATBqMFIxCzAJBgNVBAYTAktaMUMwQQYDVQQDDDrSsNCb0KLQotCr0pog0JrQo9OY0JvQkNCd0JTQq9Cg0KPQqNCrINCe0KDQotCQ0JvQq9KaIChSU0EpAhQ9neVtXyecXQbsfYuDqlC960N9NDANBglghkgBZQMEAgEFAKCBmDAaBgkqhkiG9w0BCQMxDQYLKoZIhvcNAQkQAQQwHAYJKoZIhvcNAQkFMQ8XDTIyMDMyNTE5MzIzMlowKwYLKoZIhvcNAQkQAgwxHDAaMBgwFgQUyyVFl1vW4xC%2B7Z6ZVsJ4kYybLQgwLwYJKoZIhvcNAQkEMSIEIHXsef8DZLj3D%2FcmpxpqaWa5hf3iaxZnZ5zb2REu8COzMA0GCSqGSIb3DQEBCwUABIIBAG65V3st8W3tAvq974mF11opa%2F2EdBoEEmwG5MVmVkJ7fGc54n%2F0XKqEscrXPAHKJbvD2LWlqLJz%2FuKNVW87lN0BuOYm00JeD2nHZxO08OHLAa8mItsQdy0ujQBCjw7dSxIVSR1LbpmkaQIFloBC93P3wCMnbQbmLFJKEgpKpzl1rL75kOzUluU9ZCRi6NjYPKFYfzAkhiF38MGL56AQ%2B5SionZAtBZclHuhbrTLa0R5pNWWezAAY7N1w9dooZBWRoiu61asoHaoo%2BtzWhSF7y6Ckvej7fJerDvxr%2FgaiBzmqL3XGOcl5Z4SnSvMhgfGn%2F8n0LhdlvSY5eKdi64R1Tc%3D&signData%5B18457977%5D=MIIP7gYJKoZIhvcNAQcCoIIP3zCCD9sCAQExDjAMBggqgw4DCgEDAQUAMAsGCSqGSIb3DQEHAaCCBNowggTWMIIEgKADAgECAhQsEW5waAgTAPIdPyOMUwKjbFTsfjANBgkqgw4DCgEBAQIFADBTMQswCQYDVQQGEwJLWjFEMEIGA1UEAww70rDQm9Ci0KLQq9KaINCa0KPTmNCb0JDQndCU0KvQoNCj0KjQqyDQntCg0KLQkNCb0KvSmiAoR09TVCkwHhcNMjEwNjA0MDc1MTU1WhcNMjIwNjA0MDc1MTU1WjCCAXUxKjAoBgNVBAMMIdCa0JDQmdCT0J7QoNCe0JTQptCV0JLQkCDQnNCQ0JnQrzEhMB8GA1UEBAwY0JrQkNCZ0JPQntCg0J7QlNCm0JXQktCQMRgwFgYDVQQFEw9JSU44MzA1MDQ0MDAwNDkxCzAJBgNVBAYTAktaMYGcMIGZBgNVBAoMgZHQotCe0JLQkNCg0JjQqdCV0KHQotCS0J4g0KEg0J7Qk9Cg0JDQndCY0KfQldCd0J3QntCZINCe0KLQktCV0KLQodCi0JLQldCd0J3QntCh0KLQrNCuICLQmNCd0JbQmNCd0JjQoNCY0J3Qk9Ce0JLQkNCvINCa0J7QnNCf0JDQndCY0K8gItCb0JjQlNCV0KAiMRgwFgYDVQQLDA9CSU4xMTA5NDAwMDQ5MDkxIzAhBgNVBCoMGtCh0KLQkNCd0JjQodCb0JDQktCe0JLQndCQMR8wHQYJKoZIhvcNAQkBFhBDRU9ATEVBREVSRU5HLktaMGwwJQYJKoMOAwoBAQEBMBgGCiqDDgMKAQEBAQEGCiqDDgMKAQMBAQADQwAEQOTfqVEh24kddh%2FaZlKhpjjqrTqnPx9OQicP3VjyFBJs193DrOnsUHVCnqkhqXSwisiXvQmmBA6PHMNg0hX%2BaSejggH2MIIB8jAOBgNVHQ8BAf8EBAMCBsAwMwYDVR0lBCwwKgYIKwYBBQUHAwQGCCqDDgMDBAECBgkqgw4DAwQBAgEGCSqDDgMDBAMCATAPBgNVHSMECDAGgARbanPpMB0GA1UdDgQWBBTeETtao0pNOO6INsPiri48jiPQbDBeBgNVHSAEVzBVMFMGByqDDgMDAgEwSDAhBggrBgEFBQcCARYVaHR0cDovL3BraS5nb3Yua3ovY3BzMCMGCCsGAQUFBwICMBcMFWh0dHA6Ly9wa2kuZ292Lmt6L2NwczBYBgNVHR8EUTBPME2gS6BJhiJodHRwOi8vY3JsLnBraS5nb3Yua3ovbmNhX2dvc3QuY3JshiNodHRwOi8vY3JsMS5wa2kuZ292Lmt6L25jYV9nb3N0LmNybDBcBgNVHS4EVTBTMFGgT6BNhiRodHRwOi8vY3JsLnBraS5nb3Yua3ovbmNhX2RfZ29zdC5jcmyGJWh0dHA6Ly9jcmwxLnBraS5nb3Yua3ovbmNhX2RfZ29zdC5jcmwwYwYIKwYBBQUHAQEEVzBVMC8GCCsGAQUFBzAChiNodHRwOi8vcGtpLmdvdi5rei9jZXJ0L25jYV9nb3N0LmNlcjAiBggrBgEFBQcwAYYWaHR0cDovL29jc3AucGtpLmdvdi5rejANBgkqgw4DCgEBAQIFAANBAF6nzMbWq5xVRWa5wASYXDr9StJo7NnVQSskyKrwhdnNDoRbtZuNoPiM5nt4FmOCS4qp5wLGCJADSDgT5k%2F0yDExggrZMIIK1QIBATBrMFMxCzAJBgNVBAYTAktaMUQwQgYDVQQDDDvSsNCb0KLQotCr0pog0JrQo9OY0JvQkNCd0JTQq9Cg0KPQqNCrINCe0KDQotCQ0JvQq9KaIChHT1NUKQIULBFucGgIEwDyHT8jjFMCo2xU7H4wDAYIKoMOAwoBAwEFAKCBojAYBgkqhkiG9w0BCQMxCwYJKoZIhvcNAQcBMBwGCSqGSIb3DQEJBTEPFw0yMjAzMjUxOTMyMzFaMC8GCSqGSIb3DQEJBDEiBCAauslNJ6XKaCSKNO5RmeRFZTW4sE7RCdQc%2B7XRuF1KqDA3BgsqhkiG9w0BCRACLzEoMCYwJDAiBCDtSxHdcyUCMLpJK8z8EZFN1Szhqk3bdjuIAn0o7cdZcTANBgkqgw4DCgEBAQIFAARA3laKz7bun8TRbTvCDryP05gTUeEgdRmuqFmPWSJ2JIWYClNMGI4DROHRmlQCtUb%2B0Idl%2BCNoONhsl%2Fg8Y0cU7aGCCV0wgglZBgsqhkiG9w0BCRACDjGCCUgwgglEBgkqhkiG9w0BBwKgggk1MIIJMQIBAzEPMA0GCWCGSAFlAwQCAQUAMIGEBgsqhkiG9w0BCRABBKB1BHMwcQIBAQYIKoMOAwMCBgIwMTANBglghkgBZQMEAgEFAAQgt0oLTcCCAKvm%2FpUK8qZwnsPxseo7dBXwNNixsueOCW4CFJdhLhgUYpIqK6iOQz682FjR1oWMGA8yMDIyMDMyNTE5MzIzMloCCOq6KliOwFe%2BoIIGXjCCBlowggRCoAMCAQICFD2d5W1fJ5xdBux9i4OqUL3rQ300MA0GCSqGSIb3DQEBCwUAMFIxCzAJBgNVBAYTAktaMUMwQQYDVQQDDDrSsNCb0KLQotCr0pog0JrQo9OY0JvQkNCd0JTQq9Cg0KPQqNCrINCe0KDQotCQ0JvQq9KaIChSU0EpMB4XDTE5MTIxMjA1MjIwNVoXDTIyMTIxMTA1MjIwNVowggESMRQwEgYDVQQDDAtUU0EgU0VSVklDRTEYMBYGA1UEBRMPSUlONzYxMjMxMzAwMzEzMQswCQYDVQQGEwJLWjEcMBoGA1UEBwwT0J3Qo9CgLdCh0KPQm9Ci0JDQnTEcMBoGA1UECAwT0J3Qo9CgLdCh0KPQm9Ci0JDQnTF9MHsGA1UECgx00JDQmtCm0JjQntCd0JXQoNCd0J7QlSDQntCR0KnQldCh0KLQktCeICLQndCQ0KbQmNCe0J3QkNCb0KzQndCr0JUg0JjQndCk0J7QoNCc0JDQptCY0J7QndCd0KvQlSDQotCV0KXQndCe0JvQntCT0JjQmCIxGDAWBgNVBAsMD0JJTjAwMDc0MDAwMDcyODCCASIwDQYJKoZIhvcNAQEBBQADggEPADCCAQoCggEBAIuFfpSCo5uenI8Cn9234hpG8uQZnRMBT2QwrndJaxIAHAMEQ9T3sxi2KWDL2mJArmVs48gdhhkm9JRVQCnYGb%2BOZ%2BnmePaeU826nawS83OVlJggIxbtAE4xdr8n2jCKMqjYgaQjXtllIC3NV5LvnZFeaoy%2Fndih%2Bo3iNbiSfEoWsX1xB8MW7ZHIqZBnHYOQnnpSW9TRejYF4B%2BFC4Sr70IXFRYHuicNJZo4V9ijGiacBA3WWOITxOPd0TZcd54G4kNFSSzocGiqMLVtWGNWBKzRtqxHPKdvoK2M3oZ5Ihqzjf0RnuS30HSQsXzGtYE1d7b7y%2BR5fr%2BpWdfp3C2ECdUCAwEAAaOCAWQwggFgMBYGA1UdJQEB%2FwQMMAoGCCsGAQUFBwMIMA8GA1UdIwQIMAaABFtqdBEwHQYDVR0OBBYEFFqrRbGX3k3FKok5zH%2FA2RUXLTKfMFYGA1UdHwRPME0wS6BJoEeGIWh0dHA6Ly9jcmwucGtpLmdvdi5rei9uY2FfcnNhLmNybIYiaHR0cDovL2NybDEucGtpLmdvdi5rei9uY2FfcnNhLmNybDBaBgNVHS4EUzBRME%2BgTaBLhiNodHRwOi8vY3JsLnBraS5nb3Yua3ovbmNhX2RfcnNhLmNybIYkaHR0cDovL2NybDEucGtpLmdvdi5rei9uY2FfZF9yc2EuY3JsMGIGCCsGAQUFBwEBBFYwVDAuBggrBgEFBQcwAoYiaHR0cDovL3BraS5nb3Yua3ovY2VydC9uY2FfcnNhLmNlcjAiBggrBgEFBQcwAYYWaHR0cDovL29jc3AucGtpLmdvdi5rejANBgkqhkiG9w0BAQsFAAOCAgEApPnpRXedzDgnZBPKJPhOJ8dWXzuNu65vSymgTH1hDXwE1lR90n5kZvu%2FkHO1shsId6TEg6X%2F1IE6Gp8992JsTezkxUexb6MdWWnUYjNhuPYzsYfBMG%2BZ%2BUMjMhf2ER5SUjDiEXrdwqKkRPF5HfzuKEdQfjoFctYveqirISBdEOUIGD0LCiqqTD7W5dAIQaVbJZLGCG5nKgZ8rvzYPyQJU3AqizXeHglOWLxINWmYs69NCqmvk6XI7oFVyVe8UTLQ%2BqMtl1MfrNTvSHtU2Uei2n2NKcN1oof5E2RDASf3ImNDeSJuFKzIXnwdc9xmesfUCBVJJLgt4HU6RHh%2B5StrR%2BWz7%2FOYBoVuhUE0YMSM%2FH74BrKbI6k8rmPw08QoU1v4cfYEH2jGzrJt4IephTkvxbv8ENcJX5LpUUux6oJHLTIzTU9q1BjitoODmV2EmmBn1hzU%2BYaSPEffIztXt8zuoVrH1gkT1s5bzc4nqc63OYur6XSTf6d66%2B2mavaVSQObNcYPz6YPnQFbDHzUy16oxX4FxlhnFSwul6gZg4b9kYh1BTdpXTGrfo4hD%2Fc%2F4xF3OHWxprGnWb9uFxJqthhHkwEHloMcZaR5xSLT7kE4Lpy5uRz9qh4DVcbSTw2sqCJx%2F57YioRy7GUssbhG51%2BYBnDG9OEF7xhV6xdvF%2FSVVvwxggIwMIICLAIBATBqMFIxCzAJBgNVBAYTAktaMUMwQQYDVQQDDDrSsNCb0KLQotCr0pog0JrQo9OY0JvQkNCd0JTQq9Cg0KPQqNCrINCe0KDQotCQ0JvQq9KaIChSU0EpAhQ9neVtXyecXQbsfYuDqlC960N9NDANBglghkgBZQMEAgEFAKCBmDAaBgkqhkiG9w0BCQMxDQYLKoZIhvcNAQkQAQQwHAYJKoZIhvcNAQkFMQ8XDTIyMDMyNTE5MzIzMlowKwYLKoZIhvcNAQkQAgwxHDAaMBgwFgQUyyVFl1vW4xC%2B7Z6ZVsJ4kYybLQgwLwYJKoZIhvcNAQkEMSIEIDuBi30sEz0YBIeANW6t5gRk%2F6DOVLqHWmic7KFALJqmMA0GCSqGSIb3DQEBCwUABIIBAFrFTPKNlwEOEld27PGdGjCUYodCfz%2FXAXDVveUm%2BbUdThPmdOZv6sFwB30o%2FcfhcpDclZ8MrVjxx%2Bexvg5zS8oH3BO%2FlY3Rtt9ctxyK8UmheOOLmyXh%2BIKMa3KgKYT2wHvU0cSsQXzt%2BlXktHKVHJ46qR3TgpgNQj5%2BKrCJ6SmAlP1n2G%2BPPZ2uJLnkVi8fGOitqj3zjvOdPzpTzwOwDmRyKuEe7U%2FpHogfizMyoSOLu7Xwb4knHsqPDkF8l40u%2FV2eW%2ByuiAtU0tNOcOOgthOvUVxXO2zHsoTObhjUFNz8c3K7lfiXSXDgviV29uobPeAbVCArLSfAJXeKayl2HKE%3D&formData=offer%255B34280083%255D%255B18457975%255D%255Bprice%255D%3D111.11%26offer%255B34280084%255D%255B18457976%255D%255Bprice%255D%3D222.22%26offer%255B34280085%255D%255B18457977%255D%255Bprice%255D%3D333.33'
data = my_data

r3 = requests.post(
    f'https://v3bl.goszakup.gov.kz/ru/application/ajax_add_priceoffers/{self.order_id}/{self.application_id}', 
    headers=headers, cookies=cookies, data=data, verify=False
)

r3

/home/necuk/.local/lib/python3.8/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'v3bl.goszakup.gov.kz'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


<Response [200]>

In [18]:
r3.text

'{"status":1}'

In [66]:
my_data

'xmlData%5B18457977%5D=%3C%3Fxml+version%3D%221.0%22+encoding%3D%22UTF-8%22%3F%3E%3Cpriceoffer%3E%3Ctrd_buy_id%3E7196072%3C%2Ftrd_buy_id%3E%3Ctrd_buy_lot_id%3E19114252%3C%2Ftrd_buy_lot_id%3E%3Ctrd_buy_lot_point_id%3E18457977%3C%2Ftrd_buy_lot_point_id%3E%3Cprice%3E416330.36%3C%2Fprice%3E%3C%2Fpriceoffer%3E&signData%5B18457977%5D=MIIHLwYJKoZIhvcNAQcCoIIHIDCCBxwCAQExDjAMBggqgw4DCgEDAQUAMIHmBgkqhkiG9w0BBwGggdgEgdU8P3htbCB2ZXJzaW9uPSIxLjAiIGVuY29kaW5nPSJVVEYtOCI%2FPjxwcmljZW9mZmVyPjx0cmRfYnV5X2lkPjcxOTYwNzI8L3RyZF9idXlfaWQ%2BPHRyZF9idXlfbG90X2lkPjE5MTE0MjUyPC90cmRfYnV5X2xvdF9pZD48dHJkX2J1eV9sb3RfcG9pbnRfaWQ%2BMTg0NTc5Nzc8L3RyZF9idXlfbG90X3BvaW50X2lkPjxwcmljZT40MTYzMzAuMzY8L3ByaWNlPjwvcHJpY2VvZmZlcj6gggTaMIIE1jCCBICgAwIBAgIULBFucGgIEwDyHT8jjFMCo2xU7H4wDQYJKoMOAwoBAQECBQAwUzELMAkGA1UEBhMCS1oxRDBCBgNVBAMMO9Kw0JvQotCi0KvSmiDQmtCj05jQm9CQ0J3QlNCr0KDQo9Co0Ksg0J7QoNCi0JDQm9Cr0pogKEdPU1QpMB4XDTIxMDYwNDA3NTE1NVoXDTIyMDYwNDA3NTE1NVowggF1MSowKAYDVQQDDCHQmtCQ0JnQk9Ce0KDQntCU0KbQldCS0JAg0JzQkNCZ0K8xITAf

In [352]:
self.prices

[{'price': 466087.63,
  'buy_lot_id': '19114250',
  'buy_lot_point_id': '18457975',
  'app_lot_id': '34272171',
  'xml_string': '<?xml version="1.0" encoding="UTF-8"?><priceoffer><trd_buy_id>7196072</trd_buy_id><trd_buy_lot_id>19114250</trd_buy_lot_id><trd_buy_lot_point_id>18457975</trd_buy_lot_point_id><price>466087.63</price></priceoffer>'},
 {'price': 247433.04,
  'buy_lot_id': '19114251',
  'buy_lot_point_id': '18457976',
  'app_lot_id': '34272172',
  'xml_string': '<?xml version="1.0" encoding="UTF-8"?><priceoffer><trd_buy_id>7196072</trd_buy_id><trd_buy_lot_id>19114251</trd_buy_lot_id><trd_buy_lot_point_id>18457976</trd_buy_lot_point_id><price>247433.04</price></priceoffer>'},
 {'price': 416330.36,
  'buy_lot_id': '19114252',
  'buy_lot_point_id': '18457977',
  'app_lot_id': '34272173',
  'xml_string': '<?xml version="1.0" encoding="UTF-8"?><priceoffer><trd_buy_id>7196072</trd_buy_id><trd_buy_lot_id>19114252</trd_buy_lot_id><trd_buy_lot_point_id>18457977</trd_buy_lot_point_id><pr

In [76]:
self.prices[0]

{'price': 466087.63,
 'buy_lot_id': '19114250',
 'buy_lot_point_id': '18457975',
 'app_lot_id': '34272171',
 'xml_string': '<?xml version="1.0" encoding="UTF-8"?><priceoffer><trd_buy_id>7196072</trd_buy_id><trd_buy_lot_id>19114250</trd_buy_lot_id><trd_buy_lot_point_id>18457975</trd_buy_lot_point_id><price>466087.63</price></priceoffer>'}

In [119]:
data = {}

for p in self.prices:
    data[f"xmlData[{p['buy_lot_point_id']}]"] = p['xml_string']
    data[f"signData[{p['buy_lot_point_id']}]"] = sign(p['xml_string'])

200
200
200


In [130]:
data = 'xmlData%5B18457975%5D=%3C%3Fxml+version%3D%221.0%22+encoding%3D%22UTF-8%22%3F%3E%3Cpriceoffer%3E%3Ctrd_buy_id%3E7196072%3C%2Ftrd_buy_id%3E%3Ctrd_buy_lot_id%3E19114250%3C%2Ftrd_buy_lot_id%3E%3Ctrd_buy_lot_point_id%3E18457975%3C%2Ftrd_buy_lot_point_id%3E%3Cprice%3E11%3C%2Fprice%3E%3C%2Fpriceoffer%3E&xmlData%5B18457976%5D=%3C%3Fxml+version%3D%221.0%22+encoding%3D%22UTF-8%22%3F%3E%3Cpriceoffer%3E%3Ctrd_buy_id%3E7196072%3C%2Ftrd_buy_id%3E%3Ctrd_buy_lot_id%3E19114251%3C%2Ftrd_buy_lot_id%3E%3Ctrd_buy_lot_point_id%3E18457976%3C%2Ftrd_buy_lot_point_id%3E%3Cprice%3E22%3C%2Fprice%3E%3C%2Fpriceoffer%3E&xmlData%5B18457977%5D=%3C%3Fxml+version%3D%221.0%22+encoding%3D%22UTF-8%22%3F%3E%3Cpriceoffer%3E%3Ctrd_buy_id%3E7196072%3C%2Ftrd_buy_id%3E%3Ctrd_buy_lot_id%3E19114252%3C%2Ftrd_buy_lot_id%3E%3Ctrd_buy_lot_point_id%3E18457977%3C%2Ftrd_buy_lot_point_id%3E%3Cprice%3E33%3C%2Fprice%3E%3C%2Fpriceoffer%3E&signData%5B18457975%5D=MIIP7gYJKoZIhvcNAQcCoIIP3zCCD9sCAQExDjAMBggqgw4DCgEDAQUAMAsGCSqGSIb3DQEHAaCCBNowggTWMIIEgKADAgECAhQsEW5waAgTAPIdPyOMUwKjbFTsfjANBgkqgw4DCgEBAQIFADBTMQswCQYDVQQGEwJLWjFEMEIGA1UEAww70rDQm9Ci0KLQq9KaINCa0KPTmNCb0JDQndCU0KvQoNCj0KjQqyDQntCg0KLQkNCb0KvSmiAoR09TVCkwHhcNMjEwNjA0MDc1MTU1WhcNMjIwNjA0MDc1MTU1WjCCAXUxKjAoBgNVBAMMIdCa0JDQmdCT0J7QoNCe0JTQptCV0JLQkCDQnNCQ0JnQrzEhMB8GA1UEBAwY0JrQkNCZ0JPQntCg0J7QlNCm0JXQktCQMRgwFgYDVQQFEw9JSU44MzA1MDQ0MDAwNDkxCzAJBgNVBAYTAktaMYGcMIGZBgNVBAoMgZHQotCe0JLQkNCg0JjQqdCV0KHQotCS0J4g0KEg0J7Qk9Cg0JDQndCY0KfQldCd0J3QntCZINCe0KLQktCV0KLQodCi0JLQldCd0J3QntCh0KLQrNCuICLQmNCd0JbQmNCd0JjQoNCY0J3Qk9Ce0JLQkNCvINCa0J7QnNCf0JDQndCY0K8gItCb0JjQlNCV0KAiMRgwFgYDVQQLDA9CSU4xMTA5NDAwMDQ5MDkxIzAhBgNVBCoMGtCh0KLQkNCd0JjQodCb0JDQktCe0JLQndCQMR8wHQYJKoZIhvcNAQkBFhBDRU9ATEVBREVSRU5HLktaMGwwJQYJKoMOAwoBAQEBMBgGCiqDDgMKAQEBAQEGCiqDDgMKAQMBAQADQwAEQOTfqVEh24kddh%2FaZlKhpjjqrTqnPx9OQicP3VjyFBJs193DrOnsUHVCnqkhqXSwisiXvQmmBA6PHMNg0hX%2BaSejggH2MIIB8jAOBgNVHQ8BAf8EBAMCBsAwMwYDVR0lBCwwKgYIKwYBBQUHAwQGCCqDDgMDBAECBgkqgw4DAwQBAgEGCSqDDgMDBAMCATAPBgNVHSMECDAGgARbanPpMB0GA1UdDgQWBBTeETtao0pNOO6INsPiri48jiPQbDBeBgNVHSAEVzBVMFMGByqDDgMDAgEwSDAhBggrBgEFBQcCARYVaHR0cDovL3BraS5nb3Yua3ovY3BzMCMGCCsGAQUFBwICMBcMFWh0dHA6Ly9wa2kuZ292Lmt6L2NwczBYBgNVHR8EUTBPME2gS6BJhiJodHRwOi8vY3JsLnBraS5nb3Yua3ovbmNhX2dvc3QuY3JshiNodHRwOi8vY3JsMS5wa2kuZ292Lmt6L25jYV9nb3N0LmNybDBcBgNVHS4EVTBTMFGgT6BNhiRodHRwOi8vY3JsLnBraS5nb3Yua3ovbmNhX2RfZ29zdC5jcmyGJWh0dHA6Ly9jcmwxLnBraS5nb3Yua3ovbmNhX2RfZ29zdC5jcmwwYwYIKwYBBQUHAQEEVzBVMC8GCCsGAQUFBzAChiNodHRwOi8vcGtpLmdvdi5rei9jZXJ0L25jYV9nb3N0LmNlcjAiBggrBgEFBQcwAYYWaHR0cDovL29jc3AucGtpLmdvdi5rejANBgkqgw4DCgEBAQIFAANBAF6nzMbWq5xVRWa5wASYXDr9StJo7NnVQSskyKrwhdnNDoRbtZuNoPiM5nt4FmOCS4qp5wLGCJADSDgT5k%2F0yDExggrZMIIK1QIBATBrMFMxCzAJBgNVBAYTAktaMUQwQgYDVQQDDDvSsNCb0KLQotCr0pog0JrQo9OY0JvQkNCd0JTQq9Cg0KPQqNCrINCe0KDQotCQ0JvQq9KaIChHT1NUKQIULBFucGgIEwDyHT8jjFMCo2xU7H4wDAYIKoMOAwoBAwEFAKCBojAYBgkqhkiG9w0BCQMxCwYJKoZIhvcNAQcBMBwGCSqGSIb3DQEJBTEPFw0yMjAzMjUxNDQxNDJaMC8GCSqGSIb3DQEJBDEiBCB8qjCGZVAshUyFnm7LwZdksVaMcWO7JSLPXDYuSYXOezA3BgsqhkiG9w0BCRACLzEoMCYwJDAiBCDtSxHdcyUCMLpJK8z8EZFN1Szhqk3bdjuIAn0o7cdZcTANBgkqgw4DCgEBAQIFAARA2lFpl3CCwbWu0Wjqc1xKrMSwFF63Ns1azC1O3wef%2Flr95m5J2Lj7DoDa%2FpQXcLpeiS8gEHdybm4D6pO80RNVnaGCCV0wgglZBgsqhkiG9w0BCRACDjGCCUgwgglEBgkqhkiG9w0BBwKgggk1MIIJMQIBAzEPMA0GCWCGSAFlAwQCAQUAMIGEBgsqhkiG9w0BCRABBKB1BHMwcQIBAQYIKoMOAwMCBgIwMTANBglghkgBZQMEAgEFAAQgRYxRFBpofODghuXC0QFXNTuphL8MZLwFlfYzy%2BITU0kCFLjkV3bRy2zEoWnYReuD%2BIxxSBrtGA8yMDIyMDMyNTE0NDE0M1oCCD1E6MIDnqBeoIIGXjCCBlowggRCoAMCAQICFD2d5W1fJ5xdBux9i4OqUL3rQ300MA0GCSqGSIb3DQEBCwUAMFIxCzAJBgNVBAYTAktaMUMwQQYDVQQDDDrSsNCb0KLQotCr0pog0JrQo9OY0JvQkNCd0JTQq9Cg0KPQqNCrINCe0KDQotCQ0JvQq9KaIChSU0EpMB4XDTE5MTIxMjA1MjIwNVoXDTIyMTIxMTA1MjIwNVowggESMRQwEgYDVQQDDAtUU0EgU0VSVklDRTEYMBYGA1UEBRMPSUlONzYxMjMxMzAwMzEzMQswCQYDVQQGEwJLWjEcMBoGA1UEBwwT0J3Qo9CgLdCh0KPQm9Ci0JDQnTEcMBoGA1UECAwT0J3Qo9CgLdCh0KPQm9Ci0JDQnTF9MHsGA1UECgx00JDQmtCm0JjQntCd0JXQoNCd0J7QlSDQntCR0KnQldCh0KLQktCeICLQndCQ0KbQmNCe0J3QkNCb0KzQndCr0JUg0JjQndCk0J7QoNCc0JDQptCY0J7QndCd0KvQlSDQotCV0KXQndCe0JvQntCT0JjQmCIxGDAWBgNVBAsMD0JJTjAwMDc0MDAwMDcyODCCASIwDQYJKoZIhvcNAQEBBQADggEPADCCAQoCggEBAIuFfpSCo5uenI8Cn9234hpG8uQZnRMBT2QwrndJaxIAHAMEQ9T3sxi2KWDL2mJArmVs48gdhhkm9JRVQCnYGb%2BOZ%2BnmePaeU826nawS83OVlJggIxbtAE4xdr8n2jCKMqjYgaQjXtllIC3NV5LvnZFeaoy%2Fndih%2Bo3iNbiSfEoWsX1xB8MW7ZHIqZBnHYOQnnpSW9TRejYF4B%2BFC4Sr70IXFRYHuicNJZo4V9ijGiacBA3WWOITxOPd0TZcd54G4kNFSSzocGiqMLVtWGNWBKzRtqxHPKdvoK2M3oZ5Ihqzjf0RnuS30HSQsXzGtYE1d7b7y%2BR5fr%2BpWdfp3C2ECdUCAwEAAaOCAWQwggFgMBYGA1UdJQEB%2FwQMMAoGCCsGAQUFBwMIMA8GA1UdIwQIMAaABFtqdBEwHQYDVR0OBBYEFFqrRbGX3k3FKok5zH%2FA2RUXLTKfMFYGA1UdHwRPME0wS6BJoEeGIWh0dHA6Ly9jcmwucGtpLmdvdi5rei9uY2FfcnNhLmNybIYiaHR0cDovL2NybDEucGtpLmdvdi5rei9uY2FfcnNhLmNybDBaBgNVHS4EUzBRME%2BgTaBLhiNodHRwOi8vY3JsLnBraS5nb3Yua3ovbmNhX2RfcnNhLmNybIYkaHR0cDovL2NybDEucGtpLmdvdi5rei9uY2FfZF9yc2EuY3JsMGIGCCsGAQUFBwEBBFYwVDAuBggrBgEFBQcwAoYiaHR0cDovL3BraS5nb3Yua3ovY2VydC9uY2FfcnNhLmNlcjAiBggrBgEFBQcwAYYWaHR0cDovL29jc3AucGtpLmdvdi5rejANBgkqhkiG9w0BAQsFAAOCAgEApPnpRXedzDgnZBPKJPhOJ8dWXzuNu65vSymgTH1hDXwE1lR90n5kZvu%2FkHO1shsId6TEg6X%2F1IE6Gp8992JsTezkxUexb6MdWWnUYjNhuPYzsYfBMG%2BZ%2BUMjMhf2ER5SUjDiEXrdwqKkRPF5HfzuKEdQfjoFctYveqirISBdEOUIGD0LCiqqTD7W5dAIQaVbJZLGCG5nKgZ8rvzYPyQJU3AqizXeHglOWLxINWmYs69NCqmvk6XI7oFVyVe8UTLQ%2BqMtl1MfrNTvSHtU2Uei2n2NKcN1oof5E2RDASf3ImNDeSJuFKzIXnwdc9xmesfUCBVJJLgt4HU6RHh%2B5StrR%2BWz7%2FOYBoVuhUE0YMSM%2FH74BrKbI6k8rmPw08QoU1v4cfYEH2jGzrJt4IephTkvxbv8ENcJX5LpUUux6oJHLTIzTU9q1BjitoODmV2EmmBn1hzU%2BYaSPEffIztXt8zuoVrH1gkT1s5bzc4nqc63OYur6XSTf6d66%2B2mavaVSQObNcYPz6YPnQFbDHzUy16oxX4FxlhnFSwul6gZg4b9kYh1BTdpXTGrfo4hD%2Fc%2F4xF3OHWxprGnWb9uFxJqthhHkwEHloMcZaR5xSLT7kE4Lpy5uRz9qh4DVcbSTw2sqCJx%2F57YioRy7GUssbhG51%2BYBnDG9OEF7xhV6xdvF%2FSVVvwxggIwMIICLAIBATBqMFIxCzAJBgNVBAYTAktaMUMwQQYDVQQDDDrSsNCb0KLQotCr0pog0JrQo9OY0JvQkNCd0JTQq9Cg0KPQqNCrINCe0KDQotCQ0JvQq9KaIChSU0EpAhQ9neVtXyecXQbsfYuDqlC960N9NDANBglghkgBZQMEAgEFAKCBmDAaBgkqhkiG9w0BCQMxDQYLKoZIhvcNAQkQAQQwHAYJKoZIhvcNAQkFMQ8XDTIyMDMyNTE0NDE0M1owKwYLKoZIhvcNAQkQAgwxHDAaMBgwFgQUyyVFl1vW4xC%2B7Z6ZVsJ4kYybLQgwLwYJKoZIhvcNAQkEMSIEIMpzqR3sHGmOGu0XJVHsXFw6dVa3w8ktEK9f353qhdRFMA0GCSqGSIb3DQEBCwUABIIBAD3GFSl6rkv1UraXHbCbYGNaPwT0%2FtqAEMH1JgWMa2v8CQiyXERifjSTz%2B0RuKt6JDbvbRW0FWwjwVPdF4Rn79Xl2MubC7kwGkbgoYwKZqDGUxj%2Bkr3Ji0eW0C5n4HAHY%2FsHFa3A84oAleCUExFVnVnYgeqXIpy27tQ19PqcudRDsRgMTTprak8nhq8NEa6DVd83PCsSEgpKvUEmkP55wv3tCV8eBC49owswvWNKQfycr8VR1ihAvuFqdD9dPbDaCU9ArUjgIHxv9gM6%2BEdIKK9Nht8QL6K1ySli7VHOPiqUFmVtXmTgpHXeJRYNvEbw6o1%2BRFn6WYVMbwBwijOCzMc%3D&signData%5B18457976%5D=MIIP7gYJKoZIhvcNAQcCoIIP3zCCD9sCAQExDjAMBggqgw4DCgEDAQUAMAsGCSqGSIb3DQEHAaCCBNowggTWMIIEgKADAgECAhQsEW5waAgTAPIdPyOMUwKjbFTsfjANBgkqgw4DCgEBAQIFADBTMQswCQYDVQQGEwJLWjFEMEIGA1UEAww70rDQm9Ci0KLQq9KaINCa0KPTmNCb0JDQndCU0KvQoNCj0KjQqyDQntCg0KLQkNCb0KvSmiAoR09TVCkwHhcNMjEwNjA0MDc1MTU1WhcNMjIwNjA0MDc1MTU1WjCCAXUxKjAoBgNVBAMMIdCa0JDQmdCT0J7QoNCe0JTQptCV0JLQkCDQnNCQ0JnQrzEhMB8GA1UEBAwY0JrQkNCZ0JPQntCg0J7QlNCm0JXQktCQMRgwFgYDVQQFEw9JSU44MzA1MDQ0MDAwNDkxCzAJBgNVBAYTAktaMYGcMIGZBgNVBAoMgZHQotCe0JLQkNCg0JjQqdCV0KHQotCS0J4g0KEg0J7Qk9Cg0JDQndCY0KfQldCd0J3QntCZINCe0KLQktCV0KLQodCi0JLQldCd0J3QntCh0KLQrNCuICLQmNCd0JbQmNCd0JjQoNCY0J3Qk9Ce0JLQkNCvINCa0J7QnNCf0JDQndCY0K8gItCb0JjQlNCV0KAiMRgwFgYDVQQLDA9CSU4xMTA5NDAwMDQ5MDkxIzAhBgNVBCoMGtCh0KLQkNCd0JjQodCb0JDQktCe0JLQndCQMR8wHQYJKoZIhvcNAQkBFhBDRU9ATEVBREVSRU5HLktaMGwwJQYJKoMOAwoBAQEBMBgGCiqDDgMKAQEBAQEGCiqDDgMKAQMBAQADQwAEQOTfqVEh24kddh%2FaZlKhpjjqrTqnPx9OQicP3VjyFBJs193DrOnsUHVCnqkhqXSwisiXvQmmBA6PHMNg0hX%2BaSejggH2MIIB8jAOBgNVHQ8BAf8EBAMCBsAwMwYDVR0lBCwwKgYIKwYBBQUHAwQGCCqDDgMDBAECBgkqgw4DAwQBAgEGCSqDDgMDBAMCATAPBgNVHSMECDAGgARbanPpMB0GA1UdDgQWBBTeETtao0pNOO6INsPiri48jiPQbDBeBgNVHSAEVzBVMFMGByqDDgMDAgEwSDAhBggrBgEFBQcCARYVaHR0cDovL3BraS5nb3Yua3ovY3BzMCMGCCsGAQUFBwICMBcMFWh0dHA6Ly9wa2kuZ292Lmt6L2NwczBYBgNVHR8EUTBPME2gS6BJhiJodHRwOi8vY3JsLnBraS5nb3Yua3ovbmNhX2dvc3QuY3JshiNodHRwOi8vY3JsMS5wa2kuZ292Lmt6L25jYV9nb3N0LmNybDBcBgNVHS4EVTBTMFGgT6BNhiRodHRwOi8vY3JsLnBraS5nb3Yua3ovbmNhX2RfZ29zdC5jcmyGJWh0dHA6Ly9jcmwxLnBraS5nb3Yua3ovbmNhX2RfZ29zdC5jcmwwYwYIKwYBBQUHAQEEVzBVMC8GCCsGAQUFBzAChiNodHRwOi8vcGtpLmdvdi5rei9jZXJ0L25jYV9nb3N0LmNlcjAiBggrBgEFBQcwAYYWaHR0cDovL29jc3AucGtpLmdvdi5rejANBgkqgw4DCgEBAQIFAANBAF6nzMbWq5xVRWa5wASYXDr9StJo7NnVQSskyKrwhdnNDoRbtZuNoPiM5nt4FmOCS4qp5wLGCJADSDgT5k%2F0yDExggrZMIIK1QIBATBrMFMxCzAJBgNVBAYTAktaMUQwQgYDVQQDDDvSsNCb0KLQotCr0pog0JrQo9OY0JvQkNCd0JTQq9Cg0KPQqNCrINCe0KDQotCQ0JvQq9KaIChHT1NUKQIULBFucGgIEwDyHT8jjFMCo2xU7H4wDAYIKoMOAwoBAwEFAKCBojAYBgkqhkiG9w0BCQMxCwYJKoZIhvcNAQcBMBwGCSqGSIb3DQEJBTEPFw0yMjAzMjUxNDQxNDJaMC8GCSqGSIb3DQEJBDEiBCAqeLIjxykElKrkpwrNwjWQhUDh4ovQUGWJuFi%2B7SvJ8TA3BgsqhkiG9w0BCRACLzEoMCYwJDAiBCDtSxHdcyUCMLpJK8z8EZFN1Szhqk3bdjuIAn0o7cdZcTANBgkqgw4DCgEBAQIFAARAeFZL2JKD5jMhgVFJfHnxE%2FWkD3uvYNhLTsDR3MIyIaU%2BkVKfP%2Fq0BU%2BBgBzzSeAeaj1g6C0ZVlDwqLRzrpcQSKGCCV0wgglZBgsqhkiG9w0BCRACDjGCCUgwgglEBgkqhkiG9w0BBwKgggk1MIIJMQIBAzEPMA0GCWCGSAFlAwQCAQUAMIGEBgsqhkiG9w0BCRABBKB1BHMwcQIBAQYIKoMOAwMCBgIwMTANBglghkgBZQMEAgEFAAQg%2BloS2y0zZuq6WAwqn1QIe4%2FLbyvpRJ0FqeaPGELE%2F%2FECFMzDWElhYKIYp7Z%2B7DGhconFiNp0GA8yMDIyMDMyNTE0NDE0MloCCGvZCpY1vVFKoIIGXjCCBlowggRCoAMCAQICFD2d5W1fJ5xdBux9i4OqUL3rQ300MA0GCSqGSIb3DQEBCwUAMFIxCzAJBgNVBAYTAktaMUMwQQYDVQQDDDrSsNCb0KLQotCr0pog0JrQo9OY0JvQkNCd0JTQq9Cg0KPQqNCrINCe0KDQotCQ0JvQq9KaIChSU0EpMB4XDTE5MTIxMjA1MjIwNVoXDTIyMTIxMTA1MjIwNVowggESMRQwEgYDVQQDDAtUU0EgU0VSVklDRTEYMBYGA1UEBRMPSUlONzYxMjMxMzAwMzEzMQswCQYDVQQGEwJLWjEcMBoGA1UEBwwT0J3Qo9CgLdCh0KPQm9Ci0JDQnTEcMBoGA1UECAwT0J3Qo9CgLdCh0KPQm9Ci0JDQnTF9MHsGA1UECgx00JDQmtCm0JjQntCd0JXQoNCd0J7QlSDQntCR0KnQldCh0KLQktCeICLQndCQ0KbQmNCe0J3QkNCb0KzQndCr0JUg0JjQndCk0J7QoNCc0JDQptCY0J7QndCd0KvQlSDQotCV0KXQndCe0JvQntCT0JjQmCIxGDAWBgNVBAsMD0JJTjAwMDc0MDAwMDcyODCCASIwDQYJKoZIhvcNAQEBBQADggEPADCCAQoCggEBAIuFfpSCo5uenI8Cn9234hpG8uQZnRMBT2QwrndJaxIAHAMEQ9T3sxi2KWDL2mJArmVs48gdhhkm9JRVQCnYGb%2BOZ%2BnmePaeU826nawS83OVlJggIxbtAE4xdr8n2jCKMqjYgaQjXtllIC3NV5LvnZFeaoy%2Fndih%2Bo3iNbiSfEoWsX1xB8MW7ZHIqZBnHYOQnnpSW9TRejYF4B%2BFC4Sr70IXFRYHuicNJZo4V9ijGiacBA3WWOITxOPd0TZcd54G4kNFSSzocGiqMLVtWGNWBKzRtqxHPKdvoK2M3oZ5Ihqzjf0RnuS30HSQsXzGtYE1d7b7y%2BR5fr%2BpWdfp3C2ECdUCAwEAAaOCAWQwggFgMBYGA1UdJQEB%2FwQMMAoGCCsGAQUFBwMIMA8GA1UdIwQIMAaABFtqdBEwHQYDVR0OBBYEFFqrRbGX3k3FKok5zH%2FA2RUXLTKfMFYGA1UdHwRPME0wS6BJoEeGIWh0dHA6Ly9jcmwucGtpLmdvdi5rei9uY2FfcnNhLmNybIYiaHR0cDovL2NybDEucGtpLmdvdi5rei9uY2FfcnNhLmNybDBaBgNVHS4EUzBRME%2BgTaBLhiNodHRwOi8vY3JsLnBraS5nb3Yua3ovbmNhX2RfcnNhLmNybIYkaHR0cDovL2NybDEucGtpLmdvdi5rei9uY2FfZF9yc2EuY3JsMGIGCCsGAQUFBwEBBFYwVDAuBggrBgEFBQcwAoYiaHR0cDovL3BraS5nb3Yua3ovY2VydC9uY2FfcnNhLmNlcjAiBggrBgEFBQcwAYYWaHR0cDovL29jc3AucGtpLmdvdi5rejANBgkqhkiG9w0BAQsFAAOCAgEApPnpRXedzDgnZBPKJPhOJ8dWXzuNu65vSymgTH1hDXwE1lR90n5kZvu%2FkHO1shsId6TEg6X%2F1IE6Gp8992JsTezkxUexb6MdWWnUYjNhuPYzsYfBMG%2BZ%2BUMjMhf2ER5SUjDiEXrdwqKkRPF5HfzuKEdQfjoFctYveqirISBdEOUIGD0LCiqqTD7W5dAIQaVbJZLGCG5nKgZ8rvzYPyQJU3AqizXeHglOWLxINWmYs69NCqmvk6XI7oFVyVe8UTLQ%2BqMtl1MfrNTvSHtU2Uei2n2NKcN1oof5E2RDASf3ImNDeSJuFKzIXnwdc9xmesfUCBVJJLgt4HU6RHh%2B5StrR%2BWz7%2FOYBoVuhUE0YMSM%2FH74BrKbI6k8rmPw08QoU1v4cfYEH2jGzrJt4IephTkvxbv8ENcJX5LpUUux6oJHLTIzTU9q1BjitoODmV2EmmBn1hzU%2BYaSPEffIztXt8zuoVrH1gkT1s5bzc4nqc63OYur6XSTf6d66%2B2mavaVSQObNcYPz6YPnQFbDHzUy16oxX4FxlhnFSwul6gZg4b9kYh1BTdpXTGrfo4hD%2Fc%2F4xF3OHWxprGnWb9uFxJqthhHkwEHloMcZaR5xSLT7kE4Lpy5uRz9qh4DVcbSTw2sqCJx%2F57YioRy7GUssbhG51%2BYBnDG9OEF7xhV6xdvF%2FSVVvwxggIwMIICLAIBATBqMFIxCzAJBgNVBAYTAktaMUMwQQYDVQQDDDrSsNCb0KLQotCr0pog0JrQo9OY0JvQkNCd0JTQq9Cg0KPQqNCrINCe0KDQotCQ0JvQq9KaIChSU0EpAhQ9neVtXyecXQbsfYuDqlC960N9NDANBglghkgBZQMEAgEFAKCBmDAaBgkqhkiG9w0BCQMxDQYLKoZIhvcNAQkQAQQwHAYJKoZIhvcNAQkFMQ8XDTIyMDMyNTE0NDE0MlowKwYLKoZIhvcNAQkQAgwxHDAaMBgwFgQUyyVFl1vW4xC%2B7Z6ZVsJ4kYybLQgwLwYJKoZIhvcNAQkEMSIEIPnmuHkbKsQRwwhb6RxZ4k%2FNGGAj%2Bvfr5a7bTsg8D%2FELMA0GCSqGSIb3DQEBCwUABIIBAEt1WmKBKyKabCPR4m90QbLpSZ%2B%2FZjYDug3A3XDjQSNa7rbsKOUoJBu2r4ObcsSDNCfY9HN24o6EOnqnx2bPIzThTFgylzix09DxP9VhGvAoVhqAfdcKGjN6oNAxuDDvTv%2Bhahc1gnzM0Q0gI2EIIGb2A1BWE86opiUUw%2BuV9hf9saWDredTAhll2CzNje2IrTwyf51zp%2BA5wZ%2Fjlm0bzh7deY%2FEQ13yb%2Fyjc94cCE%2FZ1Ijq63F6mwTl1EAehXMFZD%2FA%2FjKukFyHeW0aD8pYOB34Vth6PYuajZrFRg1Z93l03PSO7hY2Rr1OTFKAgRxViHojh19aV2lAoU9breFcgs0%3D&signData%5B18457977%5D=MIIP7gYJKoZIhvcNAQcCoIIP3zCCD9sCAQExDjAMBggqgw4DCgEDAQUAMAsGCSqGSIb3DQEHAaCCBNowggTWMIIEgKADAgECAhQsEW5waAgTAPIdPyOMUwKjbFTsfjANBgkqgw4DCgEBAQIFADBTMQswCQYDVQQGEwJLWjFEMEIGA1UEAww70rDQm9Ci0KLQq9KaINCa0KPTmNCb0JDQndCU0KvQoNCj0KjQqyDQntCg0KLQkNCb0KvSmiAoR09TVCkwHhcNMjEwNjA0MDc1MTU1WhcNMjIwNjA0MDc1MTU1WjCCAXUxKjAoBgNVBAMMIdCa0JDQmdCT0J7QoNCe0JTQptCV0JLQkCDQnNCQ0JnQrzEhMB8GA1UEBAwY0JrQkNCZ0JPQntCg0J7QlNCm0JXQktCQMRgwFgYDVQQFEw9JSU44MzA1MDQ0MDAwNDkxCzAJBgNVBAYTAktaMYGcMIGZBgNVBAoMgZHQotCe0JLQkNCg0JjQqdCV0KHQotCS0J4g0KEg0J7Qk9Cg0JDQndCY0KfQldCd0J3QntCZINCe0KLQktCV0KLQodCi0JLQldCd0J3QntCh0KLQrNCuICLQmNCd0JbQmNCd0JjQoNCY0J3Qk9Ce0JLQkNCvINCa0J7QnNCf0JDQndCY0K8gItCb0JjQlNCV0KAiMRgwFgYDVQQLDA9CSU4xMTA5NDAwMDQ5MDkxIzAhBgNVBCoMGtCh0KLQkNCd0JjQodCb0JDQktCe0JLQndCQMR8wHQYJKoZIhvcNAQkBFhBDRU9ATEVBREVSRU5HLktaMGwwJQYJKoMOAwoBAQEBMBgGCiqDDgMKAQEBAQEGCiqDDgMKAQMBAQADQwAEQOTfqVEh24kddh%2FaZlKhpjjqrTqnPx9OQicP3VjyFBJs193DrOnsUHVCnqkhqXSwisiXvQmmBA6PHMNg0hX%2BaSejggH2MIIB8jAOBgNVHQ8BAf8EBAMCBsAwMwYDVR0lBCwwKgYIKwYBBQUHAwQGCCqDDgMDBAECBgkqgw4DAwQBAgEGCSqDDgMDBAMCATAPBgNVHSMECDAGgARbanPpMB0GA1UdDgQWBBTeETtao0pNOO6INsPiri48jiPQbDBeBgNVHSAEVzBVMFMGByqDDgMDAgEwSDAhBggrBgEFBQcCARYVaHR0cDovL3BraS5nb3Yua3ovY3BzMCMGCCsGAQUFBwICMBcMFWh0dHA6Ly9wa2kuZ292Lmt6L2NwczBYBgNVHR8EUTBPME2gS6BJhiJodHRwOi8vY3JsLnBraS5nb3Yua3ovbmNhX2dvc3QuY3JshiNodHRwOi8vY3JsMS5wa2kuZ292Lmt6L25jYV9nb3N0LmNybDBcBgNVHS4EVTBTMFGgT6BNhiRodHRwOi8vY3JsLnBraS5nb3Yua3ovbmNhX2RfZ29zdC5jcmyGJWh0dHA6Ly9jcmwxLnBraS5nb3Yua3ovbmNhX2RfZ29zdC5jcmwwYwYIKwYBBQUHAQEEVzBVMC8GCCsGAQUFBzAChiNodHRwOi8vcGtpLmdvdi5rei9jZXJ0L25jYV9nb3N0LmNlcjAiBggrBgEFBQcwAYYWaHR0cDovL29jc3AucGtpLmdvdi5rejANBgkqgw4DCgEBAQIFAANBAF6nzMbWq5xVRWa5wASYXDr9StJo7NnVQSskyKrwhdnNDoRbtZuNoPiM5nt4FmOCS4qp5wLGCJADSDgT5k%2F0yDExggrZMIIK1QIBATBrMFMxCzAJBgNVBAYTAktaMUQwQgYDVQQDDDvSsNCb0KLQotCr0pog0JrQo9OY0JvQkNCd0JTQq9Cg0KPQqNCrINCe0KDQotCQ0JvQq9KaIChHT1NUKQIULBFucGgIEwDyHT8jjFMCo2xU7H4wDAYIKoMOAwoBAwEFAKCBojAYBgkqhkiG9w0BCQMxCwYJKoZIhvcNAQcBMBwGCSqGSIb3DQEJBTEPFw0yMjAzMjUxNDQxNDJaMC8GCSqGSIb3DQEJBDEiBCBfTZm2SguleHTRCPZ256RH7bOFR6sMpqLCbFmJ1Ug6MzA3BgsqhkiG9w0BCRACLzEoMCYwJDAiBCDtSxHdcyUCMLpJK8z8EZFN1Szhqk3bdjuIAn0o7cdZcTANBgkqgw4DCgEBAQIFAARAWzRWbC4xIWfWLPk1kN96TZtpzcCxvpym4pyHi339RB5%2FoiaYfJa5aJR8SxzBIx1ggfz4kIiVzBhEZ%2BZMFgvMu6GCCV0wgglZBgsqhkiG9w0BCRACDjGCCUgwgglEBgkqhkiG9w0BBwKgggk1MIIJMQIBAzEPMA0GCWCGSAFlAwQCAQUAMIGEBgsqhkiG9w0BCRABBKB1BHMwcQIBAQYIKoMOAwMCBgIwMTANBglghkgBZQMEAgEFAAQg5Fs2mty1bH2%2Bm%2BQDaWJcUuoNH2N9rfRfbr7NRD7Iq7ECFE3%2BopvxqvkhgkmnXDDcmto8r9rZGA8yMDIyMDMyNTE0NDE0MloCCAsNIuyea1DpoIIGXjCCBlowggRCoAMCAQICFD2d5W1fJ5xdBux9i4OqUL3rQ300MA0GCSqGSIb3DQEBCwUAMFIxCzAJBgNVBAYTAktaMUMwQQYDVQQDDDrSsNCb0KLQotCr0pog0JrQo9OY0JvQkNCd0JTQq9Cg0KPQqNCrINCe0KDQotCQ0JvQq9KaIChSU0EpMB4XDTE5MTIxMjA1MjIwNVoXDTIyMTIxMTA1MjIwNVowggESMRQwEgYDVQQDDAtUU0EgU0VSVklDRTEYMBYGA1UEBRMPSUlONzYxMjMxMzAwMzEzMQswCQYDVQQGEwJLWjEcMBoGA1UEBwwT0J3Qo9CgLdCh0KPQm9Ci0JDQnTEcMBoGA1UECAwT0J3Qo9CgLdCh0KPQm9Ci0JDQnTF9MHsGA1UECgx00JDQmtCm0JjQntCd0JXQoNCd0J7QlSDQntCR0KnQldCh0KLQktCeICLQndCQ0KbQmNCe0J3QkNCb0KzQndCr0JUg0JjQndCk0J7QoNCc0JDQptCY0J7QndCd0KvQlSDQotCV0KXQndCe0JvQntCT0JjQmCIxGDAWBgNVBAsMD0JJTjAwMDc0MDAwMDcyODCCASIwDQYJKoZIhvcNAQEBBQADggEPADCCAQoCggEBAIuFfpSCo5uenI8Cn9234hpG8uQZnRMBT2QwrndJaxIAHAMEQ9T3sxi2KWDL2mJArmVs48gdhhkm9JRVQCnYGb%2BOZ%2BnmePaeU826nawS83OVlJggIxbtAE4xdr8n2jCKMqjYgaQjXtllIC3NV5LvnZFeaoy%2Fndih%2Bo3iNbiSfEoWsX1xB8MW7ZHIqZBnHYOQnnpSW9TRejYF4B%2BFC4Sr70IXFRYHuicNJZo4V9ijGiacBA3WWOITxOPd0TZcd54G4kNFSSzocGiqMLVtWGNWBKzRtqxHPKdvoK2M3oZ5Ihqzjf0RnuS30HSQsXzGtYE1d7b7y%2BR5fr%2BpWdfp3C2ECdUCAwEAAaOCAWQwggFgMBYGA1UdJQEB%2FwQMMAoGCCsGAQUFBwMIMA8GA1UdIwQIMAaABFtqdBEwHQYDVR0OBBYEFFqrRbGX3k3FKok5zH%2FA2RUXLTKfMFYGA1UdHwRPME0wS6BJoEeGIWh0dHA6Ly9jcmwucGtpLmdvdi5rei9uY2FfcnNhLmNybIYiaHR0cDovL2NybDEucGtpLmdvdi5rei9uY2FfcnNhLmNybDBaBgNVHS4EUzBRME%2BgTaBLhiNodHRwOi8vY3JsLnBraS5nb3Yua3ovbmNhX2RfcnNhLmNybIYkaHR0cDovL2NybDEucGtpLmdvdi5rei9uY2FfZF9yc2EuY3JsMGIGCCsGAQUFBwEBBFYwVDAuBggrBgEFBQcwAoYiaHR0cDovL3BraS5nb3Yua3ovY2VydC9uY2FfcnNhLmNlcjAiBggrBgEFBQcwAYYWaHR0cDovL29jc3AucGtpLmdvdi5rejANBgkqhkiG9w0BAQsFAAOCAgEApPnpRXedzDgnZBPKJPhOJ8dWXzuNu65vSymgTH1hDXwE1lR90n5kZvu%2FkHO1shsId6TEg6X%2F1IE6Gp8992JsTezkxUexb6MdWWnUYjNhuPYzsYfBMG%2BZ%2BUMjMhf2ER5SUjDiEXrdwqKkRPF5HfzuKEdQfjoFctYveqirISBdEOUIGD0LCiqqTD7W5dAIQaVbJZLGCG5nKgZ8rvzYPyQJU3AqizXeHglOWLxINWmYs69NCqmvk6XI7oFVyVe8UTLQ%2BqMtl1MfrNTvSHtU2Uei2n2NKcN1oof5E2RDASf3ImNDeSJuFKzIXnwdc9xmesfUCBVJJLgt4HU6RHh%2B5StrR%2BWz7%2FOYBoVuhUE0YMSM%2FH74BrKbI6k8rmPw08QoU1v4cfYEH2jGzrJt4IephTkvxbv8ENcJX5LpUUux6oJHLTIzTU9q1BjitoODmV2EmmBn1hzU%2BYaSPEffIztXt8zuoVrH1gkT1s5bzc4nqc63OYur6XSTf6d66%2B2mavaVSQObNcYPz6YPnQFbDHzUy16oxX4FxlhnFSwul6gZg4b9kYh1BTdpXTGrfo4hD%2Fc%2F4xF3OHWxprGnWb9uFxJqthhHkwEHloMcZaR5xSLT7kE4Lpy5uRz9qh4DVcbSTw2sqCJx%2F57YioRy7GUssbhG51%2BYBnDG9OEF7xhV6xdvF%2FSVVvwxggIwMIICLAIBATBqMFIxCzAJBgNVBAYTAktaMUMwQQYDVQQDDDrSsNCb0KLQotCr0pog0JrQo9OY0JvQkNCd0JTQq9Cg0KPQqNCrINCe0KDQotCQ0JvQq9KaIChSU0EpAhQ9neVtXyecXQbsfYuDqlC960N9NDANBglghkgBZQMEAgEFAKCBmDAaBgkqhkiG9w0BCQMxDQYLKoZIhvcNAQkQAQQwHAYJKoZIhvcNAQkFMQ8XDTIyMDMyNTE0NDE0MlowKwYLKoZIhvcNAQkQAgwxHDAaMBgwFgQUyyVFl1vW4xC%2B7Z6ZVsJ4kYybLQgwLwYJKoZIhvcNAQkEMSIEIKXvcBvdzQ%2FWjKLzP6Qt8YeZwfJ1FmiZwHU5OAVvNCv2MA0GCSqGSIb3DQEBCwUABIIBAE8VVUU4SOujXRlsNrs5Q7KaTco9DnpAy11Z%2B7ir8fw7ZOXIoTSLVXgm1AmxWdZg9CTL50H%2FbhDC8IItBPgwEgKIOHiYwel3M7QLFGqL5IUVO%2BuDLY1Iw4Yp8z5%2BaBfn3CAppjWyZxS8CtkdiiQ5CsziIOjR5htjqcS%2FvolUbqiIO8Cp%2Bv4WSmDCnTEfra4%2BSZCG4aOEZytuVnLjQBIRchDsf3ovaMaPe4sWeEXi5ecREeKWDRRmgMioMqH%2FDm1ni1v0XsnLAmypt%2FuMPkeG9tTu0%2FQBm74QuHnoDHWru4HOM6amN65AZ%2BT6TtJS5ShAHEZGP%2B3vUZJZxALYrCCCo28%3D&formData=offer%255B34272171%255D%255B18457975%255D%255Bprice%255D%3D11%26offer%255B34272172%255D%255B18457976%255D%255Bprice%255D%3D22%26offer%255B34272173%255D%255B18457977%255D%255Bprice%255D%3D33'

In [ ]:
data = 'xmlData%5B18457975%5D=%3C%3Fxml+version%3D%221.0%22+encoding%3D%22UTF-8%22%3F%3E%3Cpriceoffer%3E%3Ctrd_buy_id%3E7196072%3C%2Ftrd_buy_id%3E%3Ctrd_buy_lot_id%3E19114250%3C%2Ftrd_buy_lot_id%3E%3Ctrd_buy_lot_point_id%3E18457975%3C%2Ftrd_buy_lot_point_id%3E%3Cprice%3E11%3C%2Fprice%3E%3C%2Fpriceoffer%3E&xmlData%5B18457976%5D=%3C%3Fxml+version%3D%221.0%22+encoding%3D%22UTF-8%22%3F%3E%3Cpriceoffer%3E%3Ctrd_buy_id%3E7196072%3C%2Ftrd_buy_id%3E%3Ctrd_buy_lot_id%3E19114251%3C%2Ftrd_buy_lot_id%3E%3Ctrd_buy_lot_point_id%3E18457976%3C%2Ftrd_buy_lot_point_id%3E%3Cprice%3E22%3C%2Fprice%3E%3C%2Fpriceoffer%3E&xmlData%5B18457977%5D=%3C%3Fxml+version%3D%221.0%22+encoding%3D%22UTF-8%22%3F%3E%3Cpriceoffer%3E%3Ctrd_buy_id%3E7196072%3C%2Ftrd_buy_id%3E%3Ctrd_buy_lot_id%3E19114252%3C%2Ftrd_buy_lot_id%3E%3Ctrd_buy_lot_point_id%3E18457977%3C%2Ftrd_buy_lot_point_id%3E%3Cprice%3E33%3C%2Fprice%3E%3C%2Fpriceoffer%3E&signData%5B18457975%5D=

MIIP7gYJKoZIhvcNAQcCoIIP3zCCD9sCAQExDjAMBggqgw4DCgEDAQUAMAsGCSqGSIb3DQEHAaCCBNowggTWMIIEgKADAgECAhQsEW5waAgTAPIdPyOMUwKjbFTsfjANBgkqgw4DCgEBAQIFADBTMQswCQYDVQQGEwJLWjFEMEIGA1UEAww70rDQm9Ci0KLQq9KaINCa0KPTmNCb0JDQndCU0KvQoNCj0KjQqyDQntCg0KLQkNCb0KvSmiAoR09TVCkwHhcNMjEwNjA0MDc1MTU1WhcNMjIwNjA0MDc1MTU1WjCCAXUxKjAoBgNVBAMMIdCa0JDQmdCT0J7QoNCe0JTQptCV0JLQkCDQnNCQ0JnQrzEhMB8GA1UEBAwY0JrQkNCZ0JPQntCg0J7QlNCm0JXQktCQMRgwFgYDVQQFEw9JSU44MzA1MDQ0MDAwNDkxCzAJBgNVBAYTAktaMYGcMIGZBgNVBAoMgZHQotCe0JLQkNCg0JjQqdCV0KHQotCS0J4g0KEg0J7Qk9Cg0JDQndCY0KfQldCd0J3QntCZINCe0KLQktCV0KLQodCi0JLQldCd0J3QntCh0KLQrNCuICLQmNCd0JbQmNCd0JjQoNCY0J3Qk9Ce0JLQkNCvINCa0J7QnNCf0JDQndCY0K8gItCb0JjQlNCV0KAiMRgwFgYDVQQLDA9CSU4xMTA5NDAwMDQ5MDkxIzAhBgNVBCoMGtCh0KLQkNCd0JjQodCb0JDQktCe0JLQndCQMR8wHQYJKoZIhvcNAQkBFhBDRU9ATEVBREVSRU5HLktaMGwwJQYJKoMOAwoBAQEBMBgGCiqDDgMKAQEBAQEGCiqDDgMKAQMBAQADQwAEQOTfqVEh24kddh%2FaZlKhpjjqrTqnPx9OQicP3VjyFBJs193DrOnsUHVCnqkhqXSwisiXvQmmBA6PHMNg0hX%2BaSejggH2MIIB8jAOBgNVHQ8BAf8EBAMCBsAwMwYDVR0lBCwwKgYIKwYBBQUHAwQGCCqDDgMDBAECBgkqgw4DAwQBAgEGCSqDDgMDBAMCATAPBgNVHSMECDAGgARbanPpMB0GA1UdDgQWBBTeETtao0pNOO6INsPiri48jiPQbDBeBgNVHSAEVzBVMFMGByqDDgMDAgEwSDAhBggrBgEFBQcCARYVaHR0cDovL3BraS5nb3Yua3ovY3BzMCMGCCsGAQUFBwICMBcMFWh0dHA6Ly9wa2kuZ292Lmt6L2NwczBYBgNVHR8EUTBPME2gS6BJhiJodHRwOi8vY3JsLnBraS5nb3Yua3ovbmNhX2dvc3QuY3JshiNodHRwOi8vY3JsMS5wa2kuZ292Lmt6L25jYV9nb3N0LmNybDBcBgNVHS4EVTBTMFGgT6BNhiRodHRwOi8vY3JsLnBraS5nb3Yua3ovbmNhX2RfZ29zdC5jcmyGJWh0dHA6Ly9jcmwxLnBraS5nb3Yua3ovbmNhX2RfZ29zdC5jcmwwYwYIKwYBBQUHAQEEVzBVMC8GCCsGAQUFBzAChiNodHRwOi8vcGtpLmdvdi5rei9jZXJ0L25jYV9nb3N0LmNlcjAiBggrBgEFBQcwAYYWaHR0cDovL29jc3AucGtpLmdvdi5rejANBgkqgw4DCgEBAQIFAANBAF6nzMbWq5xVRWa5wASYXDr9StJo7NnVQSskyKrwhdnNDoRbtZuNoPiM5nt4FmOCS4qp5wLGCJADSDgT5k%2F0yDExggrZMIIK1QIBATBrMFMxCzAJBgNVBAYTAktaMUQwQgYDVQQDDDvSsNCb0KLQotCr0pog0JrQo9OY0JvQkNCd0JTQq9Cg0KPQqNCrINCe0KDQotCQ0JvQq9KaIChHT1NUKQIULBFucGgIEwDyHT8jjFMCo2xU7H4wDAYIKoMOAwoBAwEFAKCBojAYBgkqhkiG9w0BCQMxCwYJKoZIhvcNAQcBMBwGCSqGSIb3DQEJBTEPFw0yMjAzMjUxNDQxNDJaMC8GCSqGSIb3DQEJBDEiBCB8qjCGZVAshUyFnm7LwZdksVaMcWO7JSLPXDYuSYXOezA3BgsqhkiG9w0BCRACLzEoMCYwJDAiBCDtSxHdcyUCMLpJK8z8EZFN1Szhqk3bdjuIAn0o7cdZcTANBgkqgw4DCgEBAQIFAARA2lFpl3CCwbWu0Wjqc1xKrMSwFF63Ns1azC1O3wef%2Flr95m5J2Lj7DoDa%2FpQXcLpeiS8gEHdybm4D6pO80RNVnaGCCV0wgglZBgsqhkiG9w0BCRACDjGCCUgwgglEBgkqhkiG9w0BBwKgggk1MIIJMQIBAzEPMA0GCWCGSAFlAwQCAQUAMIGEBgsqhkiG9w0BCRABBKB1BHMwcQIBAQYIKoMOAwMCBgIwMTANBglghkgBZQMEAgEFAAQgRYxRFBpofODghuXC0QFXNTuphL8MZLwFlfYzy%2BITU0kCFLjkV3bRy2zEoWnYReuD%2BIxxSBrtGA8yMDIyMDMyNTE0NDE0M1oCCD1E6MIDnqBeoIIGXjCCBlowggRCoAMCAQICFD2d5W1fJ5xdBux9i4OqUL3rQ300MA0GCSqGSIb3DQEBCwUAMFIxCzAJBgNVBAYTAktaMUMwQQYDVQQDDDrSsNCb0KLQotCr0pog0JrQo9OY0JvQkNCd0JTQq9Cg0KPQqNCrINCe0KDQotCQ0JvQq9KaIChSU0EpMB4XDTE5MTIxMjA1MjIwNVoXDTIyMTIxMTA1MjIwNVowggESMRQwEgYDVQQDDAtUU0EgU0VSVklDRTEYMBYGA1UEBRMPSUlONzYxMjMxMzAwMzEzMQswCQYDVQQGEwJLWjEcMBoGA1UEBwwT0J3Qo9CgLdCh0KPQm9Ci0JDQnTEcMBoGA1UECAwT0J3Qo9CgLdCh0KPQm9Ci0JDQnTF9MHsGA1UECgx00JDQmtCm0JjQntCd0JXQoNCd0J7QlSDQntCR0KnQldCh0KLQktCeICLQndCQ0KbQmNCe0J3QkNCb0KzQndCr0JUg0JjQndCk0J7QoNCc0JDQptCY0J7QndCd0KvQlSDQotCV0KXQndCe0JvQntCT0JjQmCIxGDAWBgNVBAsMD0JJTjAwMDc0MDAwMDcyODCCASIwDQYJKoZIhvcNAQEBBQADggEPADCCAQoCggEBAIuFfpSCo5uenI8Cn9234hpG8uQZnRMBT2QwrndJaxIAHAMEQ9T3sxi2KWDL2mJArmVs48gdhhkm9JRVQCnYGb%2BOZ%2BnmePaeU826nawS83OVlJggIxbtAE4xdr8n2jCKMqjYgaQjXtllIC3NV5LvnZFeaoy%2Fndih%2Bo3iNbiSfEoWsX1xB8MW7ZHIqZBnHYOQnnpSW9TRejYF4B%2BFC4Sr70IXFRYHuicNJZo4V9ijGiacBA3WWOITxOPd0TZcd54G4kNFSSzocGiqMLVtWGNWBKzRtqxHPKdvoK2M3oZ5Ihqzjf0RnuS30HSQsXzGtYE1d7b7y%2BR5fr%2BpWdfp3C2ECdUCAwEAAaOCAWQwggFgMBYGA1UdJQEB%2FwQMMAoGCCsGAQUFBwMIMA8GA1UdIwQIMAaABFtqdBEwHQYDVR0OBBYEFFqrRbGX3k3FKok5zH%2FA2RUXLTKfMFYGA1UdHwRPME0wS6BJoEeGIWh0dHA6Ly9jcmwucGtpLmdvdi5rei9uY2FfcnNhLmNybIYiaHR0cDovL2NybDEucGtpLmdvdi5rei9uY2FfcnNhLmNybDBaBgNVHS4EUzBRME%2BgTaBLhiNodHRwOi8vY3JsLnBraS5nb3Yua3ovbmNhX2RfcnNhLmNybIYkaHR0cDovL2NybDEucGtpLmdvdi5rei9uY2FfZF9yc2EuY3JsMGIGCCsGAQUFBwEBBFYwVDAuBggrBgEFBQcwAoYiaHR0cDovL3BraS5nb3Yua3ovY2VydC9uY2FfcnNhLmNlcjAiBggrBgEFBQcwAYYWaHR0cDovL29jc3AucGtpLmdvdi5rejANBgkqhkiG9w0BAQsFAAOCAgEApPnpRXedzDgnZBPKJPhOJ8dWXzuNu65vSymgTH1hDXwE1lR90n5kZvu%2FkHO1shsId6TEg6X%2F1IE6Gp8992JsTezkxUexb6MdWWnUYjNhuPYzsYfBMG%2BZ%2BUMjMhf2ER5SUjDiEXrdwqKkRPF5HfzuKEdQfjoFctYveqirISBdEOUIGD0LCiqqTD7W5dAIQaVbJZLGCG5nKgZ8rvzYPyQJU3AqizXeHglOWLxINWmYs69NCqmvk6XI7oFVyVe8UTLQ%2BqMtl1MfrNTvSHtU2Uei2n2NKcN1oof5E2RDASf3ImNDeSJuFKzIXnwdc9xmesfUCBVJJLgt4HU6RHh%2B5StrR%2BWz7%2FOYBoVuhUE0YMSM%2FH74BrKbI6k8rmPw08QoU1v4cfYEH2jGzrJt4IephTkvxbv8ENcJX5LpUUux6oJHLTIzTU9q1BjitoODmV2EmmBn1hzU%2BYaSPEffIztXt8zuoVrH1gkT1s5bzc4nqc63OYur6XSTf6d66%2B2mavaVSQObNcYPz6YPnQFbDHzUy16oxX4FxlhnFSwul6gZg4b9kYh1BTdpXTGrfo4hD%2Fc%2F4xF3OHWxprGnWb9uFxJqthhHkwEHloMcZaR5xSLT7kE4Lpy5uRz9qh4DVcbSTw2sqCJx%2F57YioRy7GUssbhG51%2BYBnDG9OEF7xhV6xdvF%2FSVVvwxggIwMIICLAIBATBqMFIxCzAJBgNVBAYTAktaMUMwQQYDVQQDDDrSsNCb0KLQotCr0pog0JrQo9OY0JvQkNCd0JTQq9Cg0KPQqNCrINCe0KDQotCQ0JvQq9KaIChSU0EpAhQ9neVtXyecXQbsfYuDqlC960N9NDANBglghkgBZQMEAgEFAKCBmDAaBgkqhkiG9w0BCQMxDQYLKoZIhvcNAQkQAQQwHAYJKoZIhvcNAQkFMQ8XDTIyMDMyNTE0NDE0M1owKwYLKoZIhvcNAQkQAgwxHDAaMBgwFgQUyyVFl1vW4xC%2B7Z6ZVsJ4kYybLQgwLwYJKoZIhvcNAQkEMSIEIMpzqR3sHGmOGu0XJVHsXFw6dVa3w8ktEK9f353qhdRFMA0GCSqGSIb3DQEBCwUABIIBAD3GFSl6rkv1UraXHbCbYGNaPwT0%2FtqAEMH1JgWMa2v8CQiyXERifjSTz%2B0RuKt6JDbvbRW0FWwjwVPdF4Rn79Xl2MubC7kwGkbgoYwKZqDGUxj%2Bkr3Ji0eW0C5n4HAHY%2FsHFa3A84oAleCUExFVnVnYgeqXIpy27tQ19PqcudRDsRgMTTprak8nhq8NEa6DVd83PCsSEgpKvUEmkP55wv3tCV8eBC49owswvWNKQfycr8VR1ihAvuFqdD9dPbDaCU9ArUjgIHxv9gM6%2BEdIKK9Nht8QL6K1ySli7VHOPiqUFmVtXmTgpHXeJRYNvEbw6o1%2BRFn6WYVMbwBwijOCzMc


%3D&signData%5B18457976%5D=MIIP7gYJKoZIhvcNAQcCoIIP3zCCD9sCAQExDjAMBggqgw4DCgEDAQUAMAsGCSqGSIb3DQEHAaCCBNowggTWMIIEgKADAgECAhQsEW5waAgTAPIdPyOMUwKjbFTsfjANBgkqgw4DCgEBAQIFADBTMQswCQYDVQQGEwJLWjFEMEIGA1UEAww70rDQm9Ci0KLQq9KaINCa0KPTmNCb0JDQndCU0KvQoNCj0KjQqyDQntCg0KLQkNCb0KvSmiAoR09TVCkwHhcNMjEwNjA0MDc1MTU1WhcNMjIwNjA0MDc1MTU1WjCCAXUxKjAoBgNVBAMMIdCa0JDQmdCT0J7QoNCe0JTQptCV0JLQkCDQnNCQ0JnQrzEhMB8GA1UEBAwY0JrQkNCZ0JPQntCg0J7QlNCm0JXQktCQMRgwFgYDVQQFEw9JSU44MzA1MDQ0MDAwNDkxCzAJBgNVBAYTAktaMYGcMIGZBgNVBAoMgZHQotCe0JLQkNCg0JjQqdCV0KHQotCS0J4g0KEg0J7Qk9Cg0JDQndCY0KfQldCd0J3QntCZINCe0KLQktCV0KLQodCi0JLQldCd0J3QntCh0KLQrNCuICLQmNCd0JbQmNCd0JjQoNCY0J3Qk9Ce0JLQkNCvINCa0J7QnNCf0JDQndCY0K8gItCb0JjQlNCV0KAiMRgwFgYDVQQLDA9CSU4xMTA5NDAwMDQ5MDkxIzAhBgNVBCoMGtCh0KLQkNCd0JjQodCb0JDQktCe0JLQndCQMR8wHQYJKoZIhvcNAQkBFhBDRU9ATEVBREVSRU5HLktaMGwwJQYJKoMOAwoBAQEBMBgGCiqDDgMKAQEBAQEGCiqDDgMKAQMBAQADQwAEQOTfqVEh24kddh%2FaZlKhpjjqrTqnPx9OQicP3VjyFBJs193DrOnsUHVCnqkhqXSwisiXvQmmBA6PHMNg0hX%2BaSejggH2MIIB8jAOBgNVHQ8BAf8EBAMCBsAwMwYDVR0lBCwwKgYIKwYBBQUHAwQGCCqDDgMDBAECBgkqgw4DAwQBAgEGCSqDDgMDBAMCATAPBgNVHSMECDAGgARbanPpMB0GA1UdDgQWBBTeETtao0pNOO6INsPiri48jiPQbDBeBgNVHSAEVzBVMFMGByqDDgMDAgEwSDAhBggrBgEFBQcCARYVaHR0cDovL3BraS5nb3Yua3ovY3BzMCMGCCsGAQUFBwICMBcMFWh0dHA6Ly9wa2kuZ292Lmt6L2NwczBYBgNVHR8EUTBPME2gS6BJhiJodHRwOi8vY3JsLnBraS5nb3Yua3ovbmNhX2dvc3QuY3JshiNodHRwOi8vY3JsMS5wa2kuZ292Lmt6L25jYV9nb3N0LmNybDBcBgNVHS4EVTBTMFGgT6BNhiRodHRwOi8vY3JsLnBraS5nb3Yua3ovbmNhX2RfZ29zdC5jcmyGJWh0dHA6Ly9jcmwxLnBraS5nb3Yua3ovbmNhX2RfZ29zdC5jcmwwYwYIKwYBBQUHAQEEVzBVMC8GCCsGAQUFBzAChiNodHRwOi8vcGtpLmdvdi5rei9jZXJ0L25jYV9nb3N0LmNlcjAiBggrBgEFBQcwAYYWaHR0cDovL29jc3AucGtpLmdvdi5rejANBgkqgw4DCgEBAQIFAANBAF6nzMbWq5xVRWa5wASYXDr9StJo7NnVQSskyKrwhdnNDoRbtZuNoPiM5nt4FmOCS4qp5wLGCJADSDgT5k%2F0yDExggrZMIIK1QIBATBrMFMxCzAJBgNVBAYTAktaMUQwQgYDVQQDDDvSsNCb0KLQotCr0pog0JrQo9OY0JvQkNCd0JTQq9Cg0KPQqNCrINCe0KDQotCQ0JvQq9KaIChHT1NUKQIULBFucGgIEwDyHT8jjFMCo2xU7H4wDAYIKoMOAwoBAwEFAKCBojAYBgkqhkiG9w0BCQMxCwYJKoZIhvcNAQcBMBwGCSqGSIb3DQEJBTEPFw0yMjAzMjUxNDQxNDJaMC8GCSqGSIb3DQEJBDEiBCAqeLIjxykElKrkpwrNwjWQhUDh4ovQUGWJuFi%2B7SvJ8TA3BgsqhkiG9w0BCRACLzEoMCYwJDAiBCDtSxHdcyUCMLpJK8z8EZFN1Szhqk3bdjuIAn0o7cdZcTANBgkqgw4DCgEBAQIFAARAeFZL2JKD5jMhgVFJfHnxE%2FWkD3uvYNhLTsDR3MIyIaU%2BkVKfP%2Fq0BU%2BBgBzzSeAeaj1g6C0ZVlDwqLRzrpcQSKGCCV0wgglZBgsqhkiG9w0BCRACDjGCCUgwgglEBgkqhkiG9w0BBwKgggk1MIIJMQIBAzEPMA0GCWCGSAFlAwQCAQUAMIGEBgsqhkiG9w0BCRABBKB1BHMwcQIBAQYIKoMOAwMCBgIwMTANBglghkgBZQMEAgEFAAQg%2BloS2y0zZuq6WAwqn1QIe4%2FLbyvpRJ0FqeaPGELE%2F%2FECFMzDWElhYKIYp7Z%2B7DGhconFiNp0GA8yMDIyMDMyNTE0NDE0MloCCGvZCpY1vVFKoIIGXjCCBlowggRCoAMCAQICFD2d5W1fJ5xdBux9i4OqUL3rQ300MA0GCSqGSIb3DQEBCwUAMFIxCzAJBgNVBAYTAktaMUMwQQYDVQQDDDrSsNCb0KLQotCr0pog0JrQo9OY0JvQkNCd0JTQq9Cg0KPQqNCrINCe0KDQotCQ0JvQq9KaIChSU0EpMB4XDTE5MTIxMjA1MjIwNVoXDTIyMTIxMTA1MjIwNVowggESMRQwEgYDVQQDDAtUU0EgU0VSVklDRTEYMBYGA1UEBRMPSUlONzYxMjMxMzAwMzEzMQswCQYDVQQGEwJLWjEcMBoGA1UEBwwT0J3Qo9CgLdCh0KPQm9Ci0JDQnTEcMBoGA1UECAwT0J3Qo9CgLdCh0KPQm9Ci0JDQnTF9MHsGA1UECgx00JDQmtCm0JjQntCd0JXQoNCd0J7QlSDQntCR0KnQldCh0KLQktCeICLQndCQ0KbQmNCe0J3QkNCb0KzQndCr0JUg0JjQndCk0J7QoNCc0JDQptCY0J7QndCd0KvQlSDQotCV0KXQndCe0JvQntCT0JjQmCIxGDAWBgNVBAsMD0JJTjAwMDc0MDAwMDcyODCCASIwDQYJKoZIhvcNAQEBBQADggEPADCCAQoCggEBAIuFfpSCo5uenI8Cn9234hpG8uQZnRMBT2QwrndJaxIAHAMEQ9T3sxi2KWDL2mJArmVs48gdhhkm9JRVQCnYGb%2BOZ%2BnmePaeU826nawS83OVlJggIxbtAE4xdr8n2jCKMqjYgaQjXtllIC3NV5LvnZFeaoy%2Fndih%2Bo3iNbiSfEoWsX1xB8MW7ZHIqZBnHYOQnnpSW9TRejYF4B%2BFC4Sr70IXFRYHuicNJZo4V9ijGiacBA3WWOITxOPd0TZcd54G4kNFSSzocGiqMLVtWGNWBKzRtqxHPKdvoK2M3oZ5Ihqzjf0RnuS30HSQsXzGtYE1d7b7y%2BR5fr%2BpWdfp3C2ECdUCAwEAAaOCAWQwggFgMBYGA1UdJQEB%2FwQMMAoGCCsGAQUFBwMIMA8GA1UdIwQIMAaABFtqdBEwHQYDVR0OBBYEFFqrRbGX3k3FKok5zH%2FA2RUXLTKfMFYGA1UdHwRPME0wS6BJoEeGIWh0dHA6Ly9jcmwucGtpLmdvdi5rei9uY2FfcnNhLmNybIYiaHR0cDovL2NybDEucGtpLmdvdi5rei9uY2FfcnNhLmNybDBaBgNVHS4EUzBRME%2BgTaBLhiNodHRwOi8vY3JsLnBraS5nb3Yua3ovbmNhX2RfcnNhLmNybIYkaHR0cDovL2NybDEucGtpLmdvdi5rei9uY2FfZF9yc2EuY3JsMGIGCCsGAQUFBwEBBFYwVDAuBggrBgEFBQcwAoYiaHR0cDovL3BraS5nb3Yua3ovY2VydC9uY2FfcnNhLmNlcjAiBggrBgEFBQcwAYYWaHR0cDovL29jc3AucGtpLmdvdi5rejANBgkqhkiG9w0BAQsFAAOCAgEApPnpRXedzDgnZBPKJPhOJ8dWXzuNu65vSymgTH1hDXwE1lR90n5kZvu%2FkHO1shsId6TEg6X%2F1IE6Gp8992JsTezkxUexb6MdWWnUYjNhuPYzsYfBMG%2BZ%2BUMjMhf2ER5SUjDiEXrdwqKkRPF5HfzuKEdQfjoFctYveqirISBdEOUIGD0LCiqqTD7W5dAIQaVbJZLGCG5nKgZ8rvzYPyQJU3AqizXeHglOWLxINWmYs69NCqmvk6XI7oFVyVe8UTLQ%2BqMtl1MfrNTvSHtU2Uei2n2NKcN1oof5E2RDASf3ImNDeSJuFKzIXnwdc9xmesfUCBVJJLgt4HU6RHh%2B5StrR%2BWz7%2FOYBoVuhUE0YMSM%2FH74BrKbI6k8rmPw08QoU1v4cfYEH2jGzrJt4IephTkvxbv8ENcJX5LpUUux6oJHLTIzTU9q1BjitoODmV2EmmBn1hzU%2BYaSPEffIztXt8zuoVrH1gkT1s5bzc4nqc63OYur6XSTf6d66%2B2mavaVSQObNcYPz6YPnQFbDHzUy16oxX4FxlhnFSwul6gZg4b9kYh1BTdpXTGrfo4hD%2Fc%2F4xF3OHWxprGnWb9uFxJqthhHkwEHloMcZaR5xSLT7kE4Lpy5uRz9qh4DVcbSTw2sqCJx%2F57YioRy7GUssbhG51%2BYBnDG9OEF7xhV6xdvF%2FSVVvwxggIwMIICLAIBATBqMFIxCzAJBgNVBAYTAktaMUMwQQYDVQQDDDrSsNCb0KLQotCr0pog0JrQo9OY0JvQkNCd0JTQq9Cg0KPQqNCrINCe0KDQotCQ0JvQq9KaIChSU0EpAhQ9neVtXyecXQbsfYuDqlC960N9NDANBglghkgBZQMEAgEFAKCBmDAaBgkqhkiG9w0BCQMxDQYLKoZIhvcNAQkQAQQwHAYJKoZIhvcNAQkFMQ8XDTIyMDMyNTE0NDE0MlowKwYLKoZIhvcNAQkQAgwxHDAaMBgwFgQUyyVFl1vW4xC%2B7Z6ZVsJ4kYybLQgwLwYJKoZIhvcNAQkEMSIEIPnmuHkbKsQRwwhb6RxZ4k%2FNGGAj%2Bvfr5a7bTsg8D%2FELMA0GCSqGSIb3DQEBCwUABIIBAEt1WmKBKyKabCPR4m90QbLpSZ%2B%2FZjYDug3A3XDjQSNa7rbsKOUoJBu2r4ObcsSDNCfY9HN24o6EOnqnx2bPIzThTFgylzix09DxP9VhGvAoVhqAfdcKGjN6oNAxuDDvTv%2Bhahc1gnzM0Q0gI2EIIGb2A1BWE86opiUUw%2BuV9hf9saWDredTAhll2CzNje2IrTwyf51zp%2BA5wZ%2Fjlm0bzh7deY%2FEQ13yb%2Fyjc94cCE%2FZ1Ijq63F6mwTl1EAehXMFZD%2FA%2FjKukFyHeW0aD8pYOB34Vth6PYuajZrFRg1Z93l03PSO7hY2Rr1OTFKAgRxViHojh19aV2lAoU9breFcgs0%3D&signData%5B18457977%5D=MIIP7gYJKoZIhvcNAQcCoIIP3zCCD9sCAQExDjAMBggqgw4DCgEDAQUAMAsGCSqGSIb3DQEHAaCCBNowggTWMIIEgKADAgECAhQsEW5waAgTAPIdPyOMUwKjbFTsfjANBgkqgw4DCgEBAQIFADBTMQswCQYDVQQGEwJLWjFEMEIGA1UEAww70rDQm9Ci0KLQq9KaINCa0KPTmNCb0JDQndCU0KvQoNCj0KjQqyDQntCg0KLQkNCb0KvSmiAoR09TVCkwHhcNMjEwNjA0MDc1MTU1WhcNMjIwNjA0MDc1MTU1WjCCAXUxKjAoBgNVBAMMIdCa0JDQmdCT0J7QoNCe0JTQptCV0JLQkCDQnNCQ0JnQrzEhMB8GA1UEBAwY0JrQkNCZ0JPQntCg0J7QlNCm0JXQktCQMRgwFgYDVQQFEw9JSU44MzA1MDQ0MDAwNDkxCzAJBgNVBAYTAktaMYGcMIGZBgNVBAoMgZHQotCe0JLQkNCg0JjQqdCV0KHQotCS0J4g0KEg0J7Qk9Cg0JDQndCY0KfQldCd0J3QntCZINCe0KLQktCV0KLQodCi0JLQldCd0J3QntCh0KLQrNCuICLQmNCd0JbQmNCd0JjQoNCY0J3Qk9Ce0JLQkNCvINCa0J7QnNCf0JDQndCY0K8gItCb0JjQlNCV0KAiMRgwFgYDVQQLDA9CSU4xMTA5NDAwMDQ5MDkxIzAhBgNVBCoMGtCh0KLQkNCd0JjQodCb0JDQktCe0JLQndCQMR8wHQYJKoZIhvcNAQkBFhBDRU9ATEVBREVSRU5HLktaMGwwJQYJKoMOAwoBAQEBMBgGCiqDDgMKAQEBAQEGCiqDDgMKAQMBAQADQwAEQOTfqVEh24kddh%2FaZlKhpjjqrTqnPx9OQicP3VjyFBJs193DrOnsUHVCnqkhqXSwisiXvQmmBA6PHMNg0hX%2BaSejggH2MIIB8jAOBgNVHQ8BAf8EBAMCBsAwMwYDVR0lBCwwKgYIKwYBBQUHAwQGCCqDDgMDBAECBgkqgw4DAwQBAgEGCSqDDgMDBAMCATAPBgNVHSMECDAGgARbanPpMB0GA1UdDgQWBBTeETtao0pNOO6INsPiri48jiPQbDBeBgNVHSAEVzBVMFMGByqDDgMDAgEwSDAhBggrBgEFBQcCARYVaHR0cDovL3BraS5nb3Yua3ovY3BzMCMGCCsGAQUFBwICMBcMFWh0dHA6Ly9wa2kuZ292Lmt6L2NwczBYBgNVHR8EUTBPME2gS6BJhiJodHRwOi8vY3JsLnBraS5nb3Yua3ovbmNhX2dvc3QuY3JshiNodHRwOi8vY3JsMS5wa2kuZ292Lmt6L25jYV9nb3N0LmNybDBcBgNVHS4EVTBTMFGgT6BNhiRodHRwOi8vY3JsLnBraS5nb3Yua3ovbmNhX2RfZ29zdC5jcmyGJWh0dHA6Ly9jcmwxLnBraS5nb3Yua3ovbmNhX2RfZ29zdC5jcmwwYwYIKwYBBQUHAQEEVzBVMC8GCCsGAQUFBzAChiNodHRwOi8vcGtpLmdvdi5rei9jZXJ0L25jYV9nb3N0LmNlcjAiBggrBgEFBQcwAYYWaHR0cDovL29jc3AucGtpLmdvdi5rejANBgkqgw4DCgEBAQIFAANBAF6nzMbWq5xVRWa5wASYXDr9StJo7NnVQSskyKrwhdnNDoRbtZuNoPiM5nt4FmOCS4qp5wLGCJADSDgT5k%2F0yDExggrZMIIK1QIBATBrMFMxCzAJBgNVBAYTAktaMUQwQgYDVQQDDDvSsNCb0KLQotCr0pog0JrQo9OY0JvQkNCd0JTQq9Cg0KPQqNCrINCe0KDQotCQ0JvQq9KaIChHT1NUKQIULBFucGgIEwDyHT8jjFMCo2xU7H4wDAYIKoMOAwoBAwEFAKCBojAYBgkqhkiG9w0BCQMxCwYJKoZIhvcNAQcBMBwGCSqGSIb3DQEJBTEPFw0yMjAzMjUxNDQxNDJaMC8GCSqGSIb3DQEJBDEiBCBfTZm2SguleHTRCPZ256RH7bOFR6sMpqLCbFmJ1Ug6MzA3BgsqhkiG9w0BCRACLzEoMCYwJDAiBCDtSxHdcyUCMLpJK8z8EZFN1Szhqk3bdjuIAn0o7cdZcTANBgkqgw4DCgEBAQIFAARAWzRWbC4xIWfWLPk1kN96TZtpzcCxvpym4pyHi339RB5%2FoiaYfJa5aJR8SxzBIx1ggfz4kIiVzBhEZ%2BZMFgvMu6GCCV0wgglZBgsqhkiG9w0BCRACDjGCCUgwgglEBgkqhkiG9w0BBwKgggk1MIIJMQIBAzEPMA0GCWCGSAFlAwQCAQUAMIGEBgsqhkiG9w0BCRABBKB1BHMwcQIBAQYIKoMOAwMCBgIwMTANBglghkgBZQMEAgEFAAQg5Fs2mty1bH2%2Bm%2BQDaWJcUuoNH2N9rfRfbr7NRD7Iq7ECFE3%2BopvxqvkhgkmnXDDcmto8r9rZGA8yMDIyMDMyNTE0NDE0MloCCAsNIuyea1DpoIIGXjCCBlowggRCoAMCAQICFD2d5W1fJ5xdBux9i4OqUL3rQ300MA0GCSqGSIb3DQEBCwUAMFIxCzAJBgNVBAYTAktaMUMwQQYDVQQDDDrSsNCb0KLQotCr0pog0JrQo9OY0JvQkNCd0JTQq9Cg0KPQqNCrINCe0KDQotCQ0JvQq9KaIChSU0EpMB4XDTE5MTIxMjA1MjIwNVoXDTIyMTIxMTA1MjIwNVowggESMRQwEgYDVQQDDAtUU0EgU0VSVklDRTEYMBYGA1UEBRMPSUlONzYxMjMxMzAwMzEzMQswCQYDVQQGEwJLWjEcMBoGA1UEBwwT0J3Qo9CgLdCh0KPQm9Ci0JDQnTEcMBoGA1UECAwT0J3Qo9CgLdCh0KPQm9Ci0JDQnTF9MHsGA1UECgx00JDQmtCm0JjQntCd0JXQoNCd0J7QlSDQntCR0KnQldCh0KLQktCeICLQndCQ0KbQmNCe0J3QkNCb0KzQndCr0JUg0JjQndCk0J7QoNCc0JDQptCY0J7QndCd0KvQlSDQotCV0KXQndCe0JvQntCT0JjQmCIxGDAWBgNVBAsMD0JJTjAwMDc0MDAwMDcyODCCASIwDQYJKoZIhvcNAQEBBQADggEPADCCAQoCggEBAIuFfpSCo5uenI8Cn9234hpG8uQZnRMBT2QwrndJaxIAHAMEQ9T3sxi2KWDL2mJArmVs48gdhhkm9JRVQCnYGb%2BOZ%2BnmePaeU826nawS83OVlJggIxbtAE4xdr8n2jCKMqjYgaQjXtllIC3NV5LvnZFeaoy%2Fndih%2Bo3iNbiSfEoWsX1xB8MW7ZHIqZBnHYOQnnpSW9TRejYF4B%2BFC4Sr70IXFRYHuicNJZo4V9ijGiacBA3WWOITxOPd0TZcd54G4kNFSSzocGiqMLVtWGNWBKzRtqxHPKdvoK2M3oZ5Ihqzjf0RnuS30HSQsXzGtYE1d7b7y%2BR5fr%2BpWdfp3C2ECdUCAwEAAaOCAWQwggFgMBYGA1UdJQEB%2FwQMMAoGCCsGAQUFBwMIMA8GA1UdIwQIMAaABFtqdBEwHQYDVR0OBBYEFFqrRbGX3k3FKok5zH%2FA2RUXLTKfMFYGA1UdHwRPME0wS6BJoEeGIWh0dHA6Ly9jcmwucGtpLmdvdi5rei9uY2FfcnNhLmNybIYiaHR0cDovL2NybDEucGtpLmdvdi5rei9uY2FfcnNhLmNybDBaBgNVHS4EUzBRME%2BgTaBLhiNodHRwOi8vY3JsLnBraS5nb3Yua3ovbmNhX2RfcnNhLmNybIYkaHR0cDovL2NybDEucGtpLmdvdi5rei9uY2FfZF9yc2EuY3JsMGIGCCsGAQUFBwEBBFYwVDAuBggrBgEFBQcwAoYiaHR0cDovL3BraS5nb3Yua3ovY2VydC9uY2FfcnNhLmNlcjAiBggrBgEFBQcwAYYWaHR0cDovL29jc3AucGtpLmdvdi5rejANBgkqhkiG9w0BAQsFAAOCAgEApPnpRXedzDgnZBPKJPhOJ8dWXzuNu65vSymgTH1hDXwE1lR90n5kZvu%2FkHO1shsId6TEg6X%2F1IE6Gp8992JsTezkxUexb6MdWWnUYjNhuPYzsYfBMG%2BZ%2BUMjMhf2ER5SUjDiEXrdwqKkRPF5HfzuKEdQfjoFctYveqirISBdEOUIGD0LCiqqTD7W5dAIQaVbJZLGCG5nKgZ8rvzYPyQJU3AqizXeHglOWLxINWmYs69NCqmvk6XI7oFVyVe8UTLQ%2BqMtl1MfrNTvSHtU2Uei2n2NKcN1oof5E2RDASf3ImNDeSJuFKzIXnwdc9xmesfUCBVJJLgt4HU6RHh%2B5StrR%2BWz7%2FOYBoVuhUE0YMSM%2FH74BrKbI6k8rmPw08QoU1v4cfYEH2jGzrJt4IephTkvxbv8ENcJX5LpUUux6oJHLTIzTU9q1BjitoODmV2EmmBn1hzU%2BYaSPEffIztXt8zuoVrH1gkT1s5bzc4nqc63OYur6XSTf6d66%2B2mavaVSQObNcYPz6YPnQFbDHzUy16oxX4FxlhnFSwul6gZg4b9kYh1BTdpXTGrfo4hD%2Fc%2F4xF3OHWxprGnWb9uFxJqthhHkwEHloMcZaR5xSLT7kE4Lpy5uRz9qh4DVcbSTw2sqCJx%2F57YioRy7GUssbhG51%2BYBnDG9OEF7xhV6xdvF%2FSVVvwxggIwMIICLAIBATBqMFIxCzAJBgNVBAYTAktaMUMwQQYDVQQDDDrSsNCb0KLQotCr0pog0JrQo9OY0JvQkNCd0JTQq9Cg0KPQqNCrINCe0KDQotCQ0JvQq9KaIChSU0EpAhQ9neVtXyecXQbsfYuDqlC960N9NDANBglghkgBZQMEAgEFAKCBmDAaBgkqhkiG9w0BCQMxDQYLKoZIhvcNAQkQAQQwHAYJKoZIhvcNAQkFMQ8XDTIyMDMyNTE0NDE0MlowKwYLKoZIhvcNAQkQAgwxHDAaMBgwFgQUyyVFl1vW4xC%2B7Z6ZVsJ4kYybLQgwLwYJKoZIhvcNAQkEMSIEIKXvcBvdzQ%2FWjKLzP6Qt8YeZwfJ1FmiZwHU5OAVvNCv2MA0GCSqGSIb3DQEBCwUABIIBAE8VVUU4SOujXRlsNrs5Q7KaTco9DnpAy11Z%2B7ir8fw7ZOXIoTSLVXgm1AmxWdZg9CTL50H%2FbhDC8IItBPgwEgKIOHiYwel3M7QLFGqL5IUVO%2BuDLY1Iw4Yp8z5%2BaBfn3CAppjWyZxS8CtkdiiQ5CsziIOjR5htjqcS%2FvolUbqiIO8Cp%2Bv4WSmDCnTEfra4%2BSZCG4aOEZytuVnLjQBIRchDsf3ovaMaPe4sWeEXi5ecREeKWDRRmgMioMqH%2FDm1ni1v0XsnLAmypt%2FuMPkeG9tTu0%2FQBm74QuHnoDHWru4HOM6amN65AZ%2BT6TtJS5ShAHEZGP%2B3vUZJZxALYrCCCo28%3D&formData=offer%255B34272171%255D%255B18457975%255D%255Bprice%255D%3D11%26offer%255B34272172%255D%255B18457976%255D%255Bprice%255D%3D22%26offer%255B34272173%255D%255B18457977%255D%255Bprice%255D%3D33'

In [131]:
# urllib.parse.quote()

unq = urllib.parse.unquote(data)
unq

'xmlData[18457975]=<?xml+version="1.0"+encoding="UTF-8"?><priceoffer><trd_buy_id>7196072</trd_buy_id><trd_buy_lot_id>19114250</trd_buy_lot_id><trd_buy_lot_point_id>18457975</trd_buy_lot_point_id><price>11</price></priceoffer>&xmlData[18457976]=<?xml+version="1.0"+encoding="UTF-8"?><priceoffer><trd_buy_id>7196072</trd_buy_id><trd_buy_lot_id>19114251</trd_buy_lot_id><trd_buy_lot_point_id>18457976</trd_buy_lot_point_id><price>22</price></priceoffer>&xmlData[18457977]=<?xml+version="1.0"+encoding="UTF-8"?><priceoffer><trd_buy_id>7196072</trd_buy_id><trd_buy_lot_id>19114252</trd_buy_lot_id><trd_buy_lot_point_id>18457977</trd_buy_lot_point_id><price>33</price></priceoffer>&signData[18457975]=MIIP7gYJKoZIhvcNAQcCoIIP3zCCD9sCAQExDjAMBggqgw4DCgEDAQUAMAsGCSqGSIb3DQEHAaCCBNowggTWMIIEgKADAgECAhQsEW5waAgTAPIdPyOMUwKjbFTsfjANBgkqgw4DCgEBAQIFADBTMQswCQYDVQQGEwJLWjFEMEIGA1UEAww70rDQm9Ci0KLQq9KaINCa0KPTmNCb0JDQndCU0KvQoNCj0KjQqyDQntCg0KLQkNCb0KvSmiAoR09TVCkwHhcNMjEwNjA0MDc1MTU1WhcNMjIwNjA0MDc1MTU1WjCCA

In [ ]:
u = (
    'xmlData[18457975]=<?xml+version="1.0"+encoding="UTF-8"?>'
    '<priceoffer><trd_buy_id>7196072</trd_buy_id>'
    '<trd_buy_lot_id>19114250</trd_buy_lot_id>'
    '<trd_buy_lot_point_id>18457975</trd_buy_lot_point_id>'
    '<price>11</price></priceoffer>&'
    'xmlData[18457976]=<?xml+version="1.0"+encoding="UTF-8"?>'
    '<priceoffer><trd_buy_id>7196072</trd_buy_id>'
    '<trd_buy_lot_id>19114251</trd_buy_lot_id>'
    '<trd_buy_lot_point_id>18457976</trd_buy_lot_point_id>'
    '<price>22</price></priceoffer>&'
    'xmlData[18457977]=<?xml+version="1.0"+encoding="UTF-8"?>'
    '<priceoffer><trd_buy_id>7196072</trd_buy_id>'
    '<trd_buy_lot_id>19114252</trd_buy_lot_id>'
    '<trd_buy_lot_point_id>18457977</trd_buy_lot_point_id>'
    '<price>33</price></priceoffer>&'
    'signData[18457975]=MIIP7gYJKoZIhvcNAQcCoIIP3zCCD9sCAQExDjAMBggqgw4DCgEDAQUAMAsGCSqGSIb3DQEHAaCCBNowggTWMIIEgKADAgECAhQsEW5waAgTAPIdPyOMUwKjbFTsfjANBgkqgw4DCgEBAQIFADBTMQswCQYDVQQGEwJLWjFEMEIGA1UEAww70rDQm9Ci0KLQq9KaINCa0KPTmNCb0JDQndCU0KvQoNCj0KjQqyDQntCg0KLQkNCb0KvSmiAoR09TVCkwHhcNMjEwNjA0MDc1MTU1WhcNMjIwNjA0MDc1MTU1WjCCAXUxKjAoBgNVBAMMIdCa0JDQmdCT0J7QoNCe0JTQptCV0JLQkCDQnNCQ0JnQrzEhMB8GA1UEBAwY0JrQkNCZ0JPQntCg0J7QlNCm0JXQktCQMRgwFgYDVQQFEw9JSU44MzA1MDQ0MDAwNDkxCzAJBgNVBAYTAktaMYGcMIGZBgNVBAoMgZHQotCe0JLQkNCg0JjQqdCV0KHQotCS0J4g0KEg0J7Qk9Cg0JDQndCY0KfQldCd0J3QntCZINCe0KLQktCV0KLQodCi0JLQldCd0J3QntCh0KLQrNCuICLQmNCd0JbQmNCd0JjQoNCY0J3Qk9Ce0JLQkNCvINCa0J7QnNCf0JDQndCY0K8gItCb0JjQlNCV0KAiMRgwFgYDVQQLDA9CSU4xMTA5NDAwMDQ5MDkxIzAhBgNVBCoMGtCh0KLQkNCd0JjQodCb0JDQktCe0JLQndCQMR8wHQYJKoZIhvcNAQkBFhBDRU9ATEVBREVSRU5HLktaMGwwJQYJKoMOAwoBAQEBMBgGCiqDDgMKAQEBAQEGCiqDDgMKAQMBAQADQwAEQOTfqVEh24kddh/aZlKhpjjqrTqnPx9OQicP3VjyFBJs193DrOnsUHVCnqkhqXSwisiXvQmmBA6PHMNg0hX+aSejggH2MIIB8jAOBgNVHQ8BAf8EBAMCBsAwMwYDVR0lBCwwKgYIKwYBBQUHAwQGCCqDDgMDBAECBgkqgw4DAwQBAgEGCSqDDgMDBAMCATAPBgNVHSMECDAGgARbanPpMB0GA1UdDgQWBBTeETtao0pNOO6INsPiri48jiPQbDBeBgNVHSAEVzBVMFMGByqDDgMDAgEwSDAhBggrBgEFBQcCARYVaHR0cDovL3BraS5nb3Yua3ovY3BzMCMGCCsGAQUFBwICMBcMFWh0dHA6Ly9wa2kuZ292Lmt6L2NwczBYBgNVHR8EUTBPME2gS6BJhiJodHRwOi8vY3JsLnBraS5nb3Yua3ovbmNhX2dvc3QuY3JshiNodHRwOi8vY3JsMS5wa2kuZ292Lmt6L25jYV9nb3N0LmNybDBcBgNVHS4EVTBTMFGgT6BNhiRodHRwOi8vY3JsLnBraS5nb3Yua3ovbmNhX2RfZ29zdC5jcmyGJWh0dHA6Ly9jcmwxLnBraS5nb3Yua3ovbmNhX2RfZ29zdC5jcmwwYwYIKwYBBQUHAQEEVzBVMC8GCCsGAQUFBzAChiNodHRwOi8vcGtpLmdvdi5rei9jZXJ0L25jYV9nb3N0LmNlcjAiBggrBgEFBQcwAYYWaHR0cDovL29jc3AucGtpLmdvdi5rejANBgkqgw4DCgEBAQIFAANBAF6nzMbWq5xVRWa5wASYXDr9StJo7NnVQSskyKrwhdnNDoRbtZuNoPiM5nt4FmOCS4qp5wLGCJADSDgT5k/0yDExggrZMIIK1QIBATBrMFMxCzAJBgNVBAYTAktaMUQwQgYDVQQDDDvSsNCb0KLQotCr0pog0JrQo9OY0JvQkNCd0JTQq9Cg0KPQqNCrINCe0KDQotCQ0JvQq9KaIChHT1NUKQIULBFucGgIEwDyHT8jjFMCo2xU7H4wDAYIKoMOAwoBAwEFAKCBojAYBgkqhkiG9w0BCQMxCwYJKoZIhvcNAQcBMBwGCSqGSIb3DQEJBTEPFw0yMjAzMjUxNDQxNDJaMC8GCSqGSIb3DQEJBDEiBCB8qjCGZVAshUyFnm7LwZdksVaMcWO7JSLPXDYuSYXOezA3BgsqhkiG9w0BCRACLzEoMCYwJDAiBCDtSxHdcyUCMLpJK8z8EZFN1Szhqk3bdjuIAn0o7cdZcTANBgkqgw4DCgEBAQIFAARA2lFpl3CCwbWu0Wjqc1xKrMSwFF63Ns1azC1O3wef/lr95m5J2Lj7DoDa/pQXcLpeiS8gEHdybm4D6pO80RNVnaGCCV0wgglZBgsqhkiG9w0BCRACDjGCCUgwgglEBgkqhkiG9w0BBwKgggk1MIIJMQIBAzEPMA0GCWCGSAFlAwQCAQUAMIGEBgsqhkiG9w0BCRABBKB1BHMwcQIBAQYIKoMOAwMCBgIwMTANBglghkgBZQMEAgEFAAQgRYxRFBpofODghuXC0QFXNTuphL8MZLwFlfYzy+ITU0kCFLjkV3bRy2zEoWnYReuD+IxxSBrtGA8yMDIyMDMyNTE0NDE0M1oCCD1E6MIDnqBeoIIGXjCCBlowggRCoAMCAQICFD2d5W1fJ5xdBux9i4OqUL3rQ300MA0GCSqGSIb3DQEBCwUAMFIxCzAJBgNVBAYTAktaMUMwQQYDVQQDDDrSsNCb0KLQotCr0pog0JrQo9OY0JvQkNCd0JTQq9Cg0KPQqNCrINCe0KDQotCQ0JvQq9KaIChSU0EpMB4XDTE5MTIxMjA1MjIwNVoXDTIyMTIxMTA1MjIwNVowggESMRQwEgYDVQQDDAtUU0EgU0VSVklDRTEYMBYGA1UEBRMPSUlONzYxMjMxMzAwMzEzMQswCQYDVQQGEwJLWjEcMBoGA1UEBwwT0J3Qo9CgLdCh0KPQm9Ci0JDQnTEcMBoGA1UECAwT0J3Qo9CgLdCh0KPQm9Ci0JDQnTF9MHsGA1UECgx00JDQmtCm0JjQntCd0JXQoNCd0J7QlSDQntCR0KnQldCh0KLQktCeICLQndCQ0KbQmNCe0J3QkNCb0KzQndCr0JUg0JjQndCk0J7QoNCc0JDQptCY0J7QndCd0KvQlSDQotCV0KXQndCe0JvQntCT0JjQmCIxGDAWBgNVBAsMD0JJTjAwMDc0MDAwMDcyODCCASIwDQYJKoZIhvcNAQEBBQADggEPADCCAQoCggEBAIuFfpSCo5uenI8Cn9234hpG8uQZnRMBT2QwrndJaxIAHAMEQ9T3sxi2KWDL2mJArmVs48gdhhkm9JRVQCnYGb+OZ+nmePaeU826nawS83OVlJggIxbtAE4xdr8n2jCKMqjYgaQjXtllIC3NV5LvnZFeaoy/ndih+o3iNbiSfEoWsX1xB8MW7ZHIqZBnHYOQnnpSW9TRejYF4B+FC4Sr70IXFRYHuicNJZo4V9ijGiacBA3WWOITxOPd0TZcd54G4kNFSSzocGiqMLVtWGNWBKzRtqxHPKdvoK2M3oZ5Ihqzjf0RnuS30HSQsXzGtYE1d7b7y+R5fr+pWdfp3C2ECdUCAwEAAaOCAWQwggFgMBYGA1UdJQEB/wQMMAoGCCsGAQUFBwMIMA8GA1UdIwQIMAaABFtqdBEwHQYDVR0OBBYEFFqrRbGX3k3FKok5zH/A2RUXLTKfMFYGA1UdHwRPME0wS6BJoEeGIWh0dHA6Ly9jcmwucGtpLmdvdi5rei9uY2FfcnNhLmNybIYiaHR0cDovL2NybDEucGtpLmdvdi5rei9uY2FfcnNhLmNybDBaBgNVHS4EUzBRME+gTaBLhiNodHRwOi8vY3JsLnBraS5nb3Yua3ovbmNhX2RfcnNhLmNybIYkaHR0cDovL2NybDEucGtpLmdvdi5rei9uY2FfZF9yc2EuY3JsMGIGCCsGAQUFBwEBBFYwVDAuBggrBgEFBQcwAoYiaHR0cDovL3BraS5nb3Yua3ovY2VydC9uY2FfcnNhLmNlcjAiBggrBgEFBQcwAYYWaHR0cDovL29jc3AucGtpLmdvdi5rejANBgkqhkiG9w0BAQsFAAOCAgEApPnpRXedzDgnZBPKJPhOJ8dWXzuNu65vSymgTH1hDXwE1lR90n5kZvu/kHO1shsId6TEg6X/1IE6Gp8992JsTezkxUexb6MdWWnUYjNhuPYzsYfBMG+Z+UMjMhf2ER5SUjDiEXrdwqKkRPF5HfzuKEdQfjoFctYveqirISBdEOUIGD0LCiqqTD7W5dAIQaVbJZLGCG5nKgZ8rvzYPyQJU3AqizXeHglOWLxINWmYs69NCqmvk6XI7oFVyVe8UTLQ+qMtl1MfrNTvSHtU2Uei2n2NKcN1oof5E2RDASf3ImNDeSJuFKzIXnwdc9xmesfUCBVJJLgt4HU6RHh+5StrR+Wz7/OYBoVuhUE0YMSM/H74BrKbI6k8rmPw08QoU1v4cfYEH2jGzrJt4IephTkvxbv8ENcJX5LpUUux6oJHLTIzTU9q1BjitoODmV2EmmBn1hzU+YaSPEffIztXt8zuoVrH1gkT1s5bzc4nqc63OYur6XSTf6d66+2mavaVSQObNcYPz6YPnQFbDHzUy16oxX4FxlhnFSwul6gZg4b9kYh1BTdpXTGrfo4hD/c/4xF3OHWxprGnWb9uFxJqthhHkwEHloMcZaR5xSLT7kE4Lpy5uRz9qh4DVcbSTw2sqCJx/57YioRy7GUssbhG51+YBnDG9OEF7xhV6xdvF/SVVvwxggIwMIICLAIBATBqMFIxCzAJBgNVBAYTAktaMUMwQQYDVQQDDDrSsNCb0KLQotCr0pog0JrQo9OY0JvQkNCd0JTQq9Cg0KPQqNCrINCe0KDQotCQ0JvQq9KaIChSU0EpAhQ9neVtXyecXQbsfYuDqlC960N9NDANBglghkgBZQMEAgEFAKCBmDAaBgkqhkiG9w0BCQMxDQYLKoZIhvcNAQkQAQQwHAYJKoZIhvcNAQkFMQ8XDTIyMDMyNTE0NDE0M1owKwYLKoZIhvcNAQkQAgwxHDAaMBgwFgQUyyVFl1vW4xC+7Z6ZVsJ4kYybLQgwLwYJKoZIhvcNAQkEMSIEIMpzqR3sHGmOGu0XJVHsXFw6dVa3w8ktEK9f353qhdRFMA0GCSqGSIb3DQEBCwUABIIBAD3GFSl6rkv1UraXHbCbYGNaPwT0/tqAEMH1JgWMa2v8CQiyXERifjSTz+0RuKt6JDbvbRW0FWwjwVPdF4Rn79Xl2MubC7kwGkbgoYwKZqDGUxj+kr3Ji0eW0C5n4HAHY/sHFa3A84oAleCUExFVnVnYgeqXIpy27tQ19PqcudRDsRgMTTprak8nhq8NEa6DVd83PCsSEgpKvUEmkP55wv3tCV8eBC49owswvWNKQfycr8VR1ihAvuFqdD9dPbDaCU9ArUjgIHxv9gM6+EdIKK9Nht8QL6K1ySli7VHOPiqUFmVtXmTgpHXeJRYNvEbw6o1+RFn6WYVMbwBwijOCzMc=&'
    'signData[18457976]=MIIP7gYJKoZIhvcNAQcCoIIP3zCCD9sCAQExDjAMBggqgw4DCgEDAQUAMAsGCSqGSIb3DQEHAaCCBNowggTWMIIEgKADAgECAhQsEW5waAgTAPIdPyOMUwKjbFTsfjANBgkqgw4DCgEBAQIFADBTMQswCQYDVQQGEwJLWjFEMEIGA1UEAww70rDQm9Ci0KLQq9KaINCa0KPTmNCb0JDQndCU0KvQoNCj0KjQqyDQntCg0KLQkNCb0KvSmiAoR09TVCkwHhcNMjEwNjA0MDc1MTU1WhcNMjIwNjA0MDc1MTU1WjCCAXUxKjAoBgNVBAMMIdCa0JDQmdCT0J7QoNCe0JTQptCV0JLQkCDQnNCQ0JnQrzEhMB8GA1UEBAwY0JrQkNCZ0JPQntCg0J7QlNCm0JXQktCQMRgwFgYDVQQFEw9JSU44MzA1MDQ0MDAwNDkxCzAJBgNVBAYTAktaMYGcMIGZBgNVBAoMgZHQotCe0JLQkNCg0JjQqdCV0KHQotCS0J4g0KEg0J7Qk9Cg0JDQndCY0KfQldCd0J3QntCZINCe0KLQktCV0KLQodCi0JLQldCd0J3QntCh0KLQrNCuICLQmNCd0JbQmNCd0JjQoNCY0J3Qk9Ce0JLQkNCvINCa0J7QnNCf0JDQndCY0K8gItCb0JjQlNCV0KAiMRgwFgYDVQQLDA9CSU4xMTA5NDAwMDQ5MDkxIzAhBgNVBCoMGtCh0KLQkNCd0JjQodCb0JDQktCe0JLQndCQMR8wHQYJKoZIhvcNAQkBFhBDRU9ATEVBREVSRU5HLktaMGwwJQYJKoMOAwoBAQEBMBgGCiqDDgMKAQEBAQEGCiqDDgMKAQMBAQADQwAEQOTfqVEh24kddh/aZlKhpjjqrTqnPx9OQicP3VjyFBJs193DrOnsUHVCnqkhqXSwisiXvQmmBA6PHMNg0hX+aSejggH2MIIB8jAOBgNVHQ8BAf8EBAMCBsAwMwYDVR0lBCwwKgYIKwYBBQUHAwQGCCqDDgMDBAECBgkqgw4DAwQBAgEGCSqDDgMDBAMCATAPBgNVHSMECDAGgARbanPpMB0GA1UdDgQWBBTeETtao0pNOO6INsPiri48jiPQbDBeBgNVHSAEVzBVMFMGByqDDgMDAgEwSDAhBggrBgEFBQcCARYVaHR0cDovL3BraS5nb3Yua3ovY3BzMCMGCCsGAQUFBwICMBcMFWh0dHA6Ly9wa2kuZ292Lmt6L2NwczBYBgNVHR8EUTBPME2gS6BJhiJodHRwOi8vY3JsLnBraS5nb3Yua3ovbmNhX2dvc3QuY3JshiNodHRwOi8vY3JsMS5wa2kuZ292Lmt6L25jYV9nb3N0LmNybDBcBgNVHS4EVTBTMFGgT6BNhiRodHRwOi8vY3JsLnBraS5nb3Yua3ovbmNhX2RfZ29zdC5jcmyGJWh0dHA6Ly9jcmwxLnBraS5nb3Yua3ovbmNhX2RfZ29zdC5jcmwwYwYIKwYBBQUHAQEEVzBVMC8GCCsGAQUFBzAChiNodHRwOi8vcGtpLmdvdi5rei9jZXJ0L25jYV9nb3N0LmNlcjAiBggrBgEFBQcwAYYWaHR0cDovL29jc3AucGtpLmdvdi5rejANBgkqgw4DCgEBAQIFAANBAF6nzMbWq5xVRWa5wASYXDr9StJo7NnVQSskyKrwhdnNDoRbtZuNoPiM5nt4FmOCS4qp5wLGCJADSDgT5k/0yDExggrZMIIK1QIBATBrMFMxCzAJBgNVBAYTAktaMUQwQgYDVQQDDDvSsNCb0KLQotCr0pog0JrQo9OY0JvQkNCd0JTQq9Cg0KPQqNCrINCe0KDQotCQ0JvQq9KaIChHT1NUKQIULBFucGgIEwDyHT8jjFMCo2xU7H4wDAYIKoMOAwoBAwEFAKCBojAYBgkqhkiG9w0BCQMxCwYJKoZIhvcNAQcBMBwGCSqGSIb3DQEJBTEPFw0yMjAzMjUxNDQxNDJaMC8GCSqGSIb3DQEJBDEiBCAqeLIjxykElKrkpwrNwjWQhUDh4ovQUGWJuFi+7SvJ8TA3BgsqhkiG9w0BCRACLzEoMCYwJDAiBCDtSxHdcyUCMLpJK8z8EZFN1Szhqk3bdjuIAn0o7cdZcTANBgkqgw4DCgEBAQIFAARAeFZL2JKD5jMhgVFJfHnxE/WkD3uvYNhLTsDR3MIyIaU+kVKfP/q0BU+BgBzzSeAeaj1g6C0ZVlDwqLRzrpcQSKGCCV0wgglZBgsqhkiG9w0BCRACDjGCCUgwgglEBgkqhkiG9w0BBwKgggk1MIIJMQIBAzEPMA0GCWCGSAFlAwQCAQUAMIGEBgsqhkiG9w0BCRABBKB1BHMwcQIBAQYIKoMOAwMCBgIwMTANBglghkgBZQMEAgEFAAQg+loS2y0zZuq6WAwqn1QIe4/LbyvpRJ0FqeaPGELE//ECFMzDWElhYKIYp7Z+7DGhconFiNp0GA8yMDIyMDMyNTE0NDE0MloCCGvZCpY1vVFKoIIGXjCCBlowggRCoAMCAQICFD2d5W1fJ5xdBux9i4OqUL3rQ300MA0GCSqGSIb3DQEBCwUAMFIxCzAJBgNVBAYTAktaMUMwQQYDVQQDDDrSsNCb0KLQotCr0pog0JrQo9OY0JvQkNCd0JTQq9Cg0KPQqNCrINCe0KDQotCQ0JvQq9KaIChSU0EpMB4XDTE5MTIxMjA1MjIwNVoXDTIyMTIxMTA1MjIwNVowggESMRQwEgYDVQQDDAtUU0EgU0VSVklDRTEYMBYGA1UEBRMPSUlONzYxMjMxMzAwMzEzMQswCQYDVQQGEwJLWjEcMBoGA1UEBwwT0J3Qo9CgLdCh0KPQm9Ci0JDQnTEcMBoGA1UECAwT0J3Qo9CgLdCh0KPQm9Ci0JDQnTF9MHsGA1UECgx00JDQmtCm0JjQntCd0JXQoNCd0J7QlSDQntCR0KnQldCh0KLQktCeICLQndCQ0KbQmNCe0J3QkNCb0KzQndCr0JUg0JjQndCk0J7QoNCc0JDQptCY0J7QndCd0KvQlSDQotCV0KXQndCe0JvQntCT0JjQmCIxGDAWBgNVBAsMD0JJTjAwMDc0MDAwMDcyODCCASIwDQYJKoZIhvcNAQEBBQADggEPADCCAQoCggEBAIuFfpSCo5uenI8Cn9234hpG8uQZnRMBT2QwrndJaxIAHAMEQ9T3sxi2KWDL2mJArmVs48gdhhkm9JRVQCnYGb+OZ+nmePaeU826nawS83OVlJggIxbtAE4xdr8n2jCKMqjYgaQjXtllIC3NV5LvnZFeaoy/ndih+o3iNbiSfEoWsX1xB8MW7ZHIqZBnHYOQnnpSW9TRejYF4B+FC4Sr70IXFRYHuicNJZo4V9ijGiacBA3WWOITxOPd0TZcd54G4kNFSSzocGiqMLVtWGNWBKzRtqxHPKdvoK2M3oZ5Ihqzjf0RnuS30HSQsXzGtYE1d7b7y+R5fr+pWdfp3C2ECdUCAwEAAaOCAWQwggFgMBYGA1UdJQEB/wQMMAoGCCsGAQUFBwMIMA8GA1UdIwQIMAaABFtqdBEwHQYDVR0OBBYEFFqrRbGX3k3FKok5zH/A2RUXLTKfMFYGA1UdHwRPME0wS6BJoEeGIWh0dHA6Ly9jcmwucGtpLmdvdi5rei9uY2FfcnNhLmNybIYiaHR0cDovL2NybDEucGtpLmdvdi5rei9uY2FfcnNhLmNybDBaBgNVHS4EUzBRME+gTaBLhiNodHRwOi8vY3JsLnBraS5nb3Yua3ovbmNhX2RfcnNhLmNybIYkaHR0cDovL2NybDEucGtpLmdvdi5rei9uY2FfZF9yc2EuY3JsMGIGCCsGAQUFBwEBBFYwVDAuBggrBgEFBQcwAoYiaHR0cDovL3BraS5nb3Yua3ovY2VydC9uY2FfcnNhLmNlcjAiBggrBgEFBQcwAYYWaHR0cDovL29jc3AucGtpLmdvdi5rejANBgkqhkiG9w0BAQsFAAOCAgEApPnpRXedzDgnZBPKJPhOJ8dWXzuNu65vSymgTH1hDXwE1lR90n5kZvu/kHO1shsId6TEg6X/1IE6Gp8992JsTezkxUexb6MdWWnUYjNhuPYzsYfBMG+Z+UMjMhf2ER5SUjDiEXrdwqKkRPF5HfzuKEdQfjoFctYveqirISBdEOUIGD0LCiqqTD7W5dAIQaVbJZLGCG5nKgZ8rvzYPyQJU3AqizXeHglOWLxINWmYs69NCqmvk6XI7oFVyVe8UTLQ+qMtl1MfrNTvSHtU2Uei2n2NKcN1oof5E2RDASf3ImNDeSJuFKzIXnwdc9xmesfUCBVJJLgt4HU6RHh+5StrR+Wz7/OYBoVuhUE0YMSM/H74BrKbI6k8rmPw08QoU1v4cfYEH2jGzrJt4IephTkvxbv8ENcJX5LpUUux6oJHLTIzTU9q1BjitoODmV2EmmBn1hzU+YaSPEffIztXt8zuoVrH1gkT1s5bzc4nqc63OYur6XSTf6d66+2mavaVSQObNcYPz6YPnQFbDHzUy16oxX4FxlhnFSwul6gZg4b9kYh1BTdpXTGrfo4hD/c/4xF3OHWxprGnWb9uFxJqthhHkwEHloMcZaR5xSLT7kE4Lpy5uRz9qh4DVcbSTw2sqCJx/57YioRy7GUssbhG51+YBnDG9OEF7xhV6xdvF/SVVvwxggIwMIICLAIBATBqMFIxCzAJBgNVBAYTAktaMUMwQQYDVQQDDDrSsNCb0KLQotCr0pog0JrQo9OY0JvQkNCd0JTQq9Cg0KPQqNCrINCe0KDQotCQ0JvQq9KaIChSU0EpAhQ9neVtXyecXQbsfYuDqlC960N9NDANBglghkgBZQMEAgEFAKCBmDAaBgkqhkiG9w0BCQMxDQYLKoZIhvcNAQkQAQQwHAYJKoZIhvcNAQkFMQ8XDTIyMDMyNTE0NDE0MlowKwYLKoZIhvcNAQkQAgwxHDAaMBgwFgQUyyVFl1vW4xC+7Z6ZVsJ4kYybLQgwLwYJKoZIhvcNAQkEMSIEIPnmuHkbKsQRwwhb6RxZ4k/NGGAj+vfr5a7bTsg8D/ELMA0GCSqGSIb3DQEBCwUABIIBAEt1WmKBKyKabCPR4m90QbLpSZ+/ZjYDug3A3XDjQSNa7rbsKOUoJBu2r4ObcsSDNCfY9HN24o6EOnqnx2bPIzThTFgylzix09DxP9VhGvAoVhqAfdcKGjN6oNAxuDDvTv+hahc1gnzM0Q0gI2EIIGb2A1BWE86opiUUw+uV9hf9saWDredTAhll2CzNje2IrTwyf51zp+A5wZ/jlm0bzh7deY/EQ13yb/yjc94cCE/Z1Ijq63F6mwTl1EAehXMFZD/A/jKukFyHeW0aD8pYOB34Vth6PYuajZrFRg1Z93l03PSO7hY2Rr1OTFKAgRxViHojh19aV2lAoU9breFcgs0=&'
    'signData[18457977]=MIIP7gYJKoZIhvcNAQcCoIIP3zCCD9sCAQExDjAMBggqgw4DCgEDAQUAMAsGCSqGSIb3DQEHAaCCBNowggTWMIIEgKADAgECAhQsEW5waAgTAPIdPyOMUwKjbFTsfjANBgkqgw4DCgEBAQIFADBTMQswCQYDVQQGEwJLWjFEMEIGA1UEAww70rDQm9Ci0KLQq9KaINCa0KPTmNCb0JDQndCU0KvQoNCj0KjQqyDQntCg0KLQkNCb0KvSmiAoR09TVCkwHhcNMjEwNjA0MDc1MTU1WhcNMjIwNjA0MDc1MTU1WjCCAXUxKjAoBgNVBAMMIdCa0JDQmdCT0J7QoNCe0JTQptCV0JLQkCDQnNCQ0JnQrzEhMB8GA1UEBAwY0JrQkNCZ0JPQntCg0J7QlNCm0JXQktCQMRgwFgYDVQQFEw9JSU44MzA1MDQ0MDAwNDkxCzAJBgNVBAYTAktaMYGcMIGZBgNVBAoMgZHQotCe0JLQkNCg0JjQqdCV0KHQotCS0J4g0KEg0J7Qk9Cg0JDQndCY0KfQldCd0J3QntCZINCe0KLQktCV0KLQodCi0JLQldCd0J3QntCh0KLQrNCuICLQmNCd0JbQmNCd0JjQoNCY0J3Qk9Ce0JLQkNCvINCa0J7QnNCf0JDQndCY0K8gItCb0JjQlNCV0KAiMRgwFgYDVQQLDA9CSU4xMTA5NDAwMDQ5MDkxIzAhBgNVBCoMGtCh0KLQkNCd0JjQodCb0JDQktCe0JLQndCQMR8wHQYJKoZIhvcNAQkBFhBDRU9ATEVBREVSRU5HLktaMGwwJQYJKoMOAwoBAQEBMBgGCiqDDgMKAQEBAQEGCiqDDgMKAQMBAQADQwAEQOTfqVEh24kddh/aZlKhpjjqrTqnPx9OQicP3VjyFBJs193DrOnsUHVCnqkhqXSwisiXvQmmBA6PHMNg0hX+aSejggH2MIIB8jAOBgNVHQ8BAf8EBAMCBsAwMwYDVR0lBCwwKgYIKwYBBQUHAwQGCCqDDgMDBAECBgkqgw4DAwQBAgEGCSqDDgMDBAMCATAPBgNVHSMECDAGgARbanPpMB0GA1UdDgQWBBTeETtao0pNOO6INsPiri48jiPQbDBeBgNVHSAEVzBVMFMGByqDDgMDAgEwSDAhBggrBgEFBQcCARYVaHR0cDovL3BraS5nb3Yua3ovY3BzMCMGCCsGAQUFBwICMBcMFWh0dHA6Ly9wa2kuZ292Lmt6L2NwczBYBgNVHR8EUTBPME2gS6BJhiJodHRwOi8vY3JsLnBraS5nb3Yua3ovbmNhX2dvc3QuY3JshiNodHRwOi8vY3JsMS5wa2kuZ292Lmt6L25jYV9nb3N0LmNybDBcBgNVHS4EVTBTMFGgT6BNhiRodHRwOi8vY3JsLnBraS5nb3Yua3ovbmNhX2RfZ29zdC5jcmyGJWh0dHA6Ly9jcmwxLnBraS5nb3Yua3ovbmNhX2RfZ29zdC5jcmwwYwYIKwYBBQUHAQEEVzBVMC8GCCsGAQUFBzAChiNodHRwOi8vcGtpLmdvdi5rei9jZXJ0L25jYV9nb3N0LmNlcjAiBggrBgEFBQcwAYYWaHR0cDovL29jc3AucGtpLmdvdi5rejANBgkqgw4DCgEBAQIFAANBAF6nzMbWq5xVRWa5wASYXDr9StJo7NnVQSskyKrwhdnNDoRbtZuNoPiM5nt4FmOCS4qp5wLGCJADSDgT5k/0yDExggrZMIIK1QIBATBrMFMxCzAJBgNVBAYTAktaMUQwQgYDVQQDDDvSsNCb0KLQotCr0pog0JrQo9OY0JvQkNCd0JTQq9Cg0KPQqNCrINCe0KDQotCQ0JvQq9KaIChHT1NUKQIULBFucGgIEwDyHT8jjFMCo2xU7H4wDAYIKoMOAwoBAwEFAKCBojAYBgkqhkiG9w0BCQMxCwYJKoZIhvcNAQcBMBwGCSqGSIb3DQEJBTEPFw0yMjAzMjUxNDQxNDJaMC8GCSqGSIb3DQEJBDEiBCBfTZm2SguleHTRCPZ256RH7bOFR6sMpqLCbFmJ1Ug6MzA3BgsqhkiG9w0BCRACLzEoMCYwJDAiBCDtSxHdcyUCMLpJK8z8EZFN1Szhqk3bdjuIAn0o7cdZcTANBgkqgw4DCgEBAQIFAARAWzRWbC4xIWfWLPk1kN96TZtpzcCxvpym4pyHi339RB5/oiaYfJa5aJR8SxzBIx1ggfz4kIiVzBhEZ+ZMFgvMu6GCCV0wgglZBgsqhkiG9w0BCRACDjGCCUgwgglEBgkqhkiG9w0BBwKgggk1MIIJMQIBAzEPMA0GCWCGSAFlAwQCAQUAMIGEBgsqhkiG9w0BCRABBKB1BHMwcQIBAQYIKoMOAwMCBgIwMTANBglghkgBZQMEAgEFAAQg5Fs2mty1bH2+m+QDaWJcUuoNH2N9rfRfbr7NRD7Iq7ECFE3+opvxqvkhgkmnXDDcmto8r9rZGA8yMDIyMDMyNTE0NDE0MloCCAsNIuyea1DpoIIGXjCCBlowggRCoAMCAQICFD2d5W1fJ5xdBux9i4OqUL3rQ300MA0GCSqGSIb3DQEBCwUAMFIxCzAJBgNVBAYTAktaMUMwQQYDVQQDDDrSsNCb0KLQotCr0pog0JrQo9OY0JvQkNCd0JTQq9Cg0KPQqNCrINCe0KDQotCQ0JvQq9KaIChSU0EpMB4XDTE5MTIxMjA1MjIwNVoXDTIyMTIxMTA1MjIwNVowggESMRQwEgYDVQQDDAtUU0EgU0VSVklDRTEYMBYGA1UEBRMPSUlONzYxMjMxMzAwMzEzMQswCQYDVQQGEwJLWjEcMBoGA1UEBwwT0J3Qo9CgLdCh0KPQm9Ci0JDQnTEcMBoGA1UECAwT0J3Qo9CgLdCh0KPQm9Ci0JDQnTF9MHsGA1UECgx00JDQmtCm0JjQntCd0JXQoNCd0J7QlSDQntCR0KnQldCh0KLQktCeICLQndCQ0KbQmNCe0J3QkNCb0KzQndCr0JUg0JjQndCk0J7QoNCc0JDQptCY0J7QndCd0KvQlSDQotCV0KXQndCe0JvQntCT0JjQmCIxGDAWBgNVBAsMD0JJTjAwMDc0MDAwMDcyODCCASIwDQYJKoZIhvcNAQEBBQADggEPADCCAQoCggEBAIuFfpSCo5uenI8Cn9234hpG8uQZnRMBT2QwrndJaxIAHAMEQ9T3sxi2KWDL2mJArmVs48gdhhkm9JRVQCnYGb+OZ+nmePaeU826nawS83OVlJggIxbtAE4xdr8n2jCKMqjYgaQjXtllIC3NV5LvnZFeaoy/ndih+o3iNbiSfEoWsX1xB8MW7ZHIqZBnHYOQnnpSW9TRejYF4B+FC4Sr70IXFRYHuicNJZo4V9ijGiacBA3WWOITxOPd0TZcd54G4kNFSSzocGiqMLVtWGNWBKzRtqxHPKdvoK2M3oZ5Ihqzjf0RnuS30HSQsXzGtYE1d7b7y+R5fr+pWdfp3C2ECdUCAwEAAaOCAWQwggFgMBYGA1UdJQEB/wQMMAoGCCsGAQUFBwMIMA8GA1UdIwQIMAaABFtqdBEwHQYDVR0OBBYEFFqrRbGX3k3FKok5zH/A2RUXLTKfMFYGA1UdHwRPME0wS6BJoEeGIWh0dHA6Ly9jcmwucGtpLmdvdi5rei9uY2FfcnNhLmNybIYiaHR0cDovL2NybDEucGtpLmdvdi5rei9uY2FfcnNhLmNybDBaBgNVHS4EUzBRME+gTaBLhiNodHRwOi8vY3JsLnBraS5nb3Yua3ovbmNhX2RfcnNhLmNybIYkaHR0cDovL2NybDEucGtpLmdvdi5rei9uY2FfZF9yc2EuY3JsMGIGCCsGAQUFBwEBBFYwVDAuBggrBgEFBQcwAoYiaHR0cDovL3BraS5nb3Yua3ovY2VydC9uY2FfcnNhLmNlcjAiBggrBgEFBQcwAYYWaHR0cDovL29jc3AucGtpLmdvdi5rejANBgkqhkiG9w0BAQsFAAOCAgEApPnpRXedzDgnZBPKJPhOJ8dWXzuNu65vSymgTH1hDXwE1lR90n5kZvu/kHO1shsId6TEg6X/1IE6Gp8992JsTezkxUexb6MdWWnUYjNhuPYzsYfBMG+Z+UMjMhf2ER5SUjDiEXrdwqKkRPF5HfzuKEdQfjoFctYveqirISBdEOUIGD0LCiqqTD7W5dAIQaVbJZLGCG5nKgZ8rvzYPyQJU3AqizXeHglOWLxINWmYs69NCqmvk6XI7oFVyVe8UTLQ+qMtl1MfrNTvSHtU2Uei2n2NKcN1oof5E2RDASf3ImNDeSJuFKzIXnwdc9xmesfUCBVJJLgt4HU6RHh+5StrR+Wz7/OYBoVuhUE0YMSM/H74BrKbI6k8rmPw08QoU1v4cfYEH2jGzrJt4IephTkvxbv8ENcJX5LpUUux6oJHLTIzTU9q1BjitoODmV2EmmBn1hzU+YaSPEffIztXt8zuoVrH1gkT1s5bzc4nqc63OYur6XSTf6d66+2mavaVSQObNcYPz6YPnQFbDHzUy16oxX4FxlhnFSwul6gZg4b9kYh1BTdpXTGrfo4hD/c/4xF3OHWxprGnWb9uFxJqthhHkwEHloMcZaR5xSLT7kE4Lpy5uRz9qh4DVcbSTw2sqCJx/57YioRy7GUssbhG51+YBnDG9OEF7xhV6xdvF/SVVvwxggIwMIICLAIBATBqMFIxCzAJBgNVBAYTAktaMUMwQQYDVQQDDDrSsNCb0KLQotCr0pog0JrQo9OY0JvQkNCd0JTQq9Cg0KPQqNCrINCe0KDQotCQ0JvQq9KaIChSU0EpAhQ9neVtXyecXQbsfYuDqlC960N9NDANBglghkgBZQMEAgEFAKCBmDAaBgkqhkiG9w0BCQMxDQYLKoZIhvcNAQkQAQQwHAYJKoZIhvcNAQkFMQ8XDTIyMDMyNTE0NDE0MlowKwYLKoZIhvcNAQkQAgwxHDAaMBgwFgQUyyVFl1vW4xC+7Z6ZVsJ4kYybLQgwLwYJKoZIhvcNAQkEMSIEIKXvcBvdzQ/WjKLzP6Qt8YeZwfJ1FmiZwHU5OAVvNCv2MA0GCSqGSIb3DQEBCwUABIIBAE8VVUU4SOujXRlsNrs5Q7KaTco9DnpAy11Z+7ir8fw7ZOXIoTSLVXgm1AmxWdZg9CTL50H/bhDC8IItBPgwEgKIOHiYwel3M7QLFGqL5IUVO+uDLY1Iw4Yp8z5+aBfn3CAppjWyZxS8CtkdiiQ5CsziIOjR5htjqcS/volUbqiIO8Cp+v4WSmDCnTEfra4+SZCG4aOEZytuVnLjQBIRchDsf3ovaMaPe4sWeEXi5ecREeKWDRRmgMioMqH/Dm1ni1v0XsnLAmypt/uMPkeG9tTu0/QBm74QuHnoDHWru4HOM6amN65AZ+T6TtJS5ShAHEZGP+3vUZJZxALYrCCCo28=&'
    'formData=offer%5B34272171%5D%5B18457975%5D%5Bprice%5D=11&offer%5B34272172%5D%5B18457976%5D%5Bprice%5D=22&offer%5B34272173%5D%5B18457977%5D%5Bprice%5D=33'
)

In [133]:
u2 = 'formData=offer%5B34272171%5D%5B18457975%5D%5Bprice%5D=11&offer%5B34272172%5D%5B18457976%5D%5Bprice%5D=22&offer%5B34272173%5D%5B18457977%5D%5Bprice%5D=33'
u3 = urllib.parse.unquote(u2)

In [135]:
u4 = (
    'formData=offer[34272171][18457975][price]=11&'
    'offer[34272172][18457976][price]=22&'
    'offer[34272173][18457977][price]=33'
)

In [139]:
'123' + u4

'123formData=offer[34272171][18457975][price]=11&offer[34272172][18457976][price]=22&offer[34272173][18457977][price]=33'

In [235]:
# after escape

t1 = (
    'xmlData[18457975]=<?xml+version="1.0"+encoding="UTF-8"?>'
    
    '<priceoffer><trd_buy_id>7196072</trd_buy_id>'
    '<trd_buy_lot_id>19114250</trd_buy_lot_id>'
    '<trd_buy_lot_point_id>18457975</trd_buy_lot_point_id>'
    '<price>11</price></priceoffer>&'
    
    'xmlData[18457976]=<?xml+version="1.0"+encoding="UTF-8"?>'
    '<priceoffer><trd_buy_id>7196072</trd_buy_id>'
    '<trd_buy_lot_id>19114251</trd_buy_lot_id>'
    '<trd_buy_lot_point_id>18457976</trd_buy_lot_point_id>'
    '<price>22</price></priceoffer>&'
    
    'xmlData[18457977]=<?xml+version="1.0"+encoding="UTF-8"?>'
    '<priceoffer><trd_buy_id>7196072</trd_buy_id>'
    '<trd_buy_lot_id>19114252</trd_buy_lot_id>'
    '<trd_buy_lot_point_id>18457977</trd_buy_lot_point_id>'
    '<price>33</price></priceoffer>&'
    
    'signData[18457975]=MIIP7gYJKoZIhvcNAQcCoIIP3zCCD9sCAQExDjAMBggqgw4DCgEDAQUAMAsGCSqGSIb3DQEHAaCCBNowggTWMIIEgKADAgECAhQsEW5waAgTAPIdPyOMUwKjbFTsfjANBgkqgw4DCgEBAQIFADBTMQswCQYDVQQGEwJLWjFEMEIGA1UEAww70rDQm9Ci0KLQq9KaINCa0KPTmNCb0JDQndCU0KvQoNCj0KjQqyDQntCg0KLQkNCb0KvSmiAoR09TVCkwHhcNMjEwNjA0MDc1MTU1WhcNMjIwNjA0MDc1MTU1WjCCAXUxKjAoBgNVBAMMIdCa0JDQmdCT0J7QoNCe0JTQptCV0JLQkCDQnNCQ0JnQrzEhMB8GA1UEBAwY0JrQkNCZ0JPQntCg0J7QlNCm0JXQktCQMRgwFgYDVQQFEw9JSU44MzA1MDQ0MDAwNDkxCzAJBgNVBAYTAktaMYGcMIGZBgNVBAoMgZHQotCe0JLQkNCg0JjQqdCV0KHQotCS0J4g0KEg0J7Qk9Cg0JDQndCY0KfQldCd0J3QntCZINCe0KLQktCV0KLQodCi0JLQldCd0J3QntCh0KLQrNCuICLQmNCd0JbQmNCd0JjQoNCY0J3Qk9Ce0JLQkNCvINCa0J7QnNCf0JDQndCY0K8gItCb0JjQlNCV0KAiMRgwFgYDVQQLDA9CSU4xMTA5NDAwMDQ5MDkxIzAhBgNVBCoMGtCh0KLQkNCd0JjQodCb0JDQktCe0JLQndCQMR8wHQYJKoZIhvcNAQkBFhBDRU9ATEVBREVSRU5HLktaMGwwJQYJKoMOAwoBAQEBMBgGCiqDDgMKAQEBAQEGCiqDDgMKAQMBAQADQwAEQOTfqVEh24kddh/aZlKhpjjqrTqnPx9OQicP3VjyFBJs193DrOnsUHVCnqkhqXSwisiXvQmmBA6PHMNg0hX+aSejggH2MIIB8jAOBgNVHQ8BAf8EBAMCBsAwMwYDVR0lBCwwKgYIKwYBBQUHAwQGCCqDDgMDBAECBgkqgw4DAwQBAgEGCSqDDgMDBAMCATAPBgNVHSMECDAGgARbanPpMB0GA1UdDgQWBBTeETtao0pNOO6INsPiri48jiPQbDBeBgNVHSAEVzBVMFMGByqDDgMDAgEwSDAhBggrBgEFBQcCARYVaHR0cDovL3BraS5nb3Yua3ovY3BzMCMGCCsGAQUFBwICMBcMFWh0dHA6Ly9wa2kuZ292Lmt6L2NwczBYBgNVHR8EUTBPME2gS6BJhiJodHRwOi8vY3JsLnBraS5nb3Yua3ovbmNhX2dvc3QuY3JshiNodHRwOi8vY3JsMS5wa2kuZ292Lmt6L25jYV9nb3N0LmNybDBcBgNVHS4EVTBTMFGgT6BNhiRodHRwOi8vY3JsLnBraS5nb3Yua3ovbmNhX2RfZ29zdC5jcmyGJWh0dHA6Ly9jcmwxLnBraS5nb3Yua3ovbmNhX2RfZ29zdC5jcmwwYwYIKwYBBQUHAQEEVzBVMC8GCCsGAQUFBzAChiNodHRwOi8vcGtpLmdvdi5rei9jZXJ0L25jYV9nb3N0LmNlcjAiBggrBgEFBQcwAYYWaHR0cDovL29jc3AucGtpLmdvdi5rejANBgkqgw4DCgEBAQIFAANBAF6nzMbWq5xVRWa5wASYXDr9StJo7NnVQSskyKrwhdnNDoRbtZuNoPiM5nt4FmOCS4qp5wLGCJADSDgT5k/0yDExggrZMIIK1QIBATBrMFMxCzAJBgNVBAYTAktaMUQwQgYDVQQDDDvSsNCb0KLQotCr0pog0JrQo9OY0JvQkNCd0JTQq9Cg0KPQqNCrINCe0KDQotCQ0JvQq9KaIChHT1NUKQIULBFucGgIEwDyHT8jjFMCo2xU7H4wDAYIKoMOAwoBAwEFAKCBojAYBgkqhkiG9w0BCQMxCwYJKoZIhvcNAQcBMBwGCSqGSIb3DQEJBTEPFw0yMjAzMjUxNDQxNDJaMC8GCSqGSIb3DQEJBDEiBCB8qjCGZVAshUyFnm7LwZdksVaMcWO7JSLPXDYuSYXOezA3BgsqhkiG9w0BCRACLzEoMCYwJDAiBCDtSxHdcyUCMLpJK8z8EZFN1Szhqk3bdjuIAn0o7cdZcTANBgkqgw4DCgEBAQIFAARA2lFpl3CCwbWu0Wjqc1xKrMSwFF63Ns1azC1O3wef/lr95m5J2Lj7DoDa/pQXcLpeiS8gEHdybm4D6pO80RNVnaGCCV0wgglZBgsqhkiG9w0BCRACDjGCCUgwgglEBgkqhkiG9w0BBwKgggk1MIIJMQIBAzEPMA0GCWCGSAFlAwQCAQUAMIGEBgsqhkiG9w0BCRABBKB1BHMwcQIBAQYIKoMOAwMCBgIwMTANBglghkgBZQMEAgEFAAQgRYxRFBpofODghuXC0QFXNTuphL8MZLwFlfYzy+ITU0kCFLjkV3bRy2zEoWnYReuD+IxxSBrtGA8yMDIyMDMyNTE0NDE0M1oCCD1E6MIDnqBeoIIGXjCCBlowggRCoAMCAQICFD2d5W1fJ5xdBux9i4OqUL3rQ300MA0GCSqGSIb3DQEBCwUAMFIxCzAJBgNVBAYTAktaMUMwQQYDVQQDDDrSsNCb0KLQotCr0pog0JrQo9OY0JvQkNCd0JTQq9Cg0KPQqNCrINCe0KDQotCQ0JvQq9KaIChSU0EpMB4XDTE5MTIxMjA1MjIwNVoXDTIyMTIxMTA1MjIwNVowggESMRQwEgYDVQQDDAtUU0EgU0VSVklDRTEYMBYGA1UEBRMPSUlONzYxMjMxMzAwMzEzMQswCQYDVQQGEwJLWjEcMBoGA1UEBwwT0J3Qo9CgLdCh0KPQm9Ci0JDQnTEcMBoGA1UECAwT0J3Qo9CgLdCh0KPQm9Ci0JDQnTF9MHsGA1UECgx00JDQmtCm0JjQntCd0JXQoNCd0J7QlSDQntCR0KnQldCh0KLQktCeICLQndCQ0KbQmNCe0J3QkNCb0KzQndCr0JUg0JjQndCk0J7QoNCc0JDQptCY0J7QndCd0KvQlSDQotCV0KXQndCe0JvQntCT0JjQmCIxGDAWBgNVBAsMD0JJTjAwMDc0MDAwMDcyODCCASIwDQYJKoZIhvcNAQEBBQADggEPADCCAQoCggEBAIuFfpSCo5uenI8Cn9234hpG8uQZnRMBT2QwrndJaxIAHAMEQ9T3sxi2KWDL2mJArmVs48gdhhkm9JRVQCnYGb+OZ+nmePaeU826nawS83OVlJggIxbtAE4xdr8n2jCKMqjYgaQjXtllIC3NV5LvnZFeaoy/ndih+o3iNbiSfEoWsX1xB8MW7ZHIqZBnHYOQnnpSW9TRejYF4B+FC4Sr70IXFRYHuicNJZo4V9ijGiacBA3WWOITxOPd0TZcd54G4kNFSSzocGiqMLVtWGNWBKzRtqxHPKdvoK2M3oZ5Ihqzjf0RnuS30HSQsXzGtYE1d7b7y+R5fr+pWdfp3C2ECdUCAwEAAaOCAWQwggFgMBYGA1UdJQEB/wQMMAoGCCsGAQUFBwMIMA8GA1UdIwQIMAaABFtqdBEwHQYDVR0OBBYEFFqrRbGX3k3FKok5zH/A2RUXLTKfMFYGA1UdHwRPME0wS6BJoEeGIWh0dHA6Ly9jcmwucGtpLmdvdi5rei9uY2FfcnNhLmNybIYiaHR0cDovL2NybDEucGtpLmdvdi5rei9uY2FfcnNhLmNybDBaBgNVHS4EUzBRME+gTaBLhiNodHRwOi8vY3JsLnBraS5nb3Yua3ovbmNhX2RfcnNhLmNybIYkaHR0cDovL2NybDEucGtpLmdvdi5rei9uY2FfZF9yc2EuY3JsMGIGCCsGAQUFBwEBBFYwVDAuBggrBgEFBQcwAoYiaHR0cDovL3BraS5nb3Yua3ovY2VydC9uY2FfcnNhLmNlcjAiBggrBgEFBQcwAYYWaHR0cDovL29jc3AucGtpLmdvdi5rejANBgkqhkiG9w0BAQsFAAOCAgEApPnpRXedzDgnZBPKJPhOJ8dWXzuNu65vSymgTH1hDXwE1lR90n5kZvu/kHO1shsId6TEg6X/1IE6Gp8992JsTezkxUexb6MdWWnUYjNhuPYzsYfBMG+Z+UMjMhf2ER5SUjDiEXrdwqKkRPF5HfzuKEdQfjoFctYveqirISBdEOUIGD0LCiqqTD7W5dAIQaVbJZLGCG5nKgZ8rvzYPyQJU3AqizXeHglOWLxINWmYs69NCqmvk6XI7oFVyVe8UTLQ+qMtl1MfrNTvSHtU2Uei2n2NKcN1oof5E2RDASf3ImNDeSJuFKzIXnwdc9xmesfUCBVJJLgt4HU6RHh+5StrR+Wz7/OYBoVuhUE0YMSM/H74BrKbI6k8rmPw08QoU1v4cfYEH2jGzrJt4IephTkvxbv8ENcJX5LpUUux6oJHLTIzTU9q1BjitoODmV2EmmBn1hzU+YaSPEffIztXt8zuoVrH1gkT1s5bzc4nqc63OYur6XSTf6d66+2mavaVSQObNcYPz6YPnQFbDHzUy16oxX4FxlhnFSwul6gZg4b9kYh1BTdpXTGrfo4hD/c/4xF3OHWxprGnWb9uFxJqthhHkwEHloMcZaR5xSLT7kE4Lpy5uRz9qh4DVcbSTw2sqCJx/57YioRy7GUssbhG51+YBnDG9OEF7xhV6xdvF/SVVvwxggIwMIICLAIBATBqMFIxCzAJBgNVBAYTAktaMUMwQQYDVQQDDDrSsNCb0KLQotCr0pog0JrQo9OY0JvQkNCd0JTQq9Cg0KPQqNCrINCe0KDQotCQ0JvQq9KaIChSU0EpAhQ9neVtXyecXQbsfYuDqlC960N9NDANBglghkgBZQMEAgEFAKCBmDAaBgkqhkiG9w0BCQMxDQYLKoZIhvcNAQkQAQQwHAYJKoZIhvcNAQkFMQ8XDTIyMDMyNTE0NDE0M1owKwYLKoZIhvcNAQkQAgwxHDAaMBgwFgQUyyVFl1vW4xC+7Z6ZVsJ4kYybLQgwLwYJKoZIhvcNAQkEMSIEIMpzqR3sHGmOGu0XJVHsXFw6dVa3w8ktEK9f353qhdRFMA0GCSqGSIb3DQEBCwUABIIBAD3GFSl6rkv1UraXHbCbYGNaPwT0/tqAEMH1JgWMa2v8CQiyXERifjSTz+0RuKt6JDbvbRW0FWwjwVPdF4Rn79Xl2MubC7kwGkbgoYwKZqDGUxj+kr3Ji0eW0C5n4HAHY/sHFa3A84oAleCUExFVnVnYgeqXIpy27tQ19PqcudRDsRgMTTprak8nhq8NEa6DVd83PCsSEgpKvUEmkP55wv3tCV8eBC49owswvWNKQfycr8VR1ihAvuFqdD9dPbDaCU9ArUjgIHxv9gM6+EdIKK9Nht8QL6K1ySli7VHOPiqUFmVtXmTgpHXeJRYNvEbw6o1+RFn6WYVMbwBwijOCzMc=&'
    
    'signData[18457976]=MIIP7gYJKoZIhvcNAQcCoIIP3zCCD9sCAQExDjAMBggqgw4DCgEDAQUAMAsGCSqGSIb3DQEHAaCCBNowggTWMIIEgKADAgECAhQsEW5waAgTAPIdPyOMUwKjbFTsfjANBgkqgw4DCgEBAQIFADBTMQswCQYDVQQGEwJLWjFEMEIGA1UEAww70rDQm9Ci0KLQq9KaINCa0KPTmNCb0JDQndCU0KvQoNCj0KjQqyDQntCg0KLQkNCb0KvSmiAoR09TVCkwHhcNMjEwNjA0MDc1MTU1WhcNMjIwNjA0MDc1MTU1WjCCAXUxKjAoBgNVBAMMIdCa0JDQmdCT0J7QoNCe0JTQptCV0JLQkCDQnNCQ0JnQrzEhMB8GA1UEBAwY0JrQkNCZ0JPQntCg0J7QlNCm0JXQktCQMRgwFgYDVQQFEw9JSU44MzA1MDQ0MDAwNDkxCzAJBgNVBAYTAktaMYGcMIGZBgNVBAoMgZHQotCe0JLQkNCg0JjQqdCV0KHQotCS0J4g0KEg0J7Qk9Cg0JDQndCY0KfQldCd0J3QntCZINCe0KLQktCV0KLQodCi0JLQldCd0J3QntCh0KLQrNCuICLQmNCd0JbQmNCd0JjQoNCY0J3Qk9Ce0JLQkNCvINCa0J7QnNCf0JDQndCY0K8gItCb0JjQlNCV0KAiMRgwFgYDVQQLDA9CSU4xMTA5NDAwMDQ5MDkxIzAhBgNVBCoMGtCh0KLQkNCd0JjQodCb0JDQktCe0JLQndCQMR8wHQYJKoZIhvcNAQkBFhBDRU9ATEVBREVSRU5HLktaMGwwJQYJKoMOAwoBAQEBMBgGCiqDDgMKAQEBAQEGCiqDDgMKAQMBAQADQwAEQOTfqVEh24kddh/aZlKhpjjqrTqnPx9OQicP3VjyFBJs193DrOnsUHVCnqkhqXSwisiXvQmmBA6PHMNg0hX+aSejggH2MIIB8jAOBgNVHQ8BAf8EBAMCBsAwMwYDVR0lBCwwKgYIKwYBBQUHAwQGCCqDDgMDBAECBgkqgw4DAwQBAgEGCSqDDgMDBAMCATAPBgNVHSMECDAGgARbanPpMB0GA1UdDgQWBBTeETtao0pNOO6INsPiri48jiPQbDBeBgNVHSAEVzBVMFMGByqDDgMDAgEwSDAhBggrBgEFBQcCARYVaHR0cDovL3BraS5nb3Yua3ovY3BzMCMGCCsGAQUFBwICMBcMFWh0dHA6Ly9wa2kuZ292Lmt6L2NwczBYBgNVHR8EUTBPME2gS6BJhiJodHRwOi8vY3JsLnBraS5nb3Yua3ovbmNhX2dvc3QuY3JshiNodHRwOi8vY3JsMS5wa2kuZ292Lmt6L25jYV9nb3N0LmNybDBcBgNVHS4EVTBTMFGgT6BNhiRodHRwOi8vY3JsLnBraS5nb3Yua3ovbmNhX2RfZ29zdC5jcmyGJWh0dHA6Ly9jcmwxLnBraS5nb3Yua3ovbmNhX2RfZ29zdC5jcmwwYwYIKwYBBQUHAQEEVzBVMC8GCCsGAQUFBzAChiNodHRwOi8vcGtpLmdvdi5rei9jZXJ0L25jYV9nb3N0LmNlcjAiBggrBgEFBQcwAYYWaHR0cDovL29jc3AucGtpLmdvdi5rejANBgkqgw4DCgEBAQIFAANBAF6nzMbWq5xVRWa5wASYXDr9StJo7NnVQSskyKrwhdnNDoRbtZuNoPiM5nt4FmOCS4qp5wLGCJADSDgT5k/0yDExggrZMIIK1QIBATBrMFMxCzAJBgNVBAYTAktaMUQwQgYDVQQDDDvSsNCb0KLQotCr0pog0JrQo9OY0JvQkNCd0JTQq9Cg0KPQqNCrINCe0KDQotCQ0JvQq9KaIChHT1NUKQIULBFucGgIEwDyHT8jjFMCo2xU7H4wDAYIKoMOAwoBAwEFAKCBojAYBgkqhkiG9w0BCQMxCwYJKoZIhvcNAQcBMBwGCSqGSIb3DQEJBTEPFw0yMjAzMjUxNDQxNDJaMC8GCSqGSIb3DQEJBDEiBCAqeLIjxykElKrkpwrNwjWQhUDh4ovQUGWJuFi+7SvJ8TA3BgsqhkiG9w0BCRACLzEoMCYwJDAiBCDtSxHdcyUCMLpJK8z8EZFN1Szhqk3bdjuIAn0o7cdZcTANBgkqgw4DCgEBAQIFAARAeFZL2JKD5jMhgVFJfHnxE/WkD3uvYNhLTsDR3MIyIaU+kVKfP/q0BU+BgBzzSeAeaj1g6C0ZVlDwqLRzrpcQSKGCCV0wgglZBgsqhkiG9w0BCRACDjGCCUgwgglEBgkqhkiG9w0BBwKgggk1MIIJMQIBAzEPMA0GCWCGSAFlAwQCAQUAMIGEBgsqhkiG9w0BCRABBKB1BHMwcQIBAQYIKoMOAwMCBgIwMTANBglghkgBZQMEAgEFAAQg+loS2y0zZuq6WAwqn1QIe4/LbyvpRJ0FqeaPGELE//ECFMzDWElhYKIYp7Z+7DGhconFiNp0GA8yMDIyMDMyNTE0NDE0MloCCGvZCpY1vVFKoIIGXjCCBlowggRCoAMCAQICFD2d5W1fJ5xdBux9i4OqUL3rQ300MA0GCSqGSIb3DQEBCwUAMFIxCzAJBgNVBAYTAktaMUMwQQYDVQQDDDrSsNCb0KLQotCr0pog0JrQo9OY0JvQkNCd0JTQq9Cg0KPQqNCrINCe0KDQotCQ0JvQq9KaIChSU0EpMB4XDTE5MTIxMjA1MjIwNVoXDTIyMTIxMTA1MjIwNVowggESMRQwEgYDVQQDDAtUU0EgU0VSVklDRTEYMBYGA1UEBRMPSUlONzYxMjMxMzAwMzEzMQswCQYDVQQGEwJLWjEcMBoGA1UEBwwT0J3Qo9CgLdCh0KPQm9Ci0JDQnTEcMBoGA1UECAwT0J3Qo9CgLdCh0KPQm9Ci0JDQnTF9MHsGA1UECgx00JDQmtCm0JjQntCd0JXQoNCd0J7QlSDQntCR0KnQldCh0KLQktCeICLQndCQ0KbQmNCe0J3QkNCb0KzQndCr0JUg0JjQndCk0J7QoNCc0JDQptCY0J7QndCd0KvQlSDQotCV0KXQndCe0JvQntCT0JjQmCIxGDAWBgNVBAsMD0JJTjAwMDc0MDAwMDcyODCCASIwDQYJKoZIhvcNAQEBBQADggEPADCCAQoCggEBAIuFfpSCo5uenI8Cn9234hpG8uQZnRMBT2QwrndJaxIAHAMEQ9T3sxi2KWDL2mJArmVs48gdhhkm9JRVQCnYGb+OZ+nmePaeU826nawS83OVlJggIxbtAE4xdr8n2jCKMqjYgaQjXtllIC3NV5LvnZFeaoy/ndih+o3iNbiSfEoWsX1xB8MW7ZHIqZBnHYOQnnpSW9TRejYF4B+FC4Sr70IXFRYHuicNJZo4V9ijGiacBA3WWOITxOPd0TZcd54G4kNFSSzocGiqMLVtWGNWBKzRtqxHPKdvoK2M3oZ5Ihqzjf0RnuS30HSQsXzGtYE1d7b7y+R5fr+pWdfp3C2ECdUCAwEAAaOCAWQwggFgMBYGA1UdJQEB/wQMMAoGCCsGAQUFBwMIMA8GA1UdIwQIMAaABFtqdBEwHQYDVR0OBBYEFFqrRbGX3k3FKok5zH/A2RUXLTKfMFYGA1UdHwRPME0wS6BJoEeGIWh0dHA6Ly9jcmwucGtpLmdvdi5rei9uY2FfcnNhLmNybIYiaHR0cDovL2NybDEucGtpLmdvdi5rei9uY2FfcnNhLmNybDBaBgNVHS4EUzBRME+gTaBLhiNodHRwOi8vY3JsLnBraS5nb3Yua3ovbmNhX2RfcnNhLmNybIYkaHR0cDovL2NybDEucGtpLmdvdi5rei9uY2FfZF9yc2EuY3JsMGIGCCsGAQUFBwEBBFYwVDAuBggrBgEFBQcwAoYiaHR0cDovL3BraS5nb3Yua3ovY2VydC9uY2FfcnNhLmNlcjAiBggrBgEFBQcwAYYWaHR0cDovL29jc3AucGtpLmdvdi5rejANBgkqhkiG9w0BAQsFAAOCAgEApPnpRXedzDgnZBPKJPhOJ8dWXzuNu65vSymgTH1hDXwE1lR90n5kZvu/kHO1shsId6TEg6X/1IE6Gp8992JsTezkxUexb6MdWWnUYjNhuPYzsYfBMG+Z+UMjMhf2ER5SUjDiEXrdwqKkRPF5HfzuKEdQfjoFctYveqirISBdEOUIGD0LCiqqTD7W5dAIQaVbJZLGCG5nKgZ8rvzYPyQJU3AqizXeHglOWLxINWmYs69NCqmvk6XI7oFVyVe8UTLQ+qMtl1MfrNTvSHtU2Uei2n2NKcN1oof5E2RDASf3ImNDeSJuFKzIXnwdc9xmesfUCBVJJLgt4HU6RHh+5StrR+Wz7/OYBoVuhUE0YMSM/H74BrKbI6k8rmPw08QoU1v4cfYEH2jGzrJt4IephTkvxbv8ENcJX5LpUUux6oJHLTIzTU9q1BjitoODmV2EmmBn1hzU+YaSPEffIztXt8zuoVrH1gkT1s5bzc4nqc63OYur6XSTf6d66+2mavaVSQObNcYPz6YPnQFbDHzUy16oxX4FxlhnFSwul6gZg4b9kYh1BTdpXTGrfo4hD/c/4xF3OHWxprGnWb9uFxJqthhHkwEHloMcZaR5xSLT7kE4Lpy5uRz9qh4DVcbSTw2sqCJx/57YioRy7GUssbhG51+YBnDG9OEF7xhV6xdvF/SVVvwxggIwMIICLAIBATBqMFIxCzAJBgNVBAYTAktaMUMwQQYDVQQDDDrSsNCb0KLQotCr0pog0JrQo9OY0JvQkNCd0JTQq9Cg0KPQqNCrINCe0KDQotCQ0JvQq9KaIChSU0EpAhQ9neVtXyecXQbsfYuDqlC960N9NDANBglghkgBZQMEAgEFAKCBmDAaBgkqhkiG9w0BCQMxDQYLKoZIhvcNAQkQAQQwHAYJKoZIhvcNAQkFMQ8XDTIyMDMyNTE0NDE0MlowKwYLKoZIhvcNAQkQAgwxHDAaMBgwFgQUyyVFl1vW4xC+7Z6ZVsJ4kYybLQgwLwYJKoZIhvcNAQkEMSIEIPnmuHkbKsQRwwhb6RxZ4k/NGGAj+vfr5a7bTsg8D/ELMA0GCSqGSIb3DQEBCwUABIIBAEt1WmKBKyKabCPR4m90QbLpSZ+/ZjYDug3A3XDjQSNa7rbsKOUoJBu2r4ObcsSDNCfY9HN24o6EOnqnx2bPIzThTFgylzix09DxP9VhGvAoVhqAfdcKGjN6oNAxuDDvTv+hahc1gnzM0Q0gI2EIIGb2A1BWE86opiUUw+uV9hf9saWDredTAhll2CzNje2IrTwyf51zp+A5wZ/jlm0bzh7deY/EQ13yb/yjc94cCE/Z1Ijq63F6mwTl1EAehXMFZD/A/jKukFyHeW0aD8pYOB34Vth6PYuajZrFRg1Z93l03PSO7hY2Rr1OTFKAgRxViHojh19aV2lAoU9breFcgs0=&'
    
    'signData[18457977]=MIIP7gYJKoZIhvcNAQcCoIIP3zCCD9sCAQExDjAMBggqgw4DCgEDAQUAMAsGCSqGSIb3DQEHAaCCBNowggTWMIIEgKADAgECAhQsEW5waAgTAPIdPyOMUwKjbFTsfjANBgkqgw4DCgEBAQIFADBTMQswCQYDVQQGEwJLWjFEMEIGA1UEAww70rDQm9Ci0KLQq9KaINCa0KPTmNCb0JDQndCU0KvQoNCj0KjQqyDQntCg0KLQkNCb0KvSmiAoR09TVCkwHhcNMjEwNjA0MDc1MTU1WhcNMjIwNjA0MDc1MTU1WjCCAXUxKjAoBgNVBAMMIdCa0JDQmdCT0J7QoNCe0JTQptCV0JLQkCDQnNCQ0JnQrzEhMB8GA1UEBAwY0JrQkNCZ0JPQntCg0J7QlNCm0JXQktCQMRgwFgYDVQQFEw9JSU44MzA1MDQ0MDAwNDkxCzAJBgNVBAYTAktaMYGcMIGZBgNVBAoMgZHQotCe0JLQkNCg0JjQqdCV0KHQotCS0J4g0KEg0J7Qk9Cg0JDQndCY0KfQldCd0J3QntCZINCe0KLQktCV0KLQodCi0JLQldCd0J3QntCh0KLQrNCuICLQmNCd0JbQmNCd0JjQoNCY0J3Qk9Ce0JLQkNCvINCa0J7QnNCf0JDQndCY0K8gItCb0JjQlNCV0KAiMRgwFgYDVQQLDA9CSU4xMTA5NDAwMDQ5MDkxIzAhBgNVBCoMGtCh0KLQkNCd0JjQodCb0JDQktCe0JLQndCQMR8wHQYJKoZIhvcNAQkBFhBDRU9ATEVBREVSRU5HLktaMGwwJQYJKoMOAwoBAQEBMBgGCiqDDgMKAQEBAQEGCiqDDgMKAQMBAQADQwAEQOTfqVEh24kddh/aZlKhpjjqrTqnPx9OQicP3VjyFBJs193DrOnsUHVCnqkhqXSwisiXvQmmBA6PHMNg0hX+aSejggH2MIIB8jAOBgNVHQ8BAf8EBAMCBsAwMwYDVR0lBCwwKgYIKwYBBQUHAwQGCCqDDgMDBAECBgkqgw4DAwQBAgEGCSqDDgMDBAMCATAPBgNVHSMECDAGgARbanPpMB0GA1UdDgQWBBTeETtao0pNOO6INsPiri48jiPQbDBeBgNVHSAEVzBVMFMGByqDDgMDAgEwSDAhBggrBgEFBQcCARYVaHR0cDovL3BraS5nb3Yua3ovY3BzMCMGCCsGAQUFBwICMBcMFWh0dHA6Ly9wa2kuZ292Lmt6L2NwczBYBgNVHR8EUTBPME2gS6BJhiJodHRwOi8vY3JsLnBraS5nb3Yua3ovbmNhX2dvc3QuY3JshiNodHRwOi8vY3JsMS5wa2kuZ292Lmt6L25jYV9nb3N0LmNybDBcBgNVHS4EVTBTMFGgT6BNhiRodHRwOi8vY3JsLnBraS5nb3Yua3ovbmNhX2RfZ29zdC5jcmyGJWh0dHA6Ly9jcmwxLnBraS5nb3Yua3ovbmNhX2RfZ29zdC5jcmwwYwYIKwYBBQUHAQEEVzBVMC8GCCsGAQUFBzAChiNodHRwOi8vcGtpLmdvdi5rei9jZXJ0L25jYV9nb3N0LmNlcjAiBggrBgEFBQcwAYYWaHR0cDovL29jc3AucGtpLmdvdi5rejANBgkqgw4DCgEBAQIFAANBAF6nzMbWq5xVRWa5wASYXDr9StJo7NnVQSskyKrwhdnNDoRbtZuNoPiM5nt4FmOCS4qp5wLGCJADSDgT5k/0yDExggrZMIIK1QIBATBrMFMxCzAJBgNVBAYTAktaMUQwQgYDVQQDDDvSsNCb0KLQotCr0pog0JrQo9OY0JvQkNCd0JTQq9Cg0KPQqNCrINCe0KDQotCQ0JvQq9KaIChHT1NUKQIULBFucGgIEwDyHT8jjFMCo2xU7H4wDAYIKoMOAwoBAwEFAKCBojAYBgkqhkiG9w0BCQMxCwYJKoZIhvcNAQcBMBwGCSqGSIb3DQEJBTEPFw0yMjAzMjUxNDQxNDJaMC8GCSqGSIb3DQEJBDEiBCBfTZm2SguleHTRCPZ256RH7bOFR6sMpqLCbFmJ1Ug6MzA3BgsqhkiG9w0BCRACLzEoMCYwJDAiBCDtSxHdcyUCMLpJK8z8EZFN1Szhqk3bdjuIAn0o7cdZcTANBgkqgw4DCgEBAQIFAARAWzRWbC4xIWfWLPk1kN96TZtpzcCxvpym4pyHi339RB5/oiaYfJa5aJR8SxzBIx1ggfz4kIiVzBhEZ+ZMFgvMu6GCCV0wgglZBgsqhkiG9w0BCRACDjGCCUgwgglEBgkqhkiG9w0BBwKgggk1MIIJMQIBAzEPMA0GCWCGSAFlAwQCAQUAMIGEBgsqhkiG9w0BCRABBKB1BHMwcQIBAQYIKoMOAwMCBgIwMTANBglghkgBZQMEAgEFAAQg5Fs2mty1bH2+m+QDaWJcUuoNH2N9rfRfbr7NRD7Iq7ECFE3+opvxqvkhgkmnXDDcmto8r9rZGA8yMDIyMDMyNTE0NDE0MloCCAsNIuyea1DpoIIGXjCCBlowggRCoAMCAQICFD2d5W1fJ5xdBux9i4OqUL3rQ300MA0GCSqGSIb3DQEBCwUAMFIxCzAJBgNVBAYTAktaMUMwQQYDVQQDDDrSsNCb0KLQotCr0pog0JrQo9OY0JvQkNCd0JTQq9Cg0KPQqNCrINCe0KDQotCQ0JvQq9KaIChSU0EpMB4XDTE5MTIxMjA1MjIwNVoXDTIyMTIxMTA1MjIwNVowggESMRQwEgYDVQQDDAtUU0EgU0VSVklDRTEYMBYGA1UEBRMPSUlONzYxMjMxMzAwMzEzMQswCQYDVQQGEwJLWjEcMBoGA1UEBwwT0J3Qo9CgLdCh0KPQm9Ci0JDQnTEcMBoGA1UECAwT0J3Qo9CgLdCh0KPQm9Ci0JDQnTF9MHsGA1UECgx00JDQmtCm0JjQntCd0JXQoNCd0J7QlSDQntCR0KnQldCh0KLQktCeICLQndCQ0KbQmNCe0J3QkNCb0KzQndCr0JUg0JjQndCk0J7QoNCc0JDQptCY0J7QndCd0KvQlSDQotCV0KXQndCe0JvQntCT0JjQmCIxGDAWBgNVBAsMD0JJTjAwMDc0MDAwMDcyODCCASIwDQYJKoZIhvcNAQEBBQADggEPADCCAQoCggEBAIuFfpSCo5uenI8Cn9234hpG8uQZnRMBT2QwrndJaxIAHAMEQ9T3sxi2KWDL2mJArmVs48gdhhkm9JRVQCnYGb+OZ+nmePaeU826nawS83OVlJggIxbtAE4xdr8n2jCKMqjYgaQjXtllIC3NV5LvnZFeaoy/ndih+o3iNbiSfEoWsX1xB8MW7ZHIqZBnHYOQnnpSW9TRejYF4B+FC4Sr70IXFRYHuicNJZo4V9ijGiacBA3WWOITxOPd0TZcd54G4kNFSSzocGiqMLVtWGNWBKzRtqxHPKdvoK2M3oZ5Ihqzjf0RnuS30HSQsXzGtYE1d7b7y+R5fr+pWdfp3C2ECdUCAwEAAaOCAWQwggFgMBYGA1UdJQEB/wQMMAoGCCsGAQUFBwMIMA8GA1UdIwQIMAaABFtqdBEwHQYDVR0OBBYEFFqrRbGX3k3FKok5zH/A2RUXLTKfMFYGA1UdHwRPME0wS6BJoEeGIWh0dHA6Ly9jcmwucGtpLmdvdi5rei9uY2FfcnNhLmNybIYiaHR0cDovL2NybDEucGtpLmdvdi5rei9uY2FfcnNhLmNybDBaBgNVHS4EUzBRME+gTaBLhiNodHRwOi8vY3JsLnBraS5nb3Yua3ovbmNhX2RfcnNhLmNybIYkaHR0cDovL2NybDEucGtpLmdvdi5rei9uY2FfZF9yc2EuY3JsMGIGCCsGAQUFBwEBBFYwVDAuBggrBgEFBQcwAoYiaHR0cDovL3BraS5nb3Yua3ovY2VydC9uY2FfcnNhLmNlcjAiBggrBgEFBQcwAYYWaHR0cDovL29jc3AucGtpLmdvdi5rejANBgkqhkiG9w0BAQsFAAOCAgEApPnpRXedzDgnZBPKJPhOJ8dWXzuNu65vSymgTH1hDXwE1lR90n5kZvu/kHO1shsId6TEg6X/1IE6Gp8992JsTezkxUexb6MdWWnUYjNhuPYzsYfBMG+Z+UMjMhf2ER5SUjDiEXrdwqKkRPF5HfzuKEdQfjoFctYveqirISBdEOUIGD0LCiqqTD7W5dAIQaVbJZLGCG5nKgZ8rvzYPyQJU3AqizXeHglOWLxINWmYs69NCqmvk6XI7oFVyVe8UTLQ+qMtl1MfrNTvSHtU2Uei2n2NKcN1oof5E2RDASf3ImNDeSJuFKzIXnwdc9xmesfUCBVJJLgt4HU6RHh+5StrR+Wz7/OYBoVuhUE0YMSM/H74BrKbI6k8rmPw08QoU1v4cfYEH2jGzrJt4IephTkvxbv8ENcJX5LpUUux6oJHLTIzTU9q1BjitoODmV2EmmBn1hzU+YaSPEffIztXt8zuoVrH1gkT1s5bzc4nqc63OYur6XSTf6d66+2mavaVSQObNcYPz6YPnQFbDHzUy16oxX4FxlhnFSwul6gZg4b9kYh1BTdpXTGrfo4hD/c/4xF3OHWxprGnWb9uFxJqthhHkwEHloMcZaR5xSLT7kE4Lpy5uRz9qh4DVcbSTw2sqCJx/57YioRy7GUssbhG51+YBnDG9OEF7xhV6xdvF/SVVvwxggIwMIICLAIBATBqMFIxCzAJBgNVBAYTAktaMUMwQQYDVQQDDDrSsNCb0KLQotCr0pog0JrQo9OY0JvQkNCd0JTQq9Cg0KPQqNCrINCe0KDQotCQ0JvQq9KaIChSU0EpAhQ9neVtXyecXQbsfYuDqlC960N9NDANBglghkgBZQMEAgEFAKCBmDAaBgkqhkiG9w0BCQMxDQYLKoZIhvcNAQkQAQQwHAYJKoZIhvcNAQkFMQ8XDTIyMDMyNTE0NDE0MlowKwYLKoZIhvcNAQkQAgwxHDAaMBgwFgQUyyVFl1vW4xC+7Z6ZVsJ4kYybLQgwLwYJKoZIhvcNAQkEMSIEIKXvcBvdzQ/WjKLzP6Qt8YeZwfJ1FmiZwHU5OAVvNCv2MA0GCSqGSIb3DQEBCwUABIIBAE8VVUU4SOujXRlsNrs5Q7KaTco9DnpAy11Z+7ir8fw7ZOXIoTSLVXgm1AmxWdZg9CTL50H/bhDC8IItBPgwEgKIOHiYwel3M7QLFGqL5IUVO+uDLY1Iw4Yp8z5+aBfn3CAppjWyZxS8CtkdiiQ5CsziIOjR5htjqcS/volUbqiIO8Cp+v4WSmDCnTEfra4+SZCG4aOEZytuVnLjQBIRchDsf3ovaMaPe4sWeEXi5ecREeKWDRRmgMioMqH/Dm1ni1v0XsnLAmypt/uMPkeG9tTu0/QBm74QuHnoDHWru4HOM6amN65AZ+T6TtJS5ShAHEZGP+3vUZJZxALYrCCCo28=&'
    
    'formData=offer%5B34272171%5D%5B18457975%5D%5Bprice%5D=11&offer%5B34272172%5D%5B18457976%5D%5Bprice%5D=22&offer%5B34272173%5D%5B18457977%5D%5Bprice%5D=33'
)

In [236]:
'xmlData%5B18457975%5D=%3C%3Fxml+version%3D%221.0%22+encoding%3D%22UTF-8%22%3F%3E'

'%3Cpriceoffer%3E%3Ctrd_buy_id%3E7196072%3C%2Ftrd_buy_id%3E%3Ctrd_buy_lot_id%3E19114250%3C%2Ftrd_buy_lot_id%3E%3Ctrd_buy_lot_point_id%3E18457975%3C%2Ftrd_buy_lot_point_id%3E%3Cprice%3E60%3C%2Fprice%3E%3C%2Fpriceoffer%3E&'

'xmlData%5B18457976%5D=%3C%3Fxml+version%3D%221.0%22+encoding%3D%22UTF-8%22%3F%3E%3Cpriceoffer%3E%3Ctrd_buy_id%3E7196072%3C%2Ftrd_buy_id%3E%3Ctrd_buy_lot_id%3E19114251%3C%2Ftrd_buy_lot_id%3E%3Ctrd_buy_lot_point_id%3E18457976%3C%2Ftrd_buy_lot_point_id%3E%3Cprice%3E70%3C%2Fprice%3E%3C%2Fpriceoffer%3E&'

'xmlData%5B18457977%5D=%3C%3Fxml+version%3D%221.0%22+encoding%3D%22UTF-8%22%3F%3E%3Cpriceoffer%3E%3Ctrd_buy_id%3E7196072%3C%2Ftrd_buy_id%3E%3Ctrd_buy_lot_id%3E19114252%3C%2Ftrd_buy_lot_id%3E%3Ctrd_buy_lot_point_id%3E18457977%3C%2Ftrd_buy_lot_point_id%3E%3Cprice%3E50%3C%2Fprice%3E%3C%2Fpriceoffer%3E&'

'signData%5B18457975%5D=&'
'signData%5B18457976%5D=&'
'signData%5B18457977%5D='

'xmlData%5B18457975%5D=%3C%3Fxml+version%3D%221.0%22+encoding%3D%22UTF-8%22%3F%3E%3Cpriceoffer%3E%3Ctrd_buy_id%3E7196072%3C%2Ftrd_buy_id%3E%3Ctrd_buy_lot_id%3E19114250%3C%2Ftrd_buy_lot_id%3E%3Ctrd_buy_lot_point_id%3E18457975%3C%2Ftrd_buy_lot_point_id%3E%3Cprice%3E60%3C%2Fprice%3E%3C%2Fpriceoffer%3E&xmlData%5B18457976%5D=%3C%3Fxml+version%3D%221.0%22+encoding%3D%22UTF-8%22%3F%3E%3Cpriceoffer%3E%3Ctrd_buy_id%3E7196072%3C%2Ftrd_buy_id%3E%3Ctrd_buy_lot_id%3E19114251%3C%2Ftrd_buy_lot_id%3E%3Ctrd_buy_lot_point_id%3E18457976%3C%2Ftrd_buy_lot_point_id%3E%3Cprice%3E70%3C%2Fprice%3E%3C%2Fpriceoffer%3E&xmlData%5B18457977%5D=%3C%3Fxml+version%3D%221.0%22+encoding%3D%22UTF-8%22%3F%3E%3Cpriceoffer%3E%3Ctrd_buy_id%3E7196072%3C%2Ftrd_buy_id%3E%3Ctrd_buy_lot_id%3E19114252%3C%2Ftrd_buy_lot_id%3E%3Ctrd_buy_lot_point_id%3E18457977%3C%2Ftrd_buy_lot_point_id%3E%3Cprice%3E50%3C%2Fprice%3E%3C%2Fpriceoffer%3E&signData%5B18457975%5D=MIIP7gYJKoZIhvcNAQcCoIIP3zCCD9sCAQExDjAMBggqgw4DCgEDAQUAMAsGCSqGSIb3DQEHAaCCBNo

In [245]:
sig = 'MIIP7gYJKoZIhvcNAQcCoIIP3zCCD9sCAQExDjAMBggqgw4DCgEDAQUAMAsGCSqGSIb3DQEHAaCCBNowggTWMIIEgKADAgECAhQsEW5waAgTAPIdPyOMUwKjbFTsfjANBgkqgw4DCgEBAQIFADBTMQswCQYDVQQGEwJLWjFEMEIGA1UEAww70rDQm9Ci0KLQq9KaINCa0KPTmNCb0JDQndCU0KvQoNCj0KjQqyDQntCg0KLQkNCb0KvSmiAoR09TVCkwHhcNMjEwNjA0MDc1MTU1WhcNMjIwNjA0MDc1MTU1WjCCAXUxKjAoBgNVBAMMIdCa0JDQmdCT0J7QoNCe0JTQptCV0JLQkCDQnNCQ0JnQrzEhMB8GA1UEBAwY0JrQkNCZ0JPQntCg0J7QlNCm0JXQktCQMRgwFgYDVQQFEw9JSU44MzA1MDQ0MDAwNDkxCzAJBgNVBAYTAktaMYGcMIGZBgNVBAoMgZHQotCe0JLQkNCg0JjQqdCV0KHQotCS0J4g0KEg0J7Qk9Cg0JDQndCY0KfQldCd0J3QntCZINCe0KLQktCV0KLQodCi0JLQldCd0J3QntCh0KLQrNCuICLQmNCd0JbQmNCd0JjQoNCY0J3Qk9Ce0JLQkNCvINCa0J7QnNCf0JDQndCY0K8gItCb0JjQlNCV0KAiMRgwFgYDVQQLDA9CSU4xMTA5NDAwMDQ5MDkxIzAhBgNVBCoMGtCh0KLQkNCd0JjQodCb0JDQktCe0JLQndCQMR8wHQYJKoZIhvcNAQkBFhBDRU9ATEVBREVSRU5HLktaMGwwJQYJKoMOAwoBAQEBMBgGCiqDDgMKAQEBAQEGCiqDDgMKAQMBAQADQwAEQOTfqVEh24kddh/aZlKhpjjqrTqnPx9OQicP3VjyFBJs193DrOnsUHVCnqkhqXSwisiXvQmmBA6PHMNg0hX+aSejggH2MIIB8jAOBgNVHQ8BAf8EBAMCBsAwMwYDVR0lBCwwKgYIKwYBBQUHAwQGCCqDDgMDBAECBgkqgw4DAwQBAgEGCSqDDgMDBAMCATAPBgNVHSMECDAGgARbanPpMB0GA1UdDgQWBBTeETtao0pNOO6INsPiri48jiPQbDBeBgNVHSAEVzBVMFMGByqDDgMDAgEwSDAhBggrBgEFBQcCARYVaHR0cDovL3BraS5nb3Yua3ovY3BzMCMGCCsGAQUFBwICMBcMFWh0dHA6Ly9wa2kuZ292Lmt6L2NwczBYBgNVHR8EUTBPME2gS6BJhiJodHRwOi8vY3JsLnBraS5nb3Yua3ovbmNhX2dvc3QuY3JshiNodHRwOi8vY3JsMS5wa2kuZ292Lmt6L25jYV9nb3N0LmNybDBcBgNVHS4EVTBTMFGgT6BNhiRodHRwOi8vY3JsLnBraS5nb3Yua3ovbmNhX2RfZ29zdC5jcmyGJWh0dHA6Ly9jcmwxLnBraS5nb3Yua3ovbmNhX2RfZ29zdC5jcmwwYwYIKwYBBQUHAQEEVzBVMC8GCCsGAQUFBzAChiNodHRwOi8vcGtpLmdvdi5rei9jZXJ0L25jYV9nb3N0LmNlcjAiBggrBgEFBQcwAYYWaHR0cDovL29jc3AucGtpLmdvdi5rejANBgkqgw4DCgEBAQIFAANBAF6nzMbWq5xVRWa5wASYXDr9StJo7NnVQSskyKrwhdnNDoRbtZuNoPiM5nt4FmOCS4qp5wLGCJADSDgT5k/0yDExggrZMIIK1QIBATBrMFMxCzAJBgNVBAYTAktaMUQwQgYDVQQDDDvSsNCb0KLQotCr0pog0JrQo9OY0JvQkNCd0JTQq9Cg0KPQqNCrINCe0KDQotCQ0JvQq9KaIChHT1NUKQIULBFucGgIEwDyHT8jjFMCo2xU7H4wDAYIKoMOAwoBAwEFAKCBojAYBgkqhkiG9w0BCQMxCwYJKoZIhvcNAQcBMBwGCSqGSIb3DQEJBTEPFw0yMjAzMjUxNDQxNDJaMC8GCSqGSIb3DQEJBDEiBCB8qjCGZVAshUyFnm7LwZdksVaMcWO7JSLPXDYuSYXOezA3BgsqhkiG9w0BCRACLzEoMCYwJDAiBCDtSxHdcyUCMLpJK8z8EZFN1Szhqk3bdjuIAn0o7cdZcTANBgkqgw4DCgEBAQIFAARA2lFpl3CCwbWu0Wjqc1xKrMSwFF63Ns1azC1O3wef/lr95m5J2Lj7DoDa/pQXcLpeiS8gEHdybm4D6pO80RNVnaGCCV0wgglZBgsqhkiG9w0BCRACDjGCCUgwgglEBgkqhkiG9w0BBwKgggk1MIIJMQIBAzEPMA0GCWCGSAFlAwQCAQUAMIGEBgsqhkiG9w0BCRABBKB1BHMwcQIBAQYIKoMOAwMCBgIwMTANBglghkgBZQMEAgEFAAQgRYxRFBpofODghuXC0QFXNTuphL8MZLwFlfYzy+ITU0kCFLjkV3bRy2zEoWnYReuD+IxxSBrtGA8yMDIyMDMyNTE0NDE0M1oCCD1E6MIDnqBeoIIGXjCCBlowggRCoAMCAQICFD2d5W1fJ5xdBux9i4OqUL3rQ300MA0GCSqGSIb3DQEBCwUAMFIxCzAJBgNVBAYTAktaMUMwQQYDVQQDDDrSsNCb0KLQotCr0pog0JrQo9OY0JvQkNCd0JTQq9Cg0KPQqNCrINCe0KDQotCQ0JvQq9KaIChSU0EpMB4XDTE5MTIxMjA1MjIwNVoXDTIyMTIxMTA1MjIwNVowggESMRQwEgYDVQQDDAtUU0EgU0VSVklDRTEYMBYGA1UEBRMPSUlONzYxMjMxMzAwMzEzMQswCQYDVQQGEwJLWjEcMBoGA1UEBwwT0J3Qo9CgLdCh0KPQm9Ci0JDQnTEcMBoGA1UECAwT0J3Qo9CgLdCh0KPQm9Ci0JDQnTF9MHsGA1UECgx00JDQmtCm0JjQntCd0JXQoNCd0J7QlSDQntCR0KnQldCh0KLQktCeICLQndCQ0KbQmNCe0J3QkNCb0KzQndCr0JUg0JjQndCk0J7QoNCc0JDQptCY0J7QndCd0KvQlSDQotCV0KXQndCe0JvQntCT0JjQmCIxGDAWBgNVBAsMD0JJTjAwMDc0MDAwMDcyODCCASIwDQYJKoZIhvcNAQEBBQADggEPADCCAQoCggEBAIuFfpSCo5uenI8Cn9234hpG8uQZnRMBT2QwrndJaxIAHAMEQ9T3sxi2KWDL2mJArmVs48gdhhkm9JRVQCnYGb+OZ+nmePaeU826nawS83OVlJggIxbtAE4xdr8n2jCKMqjYgaQjXtllIC3NV5LvnZFeaoy/ndih+o3iNbiSfEoWsX1xB8MW7ZHIqZBnHYOQnnpSW9TRejYF4B+FC4Sr70IXFRYHuicNJZo4V9ijGiacBA3WWOITxOPd0TZcd54G4kNFSSzocGiqMLVtWGNWBKzRtqxHPKdvoK2M3oZ5Ihqzjf0RnuS30HSQsXzGtYE1d7b7y+R5fr+pWdfp3C2ECdUCAwEAAaOCAWQwggFgMBYGA1UdJQEB/wQMMAoGCCsGAQUFBwMIMA8GA1UdIwQIMAaABFtqdBEwHQYDVR0OBBYEFFqrRbGX3k3FKok5zH/A2RUXLTKfMFYGA1UdHwRPME0wS6BJoEeGIWh0dHA6Ly9jcmwucGtpLmdvdi5rei9uY2FfcnNhLmNybIYiaHR0cDovL2NybDEucGtpLmdvdi5rei9uY2FfcnNhLmNybDBaBgNVHS4EUzBRME+gTaBLhiNodHRwOi8vY3JsLnBraS5nb3Yua3ovbmNhX2RfcnNhLmNybIYkaHR0cDovL2NybDEucGtpLmdvdi5rei9uY2FfZF9yc2EuY3JsMGIGCCsGAQUFBwEBBFYwVDAuBggrBgEFBQcwAoYiaHR0cDovL3BraS5nb3Yua3ovY2VydC9uY2FfcnNhLmNlcjAiBggrBgEFBQcwAYYWaHR0cDovL29jc3AucGtpLmdvdi5rejANBgkqhkiG9w0BAQsFAAOCAgEApPnpRXedzDgnZBPKJPhOJ8dWXzuNu65vSymgTH1hDXwE1lR90n5kZvu/kHO1shsId6TEg6X/1IE6Gp8992JsTezkxUexb6MdWWnUYjNhuPYzsYfBMG+Z+UMjMhf2ER5SUjDiEXrdwqKkRPF5HfzuKEdQfjoFctYveqirISBdEOUIGD0LCiqqTD7W5dAIQaVbJZLGCG5nKgZ8rvzYPyQJU3AqizXeHglOWLxINWmYs69NCqmvk6XI7oFVyVe8UTLQ+qMtl1MfrNTvSHtU2Uei2n2NKcN1oof5E2RDASf3ImNDeSJuFKzIXnwdc9xmesfUCBVJJLgt4HU6RHh+5StrR+Wz7/OYBoVuhUE0YMSM/H74BrKbI6k8rmPw08QoU1v4cfYEH2jGzrJt4IephTkvxbv8ENcJX5LpUUux6oJHLTIzTU9q1BjitoODmV2EmmBn1hzU+YaSPEffIztXt8zuoVrH1gkT1s5bzc4nqc63OYur6XSTf6d66+2mavaVSQObNcYPz6YPnQFbDHzUy16oxX4FxlhnFSwul6gZg4b9kYh1BTdpXTGrfo4hD/c/4xF3OHWxprGnWb9uFxJqthhHkwEHloMcZaR5xSLT7kE4Lpy5uRz9qh4DVcbSTw2sqCJx/57YioRy7GUssbhG51+YBnDG9OEF7xhV6xdvF/SVVvwxggIwMIICLAIBATBqMFIxCzAJBgNVBAYTAktaMUMwQQYDVQQDDDrSsNCb0KLQotCr0pog0JrQo9OY0JvQkNCd0JTQq9Cg0KPQqNCrINCe0KDQotCQ0JvQq9KaIChSU0EpAhQ9neVtXyecXQbsfYuDqlC960N9NDANBglghkgBZQMEAgEFAKCBmDAaBgkqhkiG9w0BCQMxDQYLKoZIhvcNAQkQAQQwHAYJKoZIhvcNAQkFMQ8XDTIyMDMyNTE0NDE0M1owKwYLKoZIhvcNAQkQAgwxHDAaMBgwFgQUyyVFl1vW4xC+7Z6ZVsJ4kYybLQgwLwYJKoZIhvcNAQkEMSIEIMpzqR3sHGmOGu0XJVHsXFw6dVa3w8ktEK9f353qhdRFMA0GCSqGSIb3DQEBCwUABIIBAD3GFSl6rkv1UraXHbCbYGNaPwT0/tqAEMH1JgWMa2v8CQiyXERifjSTz+0RuKt6JDbvbRW0FWwjwVPdF4Rn79Xl2MubC7kwGkbgoYwKZqDGUxj+kr3Ji0eW0C5n4HAHY/sHFa3A84oAleCUExFVnVnYgeqXIpy27tQ19PqcudRDsRgMTTprak8nhq8NEa6DVd83PCsSEgpKvUEmkP55wv3tCV8eBC49owswvWNKQfycr8VR1ihAvuFqdD9dPbDaCU9ArUjgIHxv9gM6+EdIKK9Nht8QL6K1ySli7VHOPiqUFmVtXmTgpHXeJRYNvEbw6o1+RFn6WYVMbwBwijOCzMc='

In [348]:
quote(unquote(sig), safe='&=+/') == sig

True

In [349]:
compare('signData%5B18457977%5D=&', '&=+/')

True

In [15]:
xmlstring=(
    '<?xml version="1.0" encoding="UTF-8"?>'
    '<priceoffer>'
    '<trd_buy_id>7196072</trd_buy_id>'
    '<trd_buy_lot_id>19114250</trd_buy_lot_id>'
    '<trd_buy_lot_point_id>18457975</trd_buy_lot_point_id>'
    '<price>466087.62</price>'
    '</priceoffer>'
)

In [ ]:
xmlstring_dummy=(
    f'<?xml version="1.0" encoding="UTF-8"?>'
    f'<priceoffer>'
    f'<trd_buy_id>{order_id}</trd_buy_id>'
    f'<trd_buy_lot_id>{buy_lot_id}</trd_buy_lot_id>'
    f'<trd_buy_lot_point_id>{buy_lot_point_id}</trd_buy_lot_point_id>'
    f'<price>{price}</price>'
    f'</priceoffer>'
)


# price - sami
# order_id - sami

# ostalnoe
# buy_lot_id          td[8]/div/@trd_lot_id
# buy_lot_point_id    td[8]/div/@buy_lot_point_id

In [44]:
import requests
import json

In [18]:
data = {
    "version": "1.0",
    "method": "XML.sign",
    "params": {
        "p12":"MIINkwIBAzCCDU0GCSqGSIb3DQEHAaCCDT4Egg06MIAwgAYJKoZIhvcNAQcBoIAEggWcMIIFmDCCBZQGCyqGSIb3DQEMCgECoIIE+jCCBPYwKAYKKoZIhvcNAQwBAzAaBBQzwa8hA6IVQ8Dkz2V+mhYE/UCE7gICBAAEggTI5tLwNi/DoOWJ7D1S9vKrozFgsHxv/Yjw9jS5C1JNN55xtazMJ3i8pKvvVbY7WqLwXpsJWaS6297OKPkUW5uc9jNLDS8cc37PgaxhA05gKyO97RdGlglnEqxJsy7qsQIEIT66FYOoNnmxpV2Tw3bjEm2xlNLLbbbETnjCyF+M7MFm8g6wA61hisDy1areY3ypTA4Qn5eMoKSvhxlYC+FEAgtLMizwBetfOgkJJk0pBehA+jmmvAzfKikTVW2b7wVtbbISclhkGUoJCh7UojnYNCm5Bbar4MZ+w8lfyLChyFeqazWYzysqjWn1F73EuhFPNvXiCIxPrZ7OJg2lyiYyASCmo46irT60pJerZ0DPAFbAShYfKfg/hvEDuF7/nmdscTp7vaLJic7cQm3i2AkMmB+raPM3UfjYgPwpTKuWL93JRUGQk5UigSZKZIoXDPjJaaxK8RuBtRwEEIkOg4Ip6WBT2R0l23tyP6A+YqlnDvvBSFWrE/8qv2DvNtURt9dVBA+WpmJUBKdisDZxzMaqtva2Z6YGejC1h1BPlCxhzDWmKLyacGbzuhSJvp4lOJmTugNaFUZLLdDgmaTQm4pQ+luhEZiT8X468DVWQYrchwkwmk//xCbp7Tu+7vxljDYZbSDCkHQVCKCUE+B9TN74H4B9zaxpychTXPJ5uIrOQhEUkj1ZpSX9qIwzn8dM2FWqrv1qZEgjcbgmKzYvSoUKZAJJzGBxnRQIIwH6bA0hFXiNQCExhigIz+H02KWqWc3Qmg987y9sr81debtsbceUQR3faGCnRT9cc3ROk9x2kOIcKdX9mDYXh4/sbDtTYiyE5r0A0UtfH0roXANipikayb3U0TQxpjOoMCaAmWRJn1DUtpXD1/3TYBVNQvEl3cysCHFBod/UmmuAS9ZG3OH55R4dXkFFvLum70rGZ9RKjTRJ/4pj414JgBky6jIS2/d3Uww4uCoLa6gtmZN49HwUxAe6/3Fdda27lf++ZbCWAXpqYrJ+vBn3XtqgOLoulqlAW3yITsgidjqx3iXzQPzeucSzJRZbb+2X5Ub6ngsuFcJij7A7/NpfApxjcmRR5YlVTlTr9p+ZZChhRKNamWuf8+SU8kguFlCViZ9H2mhhBcWYjAIWRpPAUAwYUDds5lWfDQofZtS7yQIjUmw4UKrQr0Z7WiN05jx/kbPshWSkrrnWeWNwDwrbFtRA9UHEgwNR4l9m/OdKxDcEX4X/CAX9JriurZonM1BfTX2F2D63rT6bdrOdAa7qNUP+s0Y0VZY8DPqsB0QyYBA6wTnfgZNAnISfnENRlyNTv6/C/6pULFkDztPPRHPThrWRf3ye/n9QrBN2Co6bFkPJ+aRPZhNe4cKAVMBO32Olh6htM8+liWjDeAcKSyoDBWVNYIEt2c5Xg3XHZdpc9L2rNhtypDcXVpjXP/kJ/XZBXojfgQ+SqoWOK0xH+XmVVA8ltNxF4BqgfWLp/dYiaciWTrsj8sazKjhlndiGR7B8oIZ7WbhU0u7+gOsVRQHaFWvZP0CHQvVbObXw4FvtVWgaC6ONztmcFwE4rhYaM+Ju7mcUCY7zCNAL8+AJ0Mxl8fNtkpMXnkt2+5W+ciFHgI2Yv32A55ZHEJJnpwYcwPdlMYGGMCMGCSqGSIb3DQEJFTEWBBRrNhuGTGeWAbZS/jh/YfzZMDwDJzBfBgkqhkiG9w0BCRQxUh5QADYAYgAzADYAMQBiADgANgA0AGMANgA3ADkANgAwADEAYgA2ADUAMgBmAGUAMwA4ADcAZgA2ADEAZgBjAGQAOQAzADAAMwBjADAAMwAyADcAAAAAMIAGCSqGSIb3DQEHBqCAMIACAQAwgAYJKoZIhvcNAQcBMCgGCiqGSIb3DQEMAQYwGgQU5vJcSheCI7r7SLHaZyY7j3lBctcCAgQAoIAEggcokn/1oGk1ZlYLNjFGH1ji2TTrHVUGsVGOcxr9q04HOdQnFWEpA9j3yUGKoBO9MaWUNAiQlAYVUY1JPncr2fdw5bZvWFoQrokox9eUAd2PTMgekUTqjRPv3Mx4/0ej16Wbs0DOTO28iHmUFS8h+dgV5WZQ92EjsQk40rYSCFet6E1GUVAboEVH9T6oVNP8JoHzxmqMiPbmtCqZNlJ3VWbKS1FM1o/8FI+7GKfJJw4+u0lqItpRODHsnboyvDsKqMMOk1Wj9ORH5wTkRRxnELuAemgu5s68SktPv5JDw1O9sUYJrhUnymwG8uyB2fByuudvvGHopU+6iZt7bTYvg8r13JheN/BnYqzzA33U1DIf9LahGqtH7fqNXZzT2BKubqgIZMPDCrx3CisbsUGTV8y7BbYvaD7QZdZrdQnNGrnR7C0bZSr4xUfPjkB/CCi5m6h7jT6FhTH8FU7wY7jpN3/H4fqL0eYw82FAfctcmNk5RLsLbeZ9E8nnhdnF/lIjGmOZJiJGli9BnXfylfvpdTxSQ45vfJsCokLczAZ72Mmvrqg67KvfoKfAjPqIcpTkfqrAZ4d3mNVxx0+bHxUdnLPgg7pp9FfVWZnWClKhV/tjgQ5Dj1tbRH69PnbNdRhpQarZksPE/nDamBQwbFSHAI93D5VTeDElrzuZ7j3T5K/rf8p1aEq4LWSuEMSC16UL7lSwXnan+bHujmviCcx9CdoN1+wDICtKKrWqsyFZoB6LtFdsgSVDQ6zalMGI+0M7c1l8pd92fZON19R9LeSfIQRXpNtYUiEDGlNEFILrZuh1klzLxxbpx7y+S9qPVzhlpoYuDMEbLgv0nPTPz+4LlL8DCx+2v/0cMzjKQbcJ016Vzs0rL5bS1gwb2aZ0ldT4fOSGyUP42YVHxpOw71qcymHmomiNZR9OXaJmarND3bl4ugsbrFPFGsBz1+p7SIwIjCicOj5km2SqYLIdQIubdfLHONtE3J50/NyfUmhySVt3ovdzlEZXpSJw6vGQzRPTr1jxOkLKBRaZ9KymyG/tXMnfRXaiKQ1nh/GZ4S5dlmQCx5YxocYSMavL+HgaBUTSuPM4qs86GRVG9lS/2Vkh9ft/2ufrxUFS9rVPeX8f+saCfDHVlu3P7FLYy/t5pXpnGMh/AjpPAf+wRsxg7xlUPgbfQlvMFzISQCsJawNkyZfoLoxwTilySNNpfMZeigTgHO2gporSosahc5ikTYK+HJLIJNhcnfwmmv+VkdpAZ8nEX8Olxqo6mgVbmNkgAOQUhtva9BkCNx072RgdqHSGm9KFvZD7Jtb6XGZZRFUNM0DNqvMM7rYBfv51o8N6t6corCcOmAJQZ77+nNc6ozF/ZIy8Uy5zRs6Mwa7C9LQ3TiAL7NZHjvA4xnkQ/Srszu6uvKB43VjzPC14pyeWnFVYJbEDf7V4LWU1UZMSRfq+ird7VKgS4AW4CXXsiJfjYkZKHEeYtvnveMAeXlUFWhintyIS8HOU9/e+t2WjTciaXvra1L/YzbKGjri1ECdCUhigFMttCGx/G2PJOcfMNFugz62bJrt6RWMr12WeqpWXGKOqIHaPbTvudnOC0jjD8sBLK2AvFsKujFTfkxOyoTl6BF3zgdSh00qef/xFwhTCNYFSQSQJGFGjw8RhvjAZzow9w37ccN1nUoZopw1aoeimrHKf3s5M6VUhsrpvtphPabIGd5BGTefJKKAUeR0t6BdettRyWT3AsjjM95eDKrcfioCBBPJHOpVdD3YY5o3RawoHLCYCSs8KUaNfTxIv/2/ZQVYQJFTFjLGvjj6WD0qmrCkscEHFRDH++XmJYfuBilF4YLNDC01qJUQdEPATOpSFyVSfJM9oh9LLE0QqC2LfxmxmSBxfNw0ISvACjto1G7xTYYP242DlqL2kqeGuZSYGFpQZIyXzAzYWeotdBjJD3hgGjuuP6wDw5Jyat8U6L1KutYkGDowrxC380rXbRCfbLY5rZvQkHXgXmLHNunyXrM4kqWFwJLPTapptXmkClye8+9O+DfsXkW/rQ8qSWeZnOVPCb3PTIwDAksPpRxjuy4V36z8Vl2GeePSGHWTVxGt383AR50kc5bR1ERw8A4gv6px1g5HWIjnIkP9kIYrvprP4x1rQ0T+B/Tmbf+nMb/oFIs4YQsl+3GlSYph0dAnSx3oaRhFSurU/cTgjPs3nnRTkKc+XX7A5snLtnthD4ZLwO14sPcvHwPAFyEAJXwNTpLbetQpCNUZjUg/id1bGSsp22Q9BaT/GeQyCtEGnRn/UBBLTckoULZi75C2XigUxjrr3o2aGlKJK/CQ01Z25y4G9eYslgLGexuRZs8Lijnx5kznyX7hQEz0wOfQ6DUZH4YMIzsIQa9W06v5GKNRcevZMG+wCfUEtR7yrZzn9TmzregOPGu780WwQt33HZ27Weza5/nI26gaphTYAAAAAAAAAAAAAAAAwPTAhMAkGBSsOAwIaBQAEFOy6U+YkvM72T+xeraexxa7kiKq9BBQctlNgZGGVCnKyqLcEv7BtQC2u/QICBAA=",
        "password":"Qwerty12",
        "xml": "<?xml version=\"1.0\" encoding=\"utf-8\"?><root><name>NCANode</name></root>"
    }
}

In [363]:
'23'.encode('utf-8')

b'23'

In [71]:
from base64 import b64encode
import json

def sign(string):
    cert_path='/home/necuk/Downloads/key2/GOSTKNCA_de113b5aa34a4d38ee8836c3e2ae2e3c8e23d06c.p12'
    with open (cert_path, 'rb') as f:
        cert = b64encode(f.read()).decode('utf-8')
    
    data = {
        "version": "2.0",
        "method": "cms.sign",
        "params": {
            "data": b64encode(s).decode('utf-8'),
            "p12array": [
                {
                    "alias": "",
                    "p12": cert,
                    "password":"baRsa01022020"
                }
            ]
        }
    }
    
    
    headers = {
        'Content-Type': 'application/json'
    }
    
    r = requests.post(
        url='http://127.0.0.1:14579', 
        data=json.dumps(data),
    #     data = data,
        headers=headers
    )

    print(r.status_code)
    
    return json.loads(r.text)['cms']

In [357]:
xml_string

'<?xml version="1.0" encoding="utf-8"?><root><name>NCANode</name></root>'

In [358]:
sign(xml_string)

200


'MIIGngYJKoZIhvcNAQcCoIIGjzCCBosCAQExDjAMBggqgw4DCgEDAQUAMFYGCSqGSIb3DQEHAaBJBEc8P3htbCB2ZXJzaW9uPSIxLjAiIGVuY29kaW5nPSJ1dGYtOCI/Pjxyb290PjxuYW1lPk5DQU5vZGU8L25hbWU+PC9yb290PqCCBNowggTWMIIEgKADAgECAhQsEW5waAgTAPIdPyOMUwKjbFTsfjANBgkqgw4DCgEBAQIFADBTMQswCQYDVQQGEwJLWjFEMEIGA1UEAww70rDQm9Ci0KLQq9KaINCa0KPTmNCb0JDQndCU0KvQoNCj0KjQqyDQntCg0KLQkNCb0KvSmiAoR09TVCkwHhcNMjEwNjA0MDc1MTU1WhcNMjIwNjA0MDc1MTU1WjCCAXUxKjAoBgNVBAMMIdCa0JDQmdCT0J7QoNCe0JTQptCV0JLQkCDQnNCQ0JnQrzEhMB8GA1UEBAwY0JrQkNCZ0JPQntCg0J7QlNCm0JXQktCQMRgwFgYDVQQFEw9JSU44MzA1MDQ0MDAwNDkxCzAJBgNVBAYTAktaMYGcMIGZBgNVBAoMgZHQotCe0JLQkNCg0JjQqdCV0KHQotCS0J4g0KEg0J7Qk9Cg0JDQndCY0KfQldCd0J3QntCZINCe0KLQktCV0KLQodCi0JLQldCd0J3QntCh0KLQrNCuICLQmNCd0JbQmNCd0JjQoNCY0J3Qk9Ce0JLQkNCvINCa0J7QnNCf0JDQndCY0K8gItCb0JjQlNCV0KAiMRgwFgYDVQQLDA9CSU4xMTA5NDAwMDQ5MDkxIzAhBgNVBCoMGtCh0KLQkNCd0JjQodCb0JDQktCe0JLQndCQMR8wHQYJKoZIhvcNAQkBFhBDRU9ATEVBREVSRU5HLktaMGwwJQYJKoMOAwoBAQEBMBgGCiqDDgMKAQEBAQEGCiqDDgMKAQMBAQADQwAEQOTfqVEh24kddh/aZlKhpjjqrTqnPx9OQic

In [55]:
r.text

'{"documentSizeBefore":71,"signerCountBefore":0,"cms":"MIII8AYJKoZIhvcNAQcCoIII4TCCCN0CAQExDzANBglghkgBZQMEAgEFADBWBgkqhkiG9w0BBwGgSQRHPD94bWwgdmVyc2lvbj0iMS4wIiBlbmNvZGluZz0idXRmLTgiPz48cm9vdD48bmFtZT5OQ0FOb2RlPC9uYW1lPjwvcm9vdD6gggZpMIIGZTCCBE2gAwIBAgIUFX1cSXpU\\/SdXs4r74PS8YFuVbAowDQYJKoZIhvcNAQELBQAwUjELMAkGA1UEBhMCS1oxQzBBBgNVBAMMOtKw0JvQotCi0KvSmiDQmtCj05jQm9CQ0J3QlNCr0KDQo9Co0Ksg0J7QoNCi0JDQm9Cr0pogKFJTQSkwHhcNMTgwODIyMTIxMTM2WhcNMTkwODIyMTIxMTM2WjCBpzEeMBwGA1UEAwwV0KLQldCh0KLQntCSINCi0JXQodCiMRUwEwYDVQQEDAzQotCV0KHQotCe0JIxGDAWBgNVBAUTD0lJTjEyMzQ1Njc4OTAxMTELMAkGA1UEBhMCS1oxFTATBgNVBAcMDNCQ0JvQnNCQ0KLQqzEVMBMGA1UECAwM0JDQm9Cc0JDQotCrMRkwFwYDVQQqDBDQotCV0KHQotCe0JLQmNCnMIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIBCgKCAQEAtKWLOJf9qCqA6EO\\/SVtiMuPZ8q3Sg2RjO0dWXqKQRP7BWhIyMucMv+WmpRs8RuJ987Hm3B\\/JszSdiPrmtA9BpIERKphRwp3n4QR6pfLUBEp+5QNetNsv+dbiPcefWCzgJZCqEZVbPvSkiFH20y13YQ2FhEBUp4lLOqydBD2CsDVoTusvLanEgR+AdziJPq2+iXwhttpNPShKRTXGbGkxUa4P7YMUCUqWstR7svLaJqxKDMhaR7MpEt56a2pfntm5oFxKNFoBQjR

In [71]:
import json

config = {
    'cert_path': '/home/necuk/Downloads/key2/GOSTKNCA_de113b5aa34a4d38ee8836c3e2ae2e3c8e23d06c.p12',
    'cert_password': 'baRsa01022020',
    'address': 'КАЗАХСТАН, 711110000, 010000, г.Нур-Султан, район Алматы, РАКЫМЖАН КОШКАРБАЕВ, 26, 1В',
    'iik': 'Банк: Акционерное общество "Банк Фридом Финанс Казахстан" ИИК: KZ28551A127014873KZT БИК: KSNVKZKA КБе: 15',
    'price_discount': 0.1,
    'file_1': 'file:///home/necuk/Desktop/zakup_kz/temp/file.txt',
    'file_2': 'file:///home/necuk/Desktop/zakup_kz/temp/file.txt',
    'file_15': 'file:///home/necuk/Desktop/zakup_kz/temp/file.txt',
    'file_19': 'file:///home/necuk/Desktop/zakup_kz/temp/file.txt',
    'file_18': 'file:///home/necuk/Desktop/zakup_kz/temp/file.txt',
    'file_11': 'file:///home/necuk/Desktop/zakup_kz/temp/file.txt',
    'file_license_1': 'file:///home/necuk/Desktop/zakup_kz/temp/file.txt',
    'file_license_2': 'file:///home/necuk/Desktop/zakup_kz/temp/file.txt',
    'file_sertificates': 'file:///home/necuk/Desktop/zakup_kz/temp/file.txt',
    'file_NDS_register': 'file:///home/necuk/Desktop/zakup_kz/temp/file.txt',
}

with open('../scrapy_parser/config.json' , 'w') as f:
    f.write(json.dumps(config))

In [70]:
json.load(open('../scrapy_parser/config.json'))

{'cert_path': '/home/necuk/Downloads/key2/GOSTKNCA_de113b5aa34a4d38ee8836c3e2ae2e3c8e23d06c.p12',
 'cert_password': 'baRsa01022020',
 'price_discount': 0.1,
 'file_1': 'file:///home/necuk/Desktop/zakup_kz/temp/file.txt',
 'file_2': 'file:///home/necuk/Desktop/zakup_kz/temp/file.txt',
 'file_15': 'file:///home/necuk/Desktop/zakup_kz/temp/file.txt',
 'file_19': 'file:///home/necuk/Desktop/zakup_kz/temp/file.txt',
 'file_18': 'file:///home/necuk/Desktop/zakup_kz/temp/file.txt',
 'file_11': 'file:///home/necuk/Desktop/zakup_kz/temp/file.txt',
 'file_license_1': 'file:///home/necuk/Desktop/zakup_kz/temp/file.txt',
 'file_license_2': 'file:///home/necuk/Desktop/zakup_kz/temp/file.txt',
 'file_sertificates': 'file:///home/necuk/Desktop/zakup_kz/temp/file.txt',
 'file_NDS_register': 'file:///home/necuk/Desktop/zakup_kz/temp/file.txt'}